# In Depth: Principal Component Analysis

Up until now, we have been looking in depth at supervised learning estimators: those estimators that predict labels based on labeled training data. Here we begin looking at several unsupervised estimators, which can highlight interesting aspects of the data without reference to any known labels.

In this section, we explore what is perhaps one of the most broadly used of unsupervised algorithms, principal component analysis (PCA). PCA is fundamentally a dimensionality reduction algorithm, but it can also be useful as a tool for visualization, for noise filtering, for feature extraction and engineering, and much more. After a brief conceptual discussion of the PCA algorithm, we will see a couple examples of these further applications. We begin with the standard imports:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Introducing Principal Component Analysis

Principal component analysis is a fast and flexible unsupervised method for dimensionality reduction in data, which we saw briefly in “Introducing Scikit-Learn” on page 343. Its behavior is easiest to visualize by looking at a two-dimensional dataset. Consider the following 200 points (Figure 5-80):

In [ ]:
%matplotlib notebook
rng = np.random.RandomState(1)
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal');

By eye, it is clear that there is a nearly linear relationship between the x and y variables. This is reminiscent of the linear regression data we explored in “In Depth: Linear Regression” on page 390, but the problem setting here is slightly different: rather than attempting to predict the y values from the x values, the unsupervised learning problem attempts to learn about the relationship between the x and y values.

> **Figure 5-80: _Data for demonstration of PCA_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAeoAAAFRCAIAAAA0AnnbAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3de1RU5/3v8e8gAspAZvAS5TQdDKvklxyNXdqsVdomRA1GjPEKCZigjR7aJLXLahqspsvmcgh2mqRpT8V6rdb2qKnRYKxkJUbK6vGnkUV+Y6tJNPEyvyTjBYQBBpFxZJ8/NozbYWYYYBiY4f36Czczw372jJ/9zHc/z7N1iqIIACDcRHEIAID4BgAQ3wAA4hsAiG8AAPENAOiW6ND8GZfrRl3d1Ug9iEbj0AhuXcQ3kLeP1vVbI0Yk9H3vOzp6UCSfAyO6dbx9NJDW9U8UTwCA+AYAEN8AAOIbAIhvAADxDQAgvgGA+AYAEN8AAOIbAEB8AwDxDQAgvgEAxDcAEN8AAOIbAEB8A8AAE+jN0lwu16pVq77++uvr168//fTTkydPVrcfOnSopKQkOjp63rx5OTk5HFAA6F/xvW/fPqPRaDab6+vrZ8+erca3y+Vas2bNnj17YmNj8/LypkyZkpSUxDEFgBAItHiSlZW1dOlSEWltbY2Obgv9M2fOmEwmvV4/ePDgiRMnVlZWckABoH/1vocMGSIiDodj6dKly5YtUzc6HI6EhLYbIcfHxzc2NnJAAaB/xbeIXLhwYcmSJU8++eT06dPVLXq93uFwqD83NTUlJib6ebr/O96Hu8huHW8fDaR1YRzfNTU1ixcvXr169Xe/+133xtTUVKvV2tDQEBcXV1lZuXjxYj+vUF3dGMGfnghuXcQ3kLeP1oXpmSnQ+F6/fn1DQ0NJScnatWt1Ot1jjz3W3Nyck5OzcuXKRYsWKYqSk5MzcuRIvs4AQGjoFEUJzV/i/E8DaR0NpHVB7H0zbQcAwhLxDQDENwCA+AYAEN8AQHwDAIhvAADxDQDENwCA+AYAEN8AAOIbAIhvAADxDQAgvgGA+AYAEN8AAOIbAIhvAADxDQAgvgEAxDcAEN8AAOIbAEB8AwDxDQAgvgEAxDcAEN8AgEiM7+PHj+fn52u3bN26dcaMGQsWLFiwYMH58+c5oAAQGtGBP3TTpk2lpaXx8fHajSdPnjSbzffccw+HEgD6ae/bZDKtXbvWY+PJkyfXr18/f/78DRs2cDQBoD/Gd2Zm5qBBgzw2PvLIIy+99NKf//znqqqqiooKDigAhEZ0D5+/cOFCvV4vIhkZGZ988klGRoavR44YkRDBxzGyW8fbRwNpXSTEt6Io7p8dDseMGTPKysri4uKOHj2anZ3t54nV1Y0R/OmJ4NZFfAN5+2hdmJ6ZuhzfOp1ORPbv39/c3JyTk7N8+fL8/PzY2Nj09PQHHniArzMAEBo6bW+6V3H+p4G0jgbSuiD2vpm2AwBhifgGAOIbAEB8AwCIbwAgvgEAxDcAgPgGAOIbAEB8AwCIbwAA8Q0AxDcAgPgGABDfAEB8AwCIbwAA8Q0AxDcAgPgGABDfAADiGwCIbwAA8Q0AIL4BgPgGABDfAADiGwCIbwBAJMb38ePH8/PztVsOHTqUnZ2dm5v7t7/9jaMJACETHfhDN23aVFpaGh8f797icrnWrFmzZ8+e2NjYvLy8KVOmJCUlcUwBoH/1vk0m09q1a7Vbzpw5YzKZ9Hr94MGDJ06cWFlZyQEFgH4X35mZmYMGDdJucTgcCQkJ6s/x8fGNjY0cUADod/HdkV6vdzgc6s9NTU2JiYkcUAAIjeiuPkFRFPfPqampVqu1oaEhLi6usrJy8eLFfp44YkRCBB/HyG4dbx8NpHWREN86nU5E9u/f39zcnJOTs3LlykWLFimKkpOTM3LkSD9PrK5ujOBPTwS3LuIbyNtH68L0zKTT9qZ7FR8gGkjraCCtC2J8M20HAMJSNIcAwABXW2tfsaLcak00merN5slGo4H4BoAwsGJFeWlpvojOYlFEtm/cOCcsdpviCYCBzmpNFNGJiIjOag2bAdDEN4CBzmSqF1EHcSgmU0O47DbFEwADndk8WWS71ZpoMjWYzZOIbwAID0ajIVzq3VoUTwAgLBHfAEB8AwCIbwAA8Q0AxDcAgPgGAHQP474B9FSYLvlEfAMY6MJ0yadwR/EEQE+F6ZJPxDeAgS5Ml3wKdxRPAPRUmC75RHwDGOjCdMmncEfxBADofQNAL/MYpOhxL/YBNYSR+AYQaFb2hzT0GKT4zjsL/Pw2sks6xDeAQLMyNGno/5zhf5DigBrCSHwD8KlP0tD/OcNkqrdYFBGd10GK/n9LfAPo1/xXh7v09GHDroq0peHo0TX94Zzhf5DigBrCSHwDEV7x8KgOd+npt9++WmSHSIJIo8j10Oy//x60/0GKA2oII/ENUPHw+fSGhm+JzFe3X7jwTmj2n0lAxDcwQPWw/qt9utH4VXOz+nPdxYv/TkurFalJT9e/+eajvTcKhUlAxDcw4Khl67Nn45OTX01KSklNvdGN3qu287tq1cxXX91utSZevvyJzfZLNdPLynbExJT/+teTWCQ2POJbUZQXX3zx1KlTMTExRUVFd9xxh7p969atu3fvTkpKEpGXX345JSWFYwr0FXfZWkS5775ujvPz6Pxu3GgSkalTxWZrq6iIJFitN1gkNmzi++DBg06nc+fOncePHy8uLi4pKVG3nzx50mw233PPPRxKoM/13jg/bUVFpNFkcrFIbJ8LdM2Tqqqq+++/X0TGjx9/4sQJ9/aTJ0+uX79+/vz5GzZs4GgCfSsoC7fW1toLCvZOnfphQcGeujq7utFsnpyVtdlg2GIwvJaVVW82T2KR2LDpfTscjoSEttGj0dHRra2tUVFRIvLII4888cQTer3+Jz/5SUVFRUZGBscUCIpuTFgPypgNr1URo9GwbdvjQf9bCEV86/X6pqYm9Wd3dovIwoUL9Xq9iGRkZHzyySd+4rurcwfCS2S3jrevTyxZst8do7GxO3ftyvP1yCtX7M8+W3bunH7MmMYtW2YmJRl60kCbzeiuithsRl/PHTEiwWNEuXY31q2b3nE3+HD2TXxPmDChvLx82rRpFoslLS3N3SWfMWNGWVlZXFzc0aNHs7Oz/bxCdXVjBP/nj+DWRXwD+23rTp8e4o7R06eH+NnJgoJ9atBXViotLZ5XEbvawOTkWvdMy+TkusCf6383+HAG/cwUaHxnZmYePnw4NzdXRIqLi/fv39/c3JyTk7N8+fL8/PzY2Nj09PQHHniArzNAsAQ+fDu4VxH9V0X8lHS4mNlPe986ne6ll15y/3PMmDHqDzNnzpw5cybHEQi6wIvL/oP+yhV7QcG+wGvoHWfNaCPb6bxeVrbY63jBAbVcVDjFN4AQC3zyof+gf/bZsh4O0G6/mFlvsRyIirrhq4vNxUziG0Awg/7cOX0PaxrtVZEykbzW1h3uyrhHF5vJ7sQ3gC7odHzhmDGNlZU9qmm0V0XU08B0kZ1DhlyfOlXoYhPfALqv08nr69ZNb2kJtKbh9WSgVkUqKi7a7TNEDCK5U6cyS574BtCzHvfZs/G+aiPqw2w2Y3KysmvXxEAm/vias7Nx45y6OnthIaVt4htAkHrcycmv+ipGa1ew8n/R0n0+OH/+oki9iKHjyYDSNvENRE7PtycLpfp6nUBeXzvCOikp5b77vHeKAx+IfWvQ7xCZz+A/4huI5J5vTxZK9fU6gby+doR1auoNX/vQvYk/BsO1lJR3qJAQ30CkCdbcQs3r1FdUXJo69UO1ux3I6wc4wlp9mM1mTE6uC3ziT0ZG9MaNU3ijiW8g0gRrbqHmdQ7Y7T+3WNq62yaT0unrB1iGVh8WyKogZvPklpbNR49GiVxxOuPr6uzcPYf4BoIpWHXnnlC7tGfODKqttZ49m1ZQsKd7e+LuQZ8/f81uv9nd3rVrYqc969pa+7Jlfz9ypFFkWHr6jTffnNbDQ2E0GmJjY+z2fBFdWZkSE8O4QOIbCKo+uUFXx3PGxo1zCgr2njix0mbT/fvf3dwTdw+6oGBPaenN7nYgPesVK8rLyhJFfuw/bTUDB2s7Pcew4BTxDfSi4EaMNt1WrpxYXPyx2qceNiztzjub3Hnn9ZwRxD3pxlIhVmuiyCDtDnj9XhL4wEEJ1YJT/eH7E/EN9IHuRYyvyNCmW2Vlsc22UmSniGef2mtSByvsuhRn7gdfvnxSJEU73Lvn55geLjgVYEO4wTHxjQGqexHjKzK06VZX9w0RXfuyHp0ndbBW19PuW0XFaxkZIwPJPpGM229/o6XlNZFh6emtZvPDjz9e1cNzTA9n5QSYy5RoiG8MUN2LGF+RoU03o/HL5mZFpLHjDEavSe1nTzz6oYoifrql2n2z2/+jtHRGYNlnHD164vvvT/H/vSTwgYMhq2uxJjjxDQSh5KJNt1WrZr36qjqepHjYsLQ777yqTepf/3qSGsE/+9l+kcEXLgz3UyLw6Ie2tDjfe6/t/gZO52aPG/5q903E0e3s83OOCc3txALMZdYEDyKdoiih+UvcLJEGBktXL3/V1dkLC8vdkeHx+EBaV1Cwt71q8X9F8tSQmjXrZjfZvUujRlX/53/qGhqyRcpE9AbDp62tQxsalqgPMxi2HD2aqd15ESksLK+ocNntcSLTRW7TvmyXGtK3b1/39o3/fZ22jt43IkpXL3/1fK0lTWUgwaNEoAZ3RYXLbo8VeVDkNpE1IgfUlLfbZwwe/Ct3QUbkSsedb1/Mr9xq/Ye2T9rxLKU2RN3++ONVoRy80ekpkwWtKJ4AXQrTEF3+0lQGGjyK47cOztspkieSItLi3sMhQ755/foOkQSRxvR0vded95p9HkHvLuBcvnzSZntWxBjKwRuMGCG+geCGaYguf5nNk53O9UeONLa2GocO/d/Dh9+ZmnpD7SZr41hEL6KIKCIx7pT//vclJsZltd4wmVxm84zCwkPdu3+8+4aTIldFSkViRKa//750e9pnPz9lgvhGBAr95S+j0RATM9Ru/7GIrqGhLipqXXT0PYWFh8zmydpzicHwWXr6JZHrX36ZWFtbrNen/Pd/f3roUPKwYZf27p01ZoxJenD/eO0NJ90ruzY3Dy4tzQ1Bd5gRI8Q3EJwwDdmXd2/3MXjPZltps7WVEW6N41xtL/jb3/4/1669LKKz2ZQ5c4otlp9KD+4f395t12s6+06R2aHpDjNihPgGwkaHa5Kz2u9jcMvNyfzEcftUIGmfFtSjs9StN5zUtZdoDKHpDnNlkvgGfHZv+9siGN6uSToNhi1Dh16w2R7tWEbwNlBEnQqkTgv6qoet9rjh5OjRNSLXL1zgvgrEN9DXKdl7Qxq6d4bwdk0yNiVlyK5dmV7v2NuxIXv3zpozp7iu7htG41d7984MvNXaHVbX0tLufPsQlOEmU32ANyAOl1MmiG+EkxAMafCale4IS0u7+sor93eMsFvnQ54SqRVxnj/frF6x7Pj4jg0ZM8ak1ru72mrtDrevpXVz53t+wmMUIPENBEEIhjR4zUpthLW03BxV7e6Qui/WjR5dIxJ/5Mhlu/3ndruutLQt8jz6sF1qiP8He1tL6+bO9/yExyhA4hsIghAMafCalT5GVd/skHpcrJs69UOL5ZbI0z7l739/1Wh0JCauiYq6XV0CsCet9raW1s2d7/kJj1GAAyu+FUV58cUXT506FRMTU1RUdMcdd6jbDx06VFJSEh0dPW/evJycHA4ouipYQxr81HO9ZqWPUdW3dEi1rzl6tNMj8rRPcbnura6eIbJTJDcmZruiSEHBXl/F5U5Lz9odVtfS0u58z094jAIcWPF98OBBp9O5c+fO48ePFxcXl5SUiIjL5VqzZs2ePXtiY2Pz8vKmTJmSlJTEMUWIeYzws1hu86jnej1DuCMsLa35lVcmdZwMWVtrnzx5u82WIvKpxXLXqFEnsrL+eOHCaHfkdVwsUB2U7bUvr/3TnZaePXZ440ZTcE94jAIcWPFdVVV1//33i8j48eNPnDihbjxz5ozJZNLr9SIyceLEysrKhx9+mGOKkEW21Zo4evSF//qvKxcvvqAd4RdIPdcdYeqSdR1vtb5iRXn7/Xd+IaK7eHFOTMx27RLbZvPkyspim220yA2RwSL7RE6K/MBXX96N0jOCIirAxzkcjoSEtqULo6OjW1tbPTbGx8c3NjZyQOERsgUFe6dO/bCgYE9dnT2Ir6x2YC2W2WVlt128eK/HCL+u1nNra+2FheVHj9bZ7U/Z7c+XlT2jrn3q6/477hNAeXl+Vta1QYM+FxksohP55u23v2E2TzKZ6kXUpZi97Iz/3wJB7n3r9fqmpib159bW1qioKHWjw+FQNzY1NSUm+utH+F+4NtxFduu63cAlS/a7qwSxsTt37coL1v7YbEbNCq43b5FjNJ7KzHSsWzczKakLO7x69f8rLc0X2a9J6oSamk9Frop8ItI2yzElpd7jOIwYkXDgwDNJSa/V1bWtQ+J0vp6WdseWLbOeeWbnuXP6MWO87Iz/3/L5pHVBju8JEyaUl5dPmzbNYrGkpaWpG1NTU61Wa0NDQ1xcXGVl5eLFi/28ArdrGIANPH16iDsQT58e0sOjpL3iN3y4sz2yG0Smi+wUiU9OPlFenm80Gm7cuOXz5v9S4YgRCe37efM0cPnyp+2Vk/8l8muRZJHTjY1Gr01QlCR3MxUlqbq6UWTQH/4wQ/2tx86ISGe/5fNJ6wI6MwUa35mZmYcPH87NzRWR4uLi/fv3Nzc35+TkrFy5ctGiRYqi5OTkjBw5kq8z8KgSBDhAzX/Cdrw4mZX1x1mz1EHZDSJvX7gw3GSym835RqOh40u5l1q1WA5UVHyQkTFIO5Vxy5ZZ7fs5XWSHwXAtIyP6zBmTzVYv8rXIbpEVahOqql7zututrdXu3I+L+3rq1A+ZzYh+1PvW6XQvvfSS+59jxoxRf3jwwQcffPBBjiO80sx8ueh0RvvJtQCHargvTn75ZWJq6g0RiYkZ6vGCHV9Ku9SqOu9GO5XxmWd2rlw5sbKyuK7uG4mJ1rFjh1qtqbW1J0UOiDynraiIDPOx2/Vq7g8detFmW3LxopHZjOhH8Q10g3t0h/tekeoscLXEoX1k4EM11IuTtbXWEydWeo37ji/V3rnWa6YyJrt/PndOX1z8sZrmzc3KpUs7RGaLZERHv+Vy3VJRSU9v9bFjBpH5KSnviCTZbEZfrQCCK4pDgBDQRqrNNrawsNxjUErgQzUMhs9mzdo+bFiar7jXPvjy5U/q6uxm8+RZs7YPHnzcvV1RvnD/PGaM49bTg1ptNOr16uuoFZUts2Ztf/PNh/3smMnU0PFP89aD3jfC260zXJo6TmzxPw/QXdxQl+4bM8ZUULDn3//2XlVvH449VqTJZntm6dK3Y2NjrNbEIUMM16/vFNGLOFJSvnnXXW1/bt26mYsWlWp2T70OpqSnJ8TEqI9xmc2ZXgvZHXdb+6cLC9+lfgLiG+FNG6ki00ymdz1KHH7mAdbW2ufO3Wez/YfIqebmlDlzSsvLF/iJe6PRMHLkPTZb2wKtR49G2e1q3fyvIrlqRt911816S1JSwq2rU7kX0Z7R6bXHjrut/dPUT0DxBGFPUWT8eKPBcMpgaM7KervTiS1a7bMf56qzH222VYWF5Ypy87W9dvbdLy5ypf088YjB8Nq3v/3OrFnbPRJfUaSlxXn+fO2RI7Uiyq5dE9UlqzqeSDqdhRR4u3pvThPofQNBs2JFeVnZ02rPNyZmu3Y5Vj+rJqnD8t5/X269btn5oiLaF3c648vK1MLIbRkZt2/cOMXr7r333mJ198rKdsTElHv9KhDIMtkd2+VrTCSLboP4RhjQlkrOnBnUvhqf4udmMbW19oyMjZcufVPkv0X2iThEskQcfhYI9FrTqKuzt5ewfZ4nPC5dWq03On2Yr8JIx3KKr5hm5RMQ3wgD2kuXNTXnTpx4QY2zjz4qGjVqrNfB4CtWlF+6lCgyVOSl9ouKq0XuTk4uNpvzOy4Q6Esgq+vdemW10WRydfqwwNcq8RXTLLoN4hv9mlo6OHs2Pjn51aSklNTUG+Xlo9xxdvHivRcvzvRaPbBaE0VaRRLcD46K+sajj7rUqZXdW7HaVx3DbJ7sdG4+ciRK5Ep6ut5snuH16d37o75imkW3QXwjdCnsdVJ7x19ptzid18vK2srK9923fePGOWlp69wTYUSafFUPTKZ6i0VEGtwPHjSoxmr9n+5bTQbxHo9Go2Hbtsc7fXr3/qivmGbRbRDf6N107hh8LS2b1ZHU6oM7ZqJ2i8HwJ4/SwcSJgz78cI3I/xD5XERd5sxL9cBsnux07n///U9v3LggMlxEd/36SItltsWiOJ3rY2KGduNG6VZroki9SJmIvqLiYl2dPZDn9vC+7MQ0iG/0ok5HQWgLuO6R1OqDO9Z2b70SWOPuPqsZHRNzm0hbf3zUKLX27aV6YDQatm17sq7Ori69ff7853Z7gfqaR4402u0/7saYDZOp3mI5IJInorPbZxQWBvRchoiA+Eb/1ekoCG0Bt7X1kog6fbHxzJlBqametV3tg9PTE0TWHznS2No67KOPvpo8ed9XX9WK1IsYRHSjRo3V3r/GT9e1oKC+tPQ2tasuMqx7YzbM5skVFR/Y7V17LkNEQHwjdLWOrup0FIS2gHv06NWGhrbpi7W1xW+/ne9R27212jujsLBc7Sw3NCgXL+4UeV5kh8h0kQPnz18rKNjja4XYM2eG1taeSkpKSU11rVo1UTOU+0b7UO6ujdkwGg0ZGYNKS7v2XO3BGTWqxs89iAHiG6H+Ot/pKAjtLMekpLGXLrX1RocNS+tY2/XY0mG9QJ3BcE1ko93+c3X5VncT3Kely5c/UZcAFFFstp0nTsysrFw3cuQ9amiKSKdDubvdUv9PcTqvl5YuppAC4hu9Vevoqk4vr61YUV5a+qjIexaLMS7umMhjarbeeefVLvVe1Xk3GRnRVuu3LBa1CfUVFZfUpcCdzqvtkzOjbk3892y2lTbbzdDsdm5240Ki9ilTp35IIQXEN3qx1hH0kovVmijynrrk07Vr9ycnF48ceU+AHVi193r27NArV04nJZlSU7ebzZM0824O2O0/t1jUYSqvtYejQzO40CES309Ck7k2IL7Ru7WOoJdcTKZ6i8V9d2DjyJH3+L/k6KP3+nDHJpw/f819LdHhSGxP7e/Fxa3W6VIV5YuUlG82NJy12R7tD6HJXBsQ3+jdWkfQSy5m8+TKyj8HMUM1Q0r2uK8lulxx6k2H4+Iqr1172b24q9m8oLCwX4Qmg7hBfCPMvvgbjYby8kAz1P88TI+6jaYbro7sNoiITmcPcDVwgPgGJZegdTz9z8PU1m20sT56dHxZWdvIbqPxy+ZmSswA8T3weO3thqwP638eprZuo431rKzN06ZtPno0SuTK3XfHjh+/+cKF4ZSYAeJ7YOnted7+B7F0rNL4qttoY10Na3U6/ocfKrNmbX///Sm1tW2T5pkmAxDfA0JXr1J2dUxh4De7MZsn1dbanc7rBsOfRGrS0xO0y7F6xHrH3Wa9EYD47neljF7V1auUXU3JwG92IyIFBXvda8aqd0qTm9Pio5OTi4cNS7vzzqu3DgZv223WGwGI70guZXTU1auUXU3JLp0evL64+5i41wH3utt+/lDoT4oA8U0po9cFcpXy1lEfzi711rt0evAawT7ufulzWGHHP0RdBcQ3el3/nGl966iPP86a1YXeepcGsXiNYM0xqfvii89OnHjZaxD7+UPUVUB8o9f1z5nWt476GB34JPigfBUwmyc7neuPHGlsanJeu3ZfN4KY5UdAfAekpaXl+eefv3Llil6vX7NmjdFodP+qqKjo448/jo+PF5GSkhK9Xs9h7Ulftf9/JwhK0dloNMTEDLXbfyyyX6TR44484XtSBPpdfO/YsSMtLW3JkiUHDhwoKSl54YUX3L86efLk5s2bDQYuHA2U7wTBKjq3d/8bRbLUpU6Sk0+YzflhfVIE+l18V1VVFRQUiMgDDzxQUlLi3q4oitVqXb16dXV1dXZ29rx58zimEf+dIFhF5/bu/3SRAwbDtYyM62ZzvnZYIaNKgO7E9+7du7dt2+b+5/Dhw9WqSHx8vMPhcG+/evVqfn7+U0895XK5FixYMG7cuLS0NA5rZAtW0VnT/XeZzZmKIu4Jlu77NjCqBPBFp2juguXHT3/60x/96Efjxo1zOBx5eXnvvvuuur21tbW5uVktfP/mN7+56667Zs6cyWGNJFeu2J99tuzcOf2YMY3r1k1PSjLU1tqfeUbd4li3LispKThd48cf3/HWW2230DQaX6+r+7m6/b773j127FHeCKCbxZMJEyZUVFSMGzeuoqLiO9/5jnv7uXPnli1bVlpa6nK5qqqq5s6d6+sVqqsb+//h6N539hEjEsKidd1TULBPrXRXViotLWpHeNAf/tA28f3GjaC9s6dPD3HXZBQlyX0xMzm5rlcPb2S/fRHfwIhvXRDiOy8vb8WKFfPnz4+JiXn99ddFZOvWrSaTadKkSbNnz87JyRk8ePCcOXNSU1PD+mAxE6SjkA2v1tZk0tNbu30zYoDe9y3i4uJ+97vfabf88Ic/VH9YtGjRokWLiKpw5+ubR8iGV986EuZhLlcCwYnvAWIgzwTx9c3DbJ4cG7vz9Okhvd0RZiAgQHwHqwPY06jq89FvXdoBX988jEbDrl15kV0dBojvsBfcDmCfV9K7tAPMQQeIb3TSn+2THaiocE2d+qGfbjhz0AHiG/2lP6vdAbs9zmKZ7acbTukZIL7RW/3ZrhbT3Ttw/vzndntBH34PAEB8h5Og92e7Wkx370BBQX1p6W0iQl0bIL7RB7pdTKeuDRDf6EvdLqZT1waIb/QlOtEAiO+wRCcagFsUhwAAiG8AQIhQPAlL3EsMAPEdlliXHADFk7A0kNclB0B8hzGTqV5EvUkpsyiBAYriSVhiADgA4jssMQAcAMUTAKD33T+E1y3KAID4bhNetygDgO6JwOJJv7pFGaP6ABDfgerzQXWM6gMQAhFYPOnzQXWM6gMQAsz5cgsAAAZPSURBVDpFUULzl6qrGyP1II4YkRDBrYv4BvL20br+3Do/v2XgIACEJeIbAIhvAED/jO8PPvjgueee89j41ltvzZs3Lzc39x//+AcHFABCowsjT4qKig4fPnz33XdrN9bU1Gzfvn3v3r3Xrl3Ly8v7/ve/P3jwYA4rAPSj3veECRNefPFFj43/+te/Jk6cGB0drdfrU1JSTp06xTEFgL7sfe/evXvbtm3ufxYXF2dlZR07dszjYQ6HIyGhbWjL0KFDGxsbOaYA0JfxnZ2dnZ2d3enz9Xq9w+FQf25qakpM9DlH3P8AxnAX2a3j7aOBtC6c4jtA995775tvvul0OltaWs6ePfutb33L1yOZOEADaR0NpHVBPDN1P763bt1qMpkmTZqUn58/f/58RVGWL18eExPDNxoACAEmzXP+p4PD20frwrL3zbQdAAhLxDcAEN8AAOIbAEB8AwDxDQAgvgEAxDcAEN8AAOIbAEB8AwCIbwAgvgEAxDcAgPgGAOIbAEB8AwCIbwAgvgEAxDcAgPgGABDfAEB8AwCIbwAA8Q0AxDcAgPgGABDfAEB8AwDCQ3SXHv3BBx+89957r7/+unZjUVHRxx9/HB8fLyIlJSV6vZ7DCgD9KL6LiooOHz589913e2w/efLk5s2bDQYDRxMAQqYLxZMJEya8+OKLHhsVRbFaratXr87Ly3v77bc5oADQx73v3bt3b9u2zf3P4uLirKysY8eOeTzs6tWr+fn5Tz31lMvlWrBgwbhx49LS0jisANBn8Z2dnZ2dnd3p84cMGZKfnx8bGxsbG/vd7373s88+I74BoC/jO0Dnzp1btmxZaWmpy+WqqqqaO3eur0eOGJEQwccxslvH20cDaV1ExffWrVtNJtOkSZNmz56dk5MzePDgOXPmpKam+np8dXVjBH96Irh1Ed9A3j5aF6ZnJp2iKKHZDz5ANJDW0UBaF8T4ZtoOAIQl4hsAiG8AAPENACC+AYD4BgAQ3wAA4hsAiG8AAPENACC+AQDENwAQ3wAA4hsAQHwDAPENACC+AQDENwAQ3wAA4hsA0CtCd6tiAAC9bwAgvgEAxDcAgPgGABDfAEB8AwAGSnw3Nzc/++yzTz755KJFiy5fvhxhB9HhcDz99NP5+fm5ubkWiyVSPysffPDBc889F0ktUhTlV7/6VW5u7oIFC7788suIfNeOHz+en58fkU1zuVyFhYVPPPHEY489dujQoQhrXWtr66pVq/Ly8p544okvvviiL+P7rbfeGjt27F/+8pdHH31048aNEXag//SnP33ve9/bvn17cXHxyy+/HJH/VYqKin77299GWKMOHjzodDp37tz53HPPFRcXR967tmnTpl/+8pfXr1+PyM/kvn37jEbjX//6140bN77yyisR1rpDhw7pdLodO3YsXbr0jTfe8PqY6NDsysKFC9X5QTab7bbbbouwA/3UU0/FxMSo3YHY2NiI/K8yYcKEzMzMXbt2RVKjqqqq7r//fhEZP378iRMnIu9dM5lMa9euLSwsjMjPZFZW1rRp09SOanR0dIS17qGHHpo8ebKIfP31174ys7favHv37m3btrn/WVxcPHbs2IULF37++edbtmwJ9yPrtXXV1dWFhYUvvPBCBHx0OjYwKyvr2LFjEfY/xOFwJCQktP1PiI5ubW2Nioqoq0GZmZlff/21RKghQ4aob+LSpUuXLVsWeQ2Mior6xS9+cfDgwd///vfeH6GE1pkzZx566CEl4nz22WczZsz45z//qUSujz76aPny5ZHUouLi4rKyMvXnjIyMiHzXvvrqq8cffzxSP5M2m23u3Ll79uyJ4P93NTU1kyZNam5u7virEPU1NmzYUFpaKiJDhw4dNGhQhJ0kv/jii5/97GevvfbaD37wA0FYVYQqKipExGKxpKWlRWozI3Vdo5qamsWLFz///PNz5syJvNaVlpZu2LBBRGJjY6Oiorx+LwxRwWjevHkrVqzYvXu32uWJsAP9xhtvOJ3OoqIiRVESExPXrl1LMoZLbeHw4cO5ublqgShSm6nT6SKyXevXr29oaCgpKVm7dq1Op9u0aZN6CSoyTJ06deXKlU8++aTL5XrhhRe8No0VBwEgLDFtBwCIbwAA8Q0AIL4BgPgGABDfAADiGwCIbwBA//b/AdWBYveMeAjJAAAAAElFTkSuQmCC" />

In principal component analysis, one quantifies this relationship by finding a list of the principal axes in the data, and using those axes to describe the dataset. Using Scikit-Learn’s PCA estimator, we can compute this as follows:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

Output:<br>`
PCA(copy=True, n_components=2, whiten=False)
    `

The fit learns some quantities from the data, most importantly the “components” and “explained variance”:

In [ ]:
print(pca.components_)

Output:<br>`
[[ 0.94446029  0.32862557]
 [ 0.32862557 -0.94446029]]
    `

In [ ]:
print(pca.explained_variance_)

Output:<br>`
[ 0.75871884  0.01838551]
    `

To see what these numbers mean, let’s visualize them as vectors over the input data, using the “components” to define the direction of the vector, and the “explained variance” to define the squared-length of the vector (Figure 5-81):

In [ ]:
%matplotlib notebook
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)
# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');

> **Figure 5-81: _Visualization of the principal axes in the data_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAeoAAAFRCAIAAAA0AnnbAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3de3RTZb438GfvnSY7zaVN79BLCoEAIqBFRxCh1FIugkpKqqAUXmQ5a9ZcXkc9o4fxLMczZ3HKYo0z58x76Fk6OsI4XiktxSIqiDDaUYEqoIU2kNKWtrRN27Rp0lx39vvHLjH2kt7StA3fzz/GTS772Um/+8kvv703xfM8AQCAqYbGJgAAQHwDAADiGwAAEN8AAIhvAABAfAMAwKiIQvMyHg9nNveE60ZUqSLDeHRhP0C8fRjdpBUfr5j42bdIxITzPjCsR4e3DwPE6CYnFE8AABDfAACA+AYAAMQ3AADiGwAAEN8AAID4BgBAfAMAAOIbAAAQ3wAAgPgGAEB8AwAA4hsAABDfAACIbwAAQHwDAADiGwDgFjPci6V5PJ7f/va3jY2Nbrf7Zz/72f333y8sP3nyZGFhoUgk2rRpU15eHjYoAMDkiu8jR46oVKq9e/d2dXVt3LhRiG+Px7Nnz57i4mKJRLJly5bs7OyYmBhsUwCAEBhu8WTdunVPPfUUIcTr9YpEvaFvNBrVarVcLo+IiFi8ePHZs2exQQEAJtfsWyqVEkKsVutTTz319NNPCwutVqtC0XshZJlM1t3djQ0KADC54psQcuPGjV/+8pdbt2594IEHhCVyudxqtQq3bTabUqkM8PDAV7yf6sJ7dHj7MECMbgrHd1tb286dO1988cUlS5b4Fmo0mrq6OovFwrLs2bNnd+7cGeAZTKbuMP70hPHown6AePswuim6ZxpufL/yyisWi6WwsHDfvn0URT3yyCN2uz0vL2/Xrl1PPPEEz/N5eXkJCQn4OgMAEBoUz/OheSXs/zFAjA4DxOiCOPvGYTsAAFMS4hsAAPENAACIbwAAQHwDACC+AQAA8Q0AAIhvAADENwAAIL4BAADxDQAAiG8AAMQ3AAAgvgEAAPENAID4BgAAxDcAACC+AQAQ3wAAgPgGAADENwAAIL4BABDfAACA+AYAAMQ3AADiGwAAEN8AAID4BgBAfAMAQDjG94ULF/Lz8/2X7N+/f8OGDdu2bdu2bVttbS02KABAaIiGf9fXXnuttLRUJpP5L6ysrNy7d+9tt92GTQkAMEln32q1et++fX0WVlZWvvLKK4899tirr76KrQkAMBnjOycnh2GYPgvXr1//7//+73/7298qKipOnz6NDQoAEBqiMT5++/btcrmcEJKZmXnp0qXMzMzB7hkfrwjj7Rjeo8PbhwFidOEQ3zzP+25brdYNGzYcO3aMZdmvvvpKr9cHeKDJ1B3Gn54wHl3YDxBvH0Y3RfdMI45viqIIIWVlZXa7PS8v75lnnsnPz5dIJEuXLl2xYgW+zgAAhAblP5seV9j/Y4AYHQaI0QVx9o3DdgAApiTENwAA4hsAABDfAACA+AYAQHwDAADiGwAAEN8AAIhvAABAfAMAAOIbAAAQ3wAAiG8AAEB8AwAA4hsAAPENAACIbwAAQHwDACC+AQAA8Q0AAIhvAABAfAMAIL4BAADxDQAAiG8AAMQ3AAAgvgEAAPENAID4BgCAcIzvCxcu5Ofn+y85efKkXq/fvHnzwYMHsTUBAEJGNPy7vvbaa6WlpTKZzLfE4/Hs2bOnuLhYIpFs2bIlOzs7JiYG2xQAYHLNvtVq9b59+/yXGI1GtVotl8sjIiIWL1589uxZbFAAgEkX3zk5OQzD+C+xWq0KhUK4LZPJuru7sUEBACZdfPcnl8utVqtw22azKZVKbFAAgNAQjfQBPM/7bms0mrq6OovFwrLs2bNnd+7cGeCB8fGKMN6O4T06vH0YIEYXDvFNURQhpKyszG635+Xl7dq164knnuB5Pi8vLyEhIcADTabuMP70hPHown6AePswuim6Z6L8Z9PjCh8gDBCjwwAxuiDGNw7bAQCYkkTYBABwi/N4uJoai8PBsCyn0Sj7tNhNWph9A8CtrqbGYrfH8ny03R5rNFqmymojvgHgVudwMAPeRnwDAExqLMsNeBvxDQAwqWk0Sqm0naI6pdJ2jWbKHH6Iny4B4FbHMIxWq/JfUlT03uXLl9LS1GlparVanZKSJhaLEd8AAJOa1+t97rlnrNYftZNLpZGffvrFrFmzJs96ongCAPDjWKTpt98+mJubl5ampunekLTbezo62jD7BgCYjDiOKy//vKSkqKzsSFdXp3+gv/zyn3/ykyWIbwCASYTn+YqKsyUlRaWlJa2tLcLCefPmT5s27eTJEzRN/+//vqbT6SfbaiO+AeDWdfnypZKSouLiovr6WmGJWp2em6vX6fKam2888shGiqL++78LJ2F2I74B4FZUW3vt8OFDJSVFly9fEpYkJiZt3Jir0+nvvHOxcF7Vnh7b7NnaX/3q6UcffWxyjgLxDQC3ipaW5tLS4pKSooqKc8KS6OjoBx/cqNPply5d1udUJxkZd5WXn5vMw0F8A8BYTfJTPnV2msvKjpSUFJWXf+71egkhkZGRa9euz83Vr1yZPQkbuhHfABAiwimfCCF2OzEa2/scAjNRbDbbxx9/WFJSdPLkCbfbTQiJiIhYvXqtTqdfvXqdTCab6psd8Q0AYzWpTvnkcrk+++zTkpKDH330YU9PDyGEpunly1fm5urXr38wOloVNpsd8Q0AY8WynN3+w+0JWQeO4/75zy9KSorKyko7O3tbthcvvjs3V//QQ7mJiYnht9kR3wAwVhqN0mhs99W+Q/nS/i3bLS3NwsJ58+bn5uo3btykVqeH8WZHfAPAWPU/5VMICC3bR44U19TUCEvU6nSdTq/T6efNu+1W2OyIbwCYSurqag8fPlRcfNDXsp2QkCi0bGdk3CW0bN8iEN8AMAW0tLQcOVJcXFxUUXFWWKJQRD344MM7d26/7bYMX6viFL1qJeIbAIJswtOws9N89OgHxcVF5eX/EFq2WVa6dOmGrKy8u+5apVR2L1uWZjL9cGbXydnCiPgGgFCbkDT0eLjKyuZTp06ePn34669P9WnZnj79PpadLtyzf5PiFL1qJeIbAIIsxGnocrlOnfr0wIG3P//8hMNhI4RQFLV8eWZubp6vZdtgMAdoUpwMLYyIbwAY/ezVv+IxlofTtLumxuJyMWIxp9W6x2mFOY778svykpKiDz447GvZnjv37qws/cqVq7KyZvvfOXCT4gS2MCK+ASDIFY+kpOhRP/zGjSsUJSGEUBTH83xw15Pn+W+/rSgpKTp8uNivZfu25cvX33fftmnTZhBCpNL2Po8K3KQ4IS2MiG8AmBQVD/+H2GyRs2b1pqHb3RmsNayqulxScrCk5FBt7TVhSVqacJZt/bx5t3EcZzRaHI7OW2EGjfgGgB+Msf7r/3CFwinc8Hi4lpb2+vpuQrwajVSrVY2iC+Vmy3bR5cuVwpIBW7ZvqRk04hsAekO2psbS00O1ttbEx0fJZGQUs1f/8vHKldNra9sdDqa9vcPtjnO5VISQq1fbGcYyc6ZymD2Fra2tR44UHzp00NeyHRUVvWHDQ7m5effee194t2ZPivjmef6ll16qrq4Wi8W7d+9OTU0Vlu/fv7+oqCgmJoYQ8vvf/z49PR3bFGCiCGVriiKJiTFS6Sj7/PpMfrXa3tNhGwy9ceF0Mg4HNWRPYVdXp9Cy/cUXp31n2V6zZp1Ol5eVlS2RSPB+hSi+T5w44XK53n333QsXLhQUFBQWFgrLKysr9+7de9ttt2FTAky48evzY1lOLKadTkIIkUg4lvUO9lo9PT2ffHKsuLjo5MnjLpeLEBIREZGTs0an069Z80AYnGV76sV3RUXF8uXLCSGLFi36/vvvfcsrKytfeeUVk8m0cuXKn/70p9igABMoKF3PAx5pqdEovd7Oq1frhNq3RqMyGi3+ryW0bBcXF3300Yc9PT+0bOt0+vXrH1SpYvDuTFh8W61WhULR+xiRyOv10jRNCFm/fv3jjz8ul8t/8YtfnD59OjMzE9sUIChGccB6ULqeB6yKMAwzd27s3Ll9X8tmI9XVX545c+zo0SNms1n4p4yMxTqd/uGHc5OSpuF9nPj4lsvlNptNuO3LbkLI9u3b5XI5ISQzM/PSpUsB4js+XhHG2zG8R4e3b0JcvtzBsmksSwghZnPHvHnRAYL+ypUuh4NhWe+SJcn9g35EA2xo8LJsb4mDpj0DPpbn+XPnzh069M57773X1NQkLJw/f3529sPZ2XmzZ6dptVEh+00y7P/6xhrfGRkZn3322dq1a8+fP6/Van1T8g0bNhw7doxl2a+++kqv1wd4Bv/TyoTfH38Yjy7sBzhpR9fUZOf53jlTZ6c9Lm7QlTQYzMJ8mRBRR0djn18RRzrAnp4uu703GaTSLpOJ9v/X6uoqoWX72rXes2ynpamFs2yLRNOF1WhsJP1XAx/OoO+ZhhvfOTk55eXlmzdvJoQUFBSUlZXZ7fa8vLxnnnkmPz9fIpEsXbp0xYoV+DoDECzDL2QH9xfLASsw9fV1Qsv2pUu9P33FxMTrdLmbNuUtXny30LJ98aIliKsBQ6KCfiAsZt8YIEYXFDcPPhy69u03+yb9+wVVqsivv24c3UlfW1tbP/ig5NChg+fOnRGWyOWK++7TZWXlLVq0Qi7v9H+twKuBD+eEzb4BIMSGf/Bh4F8sr1zpGulJX7u6Oj/8sKy4+ODnn/e2bEsk0nvuWbdw4QNRUQtmzEhNSZEzDN1nin1LnS5qMkB8A4R50A+/tNLT03P8+EfFxUWffvqJr2V71arVixevX7o0z2RyOxyxzc0NTmd0Q0OnWq3sU9LBwe6IbwAYgSH7C1mW8/2lD1hDd7vdvpZtm81KCKEo6r77Vuh0+g0bHlKpYgwGs90udzothJDUVCnDtLtcPVKpG1NsxDcAjN6QB69rtVEdHY39axper/fLL8uLi4vKyg77WrbvvHPx8uXr7733oZSUeP9jdozGdpa1er0kJSWKYWip1IuJNuIbAMY04+7p+eHq6n1qI8LdIiO9hJD582VCFvM8f+HCt8XFRYcPH2puviHcc86cubm5eRs3bnK7VQMes6PVqjQapdFocTgsKG0jvgEgCDPu1taaxMSYAWsjwt1YVma3i4zGdkJai4sPlpQU+bdsb9y4SafTa7Vzr13r7u5m6uu7pk+PFoK+z84ApW3EN0D4zHzHcvH1wZ5nOM/vH6zx8VFS6cD9HsLdbtyoO3z4b6dOvWs0Xrr5kISHH9bpdPq77vqJMHX39fx5vXRjoyUtTUVugWtFIr4Bbt2Z71guvj7Y8wzn+f2P6JHJyID3aW1tPXr0rY8/Pnbp0lfCEqUyasOGh3Q6/bJly0Ui0YD7g5QUeVNTI0VRqJAgvgHCTbAOcfQ91uPhDAarb/o8nOcP0GFtsXQdPfqBf8s2y0qXLVu1devmVatWD3aWbd/+gGForTZSq0VwI74Bwk5QTsrq/zxNTRaeV/G8UphusywZ8vn7l6HtdrvQsn3ixMdCy7ZIJMrOzsnNzdu69VG7fYiDqzUapcFgMhodwvlgOY7DRXAQ3wDBFKy681gIM1+rlbS1dSUkRBsM5tGtiW8GTVHW5ORk33R7/nzZkMcuejycwWCuqbF7PJzZfO7MmY/8W7aXLVsutGzHxMQSQuRyud0+xGHlDMMwDC1cRcvlGlNRCBDfAAMIVt15jPsMrVZlMJgZZuZY1sQ3g2ZZzm6nfdPt4TR4XL3a+eGH5/75z4+//fawzdYuLLzzzgzhLNvTpk33X/OGBm9PT9eQ+5jxu1IPIL4Bghwx/ummVsvq6my+OXVkJO/LuwH3GUFck+GfKsTXsl1UVNTW1iwsnDZtzpo1Dz355Gaej3M4mO5uLiGB819zX+Ng4L1CsIpCk//7E+IbYAKMLmIGiwz/dDt9uiYxcWZjo9nhmOVwdKrVysBJHaywG2acXbliEFq2a2qMwpLY2JTFizffdVfejBkp8+Z5eJ6McR8zxhNODXMgE/L9CfENMPFGFzGDRYZ/onV3SxITidPJEEJcLmbIpA7W2fWEdeM4b1WV1WBo0mrl/tnX0HC9pORQSUnR999fFJZER8dnZuZmZm6KjY3jOAkh9KxZLo0murLSNsZ9zBiPyhlmLqNEg/iGW9ToImawyPBPN4XCSQiRSDiHg4jF3JBJHWBN+sxDeZ4EmJYK69PQYHU6oymK2O1Ko7FdpfIcOVJSUlJ05swPLdvr1z94xx0PLFr0AMOICCEU1blwoTLw9xJhzWnaI5V2jXcT9zBzOTQlGsQ3QJiXXPzTbeXK6bW17SkpxGS6mpAQLZW2+yf1zJlKIYKrq800TblcEQFKBH3moRznvXJF4nIxYjHt9XbOnRvbf92E+T7HmT/++Mhnn71z/vwXHMcRQqRS6erV63Q6fXZ2jkQiMRjMvsuY9cm+APuY+HhFnwueTWBdC+cEDyJcbScIcLWdEBvpz1+BL1sznNH5jimvrzfzPKNWK8mPLyjjW6WICLfBYHM4Yjo7u1QqRWRkl9fLO50zhbtJpXVr1yb5rzwh5NKl1qKiD7/++uPvvvvM7XYSQkQiUVZWtk6nX7v2AblcMcyBTOzbN7p1w1/fkKPD7BvCykh//hr7uZZ81QCnkyGE8V8oBLfB0MPz8uRkRW2trbnZ5nR63e50p7M7PV3V3Fyv+uHFvb6V7+52//3vpWfOfHTs2FGrtZsQQlHUHXcs2bLlkYcf1sXExN7cJVh8gSgMRFheWWkLZVAOucvECa1QPAEYQZiSUP385asMSCSc7/uqUCIQ4tjhYHle0djY7nKJVaqo1lYrRYlpujslJZ7n5TTd7nQyEgmn0Uh7eqjvvvvis88O/uMfJRZLh/BUd9xxp06Xt3HjDy3b/fdSvgJOc3NHXJyaYZhQNm+gYwTxDRDMMCWh+vlLo1FWV7fW1Ni9Xl4i8Xq9bpmMCKUPYf8hFnNOJ3E6GbGYoyiiVsscDrlE4mEYes4cGU177XZSV3fhnXc+Ki4+1Nrae5bt9PRZjz76qE63aebMWUPupYQA9Xi46uoeg+FGSoqUEJrjHISYQzAHR8cI4hsgOGEa4p+/GIYRiZjUVLVQRjCZ6hgmxmi0aDRKYV+SkiJvaOikafOsWZE8zzscjMl0NSpK/s03tVZrx/fff3Dhwglfy3ZSUkpm5kNr1z64Zs09fc78F2AvJYRmU5OF56PdblJfTwhxzZgRaberQjAdRscI4hsgOGEasi/vvppvfb1VuI5BU5PF6UxMTJQJZQTfvmTuXE6jme7Xsm3du/f/ffHFpw0NvS3bcXHxDz+s0+ny7rrrbpoeuhWkz17KaLTY7cTpZBITIzs7m2w2sVjsSE6eFprpMDpGEN8AU4bvN0mPJ5IQvqGBbWxsuvvuZKFC4isj9NmXtLW1CS3bX3/95c25qvKOOx66++6sXbt0AebaQ+6l/C84OXt2UkODlaIYYW8RgukwfplEfAMMOr2dbCfB8P0m2dREEeJKTEwwmRqbmhqVSndcXHqfMkJ3t+WDD468/fZ758594fUKR/2wCxeuWLRox/z5ayIi2Kioq/7ZPYpR97ngpFbr5nne7e7EdBjxDTCRKUnGs6VhdHsI32+SHo+E50U0zajVsrQ0av58mdFoFp5t+vSIDz44LJxl2+l0EkIYRvSTn6xZs2btjh2PiMWSU6eaurtNCoVz5crpwx+1/woL59LyX/mbLSgRLMtptbJR7PBw3ijEN0AQhKClYcCs9EXY9OlulYruH2G+3yRv3Gh2uTiRyOH1kvp6O8tyaWnS8vJ/FBcX+bdsL1y4JCtry/LlD0dFxVFUp3C4zerV6aMYtf8KC+fS8l/5se/w0AWI+AYIghC0NAyYlb4I6+mRdXTU+7qqfRNS3491mZkRPC8yGm0eT1RHR92ePQe/+KKks7O3ZVurXbR69YM7dmy22xXCEw5nIIFH3f9cWv7Lx77DQxcg4hsgCELQ0jBgVg7YVe0/IfX9WMfz/HffXTh27K1Tp8pMpkbhIbNna1es2HDvvVs9noTGRufRo6bUVJfH08UwEbNmsRpN9FhG3f9cWv4rP/YdHroAb6345nn+pZdeqq6uFovFu3fvFi6qRAg5efJkYWGhSCTatGlTXl4eNiiMVLBaGgLUcwfMygG7qvske1VV1RtvvH38+JGGhhphSUJC6sqV+nXrch54YNl333XX1tINDbTLFdvYSLndkenp7tRUFU238zwxGMyDFZeHLD37r7BwLi3/lR/7Dg9dgLdWfJ84ccLlcr377rsXLlwoKCgoLCwkhHg8nj179hQXF0skki1btmRnZ8fExGCbQoj1OeuI3c70qecOuIfwRVhkpDM5uberWtDVdf1//udEcfFB31m25fLYe+5ZpdNt0mqXCBfioSiKZTmXK8LtpgghFMW73bRwuvAB5/L+Lz1k6bnPCmu14uDu8NAFeGvFd0VFxfLlywkhixYt+v7774WFRqNRrVbL5XJCyOLFi8+ePbtmzRpsUwhZZDscjEjkqq+3Wq2Jzc2e+PjoxsbOtDTVcOq5vggTTlmn0SjPnKkqKjry1VdlV658c3NWrtBq182dmz979p3JyRELFzq02mj/HUBt7XWxWOx222na3dbWLZPZZ86UyeVc4OIySs8Q0vi2Wq0KRe+pC0UikdfrpWnaf6FMJuvu7sYGhRFVCUbNN4E1Gs21tdKkJIXXS7W22qZNG81hLCaTKScn68aNBp73EkLEYvbee+/PzNyYlLSqtjaSkCiO6xaLPX2ilmGY7OzU1FTzqVMNNC0jxOl0iltbaxYtSvefy/dfGZSeIaTxLZfLbbbeqzEJ2S0stFqtwkKbzaZUBiqiBT5x7VQX3qMb9QAvX+5g2TSWJYQQs7lj3rzoYK1PQ4OXZWWEkJYWj1zuVSqlMpm4paU1IcGZnOzUapNHtKv45puqpqZ6QsiCBWszMzfq9VsiIz0tLZ0mk7y11WCzTYuMtC1cOC0xke+/HZKSok0maubMdOF/pdLapKTo+HiFwdAl7Lf6r0xMTGSAf8XnE6MLcnxnZGR89tlna9euPX/+vFarvfnlUVNXV2exWFiWPXv27M6dOwNOcHC5hltugE1Ndp7v3et3dtrj4sa0lfzn8l6v1+kUEUKczm6Vina5GpxOJjm5bdmyVIZhOjp6hv8lID5ekZS0MCEhtbX1elbW/507d7HbzRgMdXFx6Y2NjV5vosXSIpXGf/mlMSkp1mRi+6+YxeKw35xOu90OYVvFxUUQQgih+6yMIPC/4vOJ0Q1nzzTc+M7JySkvL9+8eTMhpKCgoKyszG635+Xl7dq164knnuB5Pi8vLyEhAV9nYHRVgsAJ2//HSbG4VSptdziYWbPcFCVcvcyt0aQyDNP/qXynWq2rsxgMzVptpP+hjDExkVKpNydny1tv7f322zfvuGOGVOqIj4/yenmXi3R29rBsakJCDMNE1daabr99gNX2ej0dHQ0qlVws5hnGdfGiBUczQgjgYmnY/4/jAH0X0BKJXIEvEem7Ghn58UXI/P/1yhUbzytYtj0tTcVx7XI5GTDu+z/VxYsWno+urzc7HLEU1T17tqylpfdQRkJIcrJToeDffvvz55/XyWTK4uIvGSaqubmjpyfq+nXJd985eF4VF+eYPp2fMcO8YUNy/xXjOK6x0UJRVrHYLVxIYcBR4POJ0QV39k0TgHEjdHcsXKgUiZientjaWvriRfbTT68L1+H1N5xWDeE8f0JzXltbl90ey/PRdnus0WgJ/FTCxN9up27csDY3W+vqLF1dYv/71NXZMjJyZs683WazlJV9zfPRcXHqlpbWxETv9OntcXEWmm5PTfXMmsUOuGIMw6SlqdLSFElJMb4dCVpKYLwhviEUHA6mocHqdEbzvMJiiTMaLR4PZzCYL160GAxmjuP8SysDtmoQQlJS5BJJJ8uapdL2hIToweKeZTmO89bVWa5csTU3d3Acp9EopdL2zs4mivLGxyc4ndFWa4f//YVnyMrKI4R8+WWpkMjJyZFz5igfemjWvfeSpUup227jZs+OHnDFfLf7vzTeekB8w9TGspzL1RuyEgnnO7DFN30WEpaiOqXS9v7HAarVspaWmmvXbrBs2+rViVqtKjKSHyzuNRplW1uty0UkEldcnNpg6BQKOElJkampTobpZtn2BQuSfC+n1UYJz7BypZ4QcvHiJ3a7jRCi0Uil0naRqHvOHO+6dUnCYfR9VqzPavd56T5fCwCCC+c8gVAQjnCxWDiJhEtOVrJsZ58SR4DjAD0e7vTpJrNZ1dnZxXGqU6easrNTAxz2zTBMUlJMYmLvQqPRIZzjgaJ4mmZmz1aSHxemfWenmjYtav78jMrKb7766uADDzyo0aiG/O2x/2r7vzTqJ4DZN0x5PE9SU+Usa6Yoa2Rkh3CJyADVEn81NRazOa6y0mk0Tq+s7OnsjDMaLYF/cf/xE3qF/yQnK2naPOAEn+cJx3nr67sXLVpLCDl3rmzAuXafgs+QLz1ks82QzwaA+IYJVlNjcbkSUlPVqampNE0LE94A1RL/gPvuO4fBYOrpUfJ8pN0ebTZb+9deAtY0pH4zZfnChcr+0VxTY7lyRWK3qxcseJKi6JMnPzWbOwYcRYAXHbCcEiCmh/NsACiewATzLyNYrb6z8ZH58we9WIzHwx0/XltdzdbUtNlsUS5XTWLibKm0S6VSsKwrcKeKf02D47ghz67ncDBCaV6pTJwzZ1lV1edHj36wdev2AKMYrDDSv5wy2AmqcOYTwOwbpgD/MkJLS1dVVYTBEFFZyRw/fn2w6kFNjeXqVfbGDbnNNsdkYlwuiUJRffvt0uho04hqL77mxQHrIb5n8F19eMmShwghJSVFYymMDCf0R/dsAIhvCBGhdNDTQ7W01Hi97VJpu8tFCx2EDQ2iK1ekg1UPHA6Goiiz2c3zUbGxKbGxsrg4yR13eLOzU4dZexlsZfrvMDQapVbrlErrpNJren2OWCz+4ot/tLQ0D1kYGemuy//26J4NAMUTGHEKD1ni6KEAABC8SURBVHZQe/9/6ndykniKIomJMUK/h8HQe5ozt5uOiOAHqx6wLJeSwlZWWjweB8vaZs2SRUR0OhyM0GU4ujNWD1bHYBhm7tzYuXN775advfrYsbLS0uKf/vTngQsjwzFYkwxOug2IbxjfdO4ffAaDiWFo3537Z6L/kuvXr9+8NFNvRqvV4n/+s8Zmk9hs7fPnJw1WPdBolBxnbm7uvnHDGBsrjYjgEhMTeD7abifV1a0iETOK89A6HIzHwzU1WZxOhmWtgz02N1d/7FhZSUmREN9jPO0tYhoQ3zCOhrz+i//s2NdJLdy5f233x1Npb5/SQUSEaNo0mcvFiERSlu2gKNGAPyoyDDNvXpxWqxIOuqmv75o+XXVzbe2pqWoy8gulsyxXV2dxOGIJIV4vMRotAz42J2etTCavqDhXW3stPX0GrssOiG+YvIbsgvA/d6DH466vNzudjETCpaQQmazvaQX976zRSHm+tabGznFELPbYbKSxsSclpffcIBRFFi5UDmfqyrKc3e5bMTrw2gaoYxgMzRQlFou5lBS5wzFwu15kZOTatQ8cOvT+4cOHfv3rf0GLCExO+OlyqtY6gnvEx5BdEP6/s7Es53DE8ny0wxFrMnX1/wnOf4lWqxKJmNRUNUWprNbZDQ2016tqbLR4PFx9vbm+vnvAIQgD/Pbbzk8+qf32W7PBYE5Pl/me0//UUSPq2WAYRquNnD1bplYrGYYO8NjcXD0hpLj4YJ+XiIhw41gbwOwbxrHWMVJDXnrc/yjH+HiVy9XpcjFiMZeQEN2/tttniTBjFRqrnU5m5kx5U1PjjRsWnlclJyfb7bRvCL4qc3NzR1xcekOD1emc5XS2T5+uPHWqLikpxrd6o75Q+jAvsp6Zeb9KpaqqunzpUuWcOXN9D+E4HoUUQHzDONY6RmrIn9dqaizd3dFNTRanM6K7u3bRotsZhiaESKXtw5na2+1ELOacTiKRcAxDa7WRDgfD876DEq03w5FzuRIIIRYL7XBYfYnf1GRxOhMTE2W+0Bx1bg7zh0SxWLxhw8Y333yjpKTohRd+53vIxYsWFFIAxRMYx1pH0EsuDgfT1GQRaiYyWXpbW+3wG5aFWkpamicq6mpKild4lG+1m5osXq/qZvd3b8lcIuGE2b1w2+lkfIfVhCw0hfpJSckhh8Pxhz/sqa6uIjjWBjD7hvGudQS95MKynNMZcfM2SUqKCfyT4yAT3uj+Q6Aoa3JyMiGE47yNjT0Oh00s5uLjpVVVBrk8xmqtnTEjqaPDHBeXHuLQvOeepUlJ0+rra/ft+++9e/+zsbHhT3/6n9FteQDEN4ysAhDEksvNM74yQs8Gy5qDNQSW5ex2mhDS0GBNTIxnGJfTyVRVGW7WZ1Kk0vY770w1Gs2hDM1/+Zdff/75qeXLMw8efPfEiU8IISpVDEETNyC+YaJKLsO8cPCAaZudnSq0YLOsOXCGBj4Os8+RL775LE13paSkCP909WqsUFsnQ50NfJy4XM5r12o6OtoJIZWV3xFC0tLU+AgB4humUsllFFP+wMdh+tdtfhzrUt8VeRQKp/9eJ/Tbau/eP7W1mU6c+ISmaYfDgfgGxDeEyICz3ZDNYQMfh+l/2z/WJRKTWGwyGh2EeNPTpSKRyXdl+on4psK+8cZbTz75fz766KiwRK1GfMPkgs6T8DTelwII3MTSvz1jsIYN/yh3uSIYhk5NTU1NVXPcNJqmFy5UzpypNBotE3KYjEQief31v61cmS38b0pKGj5XgNk3TMD8dxSz9RGVR/z/tU+VxuPhvF7v9evXCfFqNFKNRuUf5f7l+P6rPbHnG4mIiHj77YN5eRsVCrlEIsHnChDfKGWMe+fySH+lHGlKDv9iN4QQg8HsdMYL5x1kmHZh+MI2sdmIyVSTkBAdGclrNEqj0dJntSf8fCMikaikpAyfYUDxBKWMEF3VcKSXAhhpSo7o6JUBn1zYJjQdm5g4MzKSFy6F03+1A7wQLvULmH3DpCtljN1wfqX0/04gFnudzhHM1kfUxDLgV4FBrn45aFth/xfCeVwB8Q3jbiwN1+P9nUCIP7G4VSodQU/hiJpYBoxg3zbxeLiLFxsVCo1YzKWkKI1Gs/8zB3ghnMcVEN8QilLGJDzS2j/y3G7xvHnjtWIDRrBGo6yubq2psTc12ZzOuMhImdNJNzR0pqcPN4gn504RYNLFt9Pp/M1vftPe3i6Xy/fs2aNS/fDXuHv37m+++UYmkxFCCgsL5XI5NutY5qqT/ztBUH6JZRhGOA+4w2FpavK2tPRMmyZ3uRiWdU/pnSLApIvvd955R6vV/vKXv/zwww8LCwtfeOEF3z9VVla+/vrr0dHR2Jq3yHeCYBWdhem/RMIlJka3tt6gKI9S2abRpE7pnSLApIvvioqKJ598khCyYsWKwsJC33Ke5+vq6l588UWTyaTX6zdt2oRtGvbfCYJVdBam/8nJysbGzhkzOK3WrdGk+rcVhrLVEiB84ruoqOjAgQO+/42LixOqIjKZzGq1+pb39PTk5+fv2LHD4/Fs27ZtwYIFWq0WmzW8Bavo7Jv+z5nj1WiSeJ7cPCXWD9dtQFcJwIjjW6/X6/V63//+6le/stlshBCbzaZQKHzLpVJpfn6+RCKRSCRLliypqqoaLL7j4xVhvB3DeHQeD3f5coeQqlptFMMwMTGRBkPXzSXJY5kaJyX9UHO7fLmDZdNYlhBCamtr09NlwnKa9oz35g3vDyf++m714klGRsbp06cXLFhw+vTpu+66y7f82rVrTz/9dGlpqcfjqaioyM3NHewZTKbuKRFVo/jOHh+vmBKjGx2DwcyyaWazjRBRR0ejMBGOixMu3UB3dPQE64Wamuw8bxNuWywOs9l2c4rQZTLR4/rHH8ZvX9gPMOxHF4T43rJly/PPP//YY4+JxeKXX36ZELJ//361Wp2VlbVx48a8vLyIiAidTqfRaKb0xsKRIP05HAzL/nA7NDWZWbNYmkZXCUAgFO9/CfHxNCX2kBcvWni+9+s8RXUO83pg4bH/H+ybh9/sm0il47hL4zjOV/sO5c+VmH1jdGE++75F3MpHggz2zUOjUZrNHZ2d9vGeCKMREGBEEN8/EtwjQSa8+21EKzBYOyDDMPPmRcfFdePjAYD4nryCOwGc8Er6iFYAx6ADIL5hiPls6FeA47wGQ0/gaTiOQQdAfMNkmc/6VqChwUpRcp6PDjANR+kZYGrB5RrG0UivmTCkkV6gwLcCNG1OTlZO4PcAAMDseyoJ+nx2pMV03wqwLGe3MxP4PQAAMPu+pY26mB707wEAgNk3jMCoi+moawNg9g0TCZNoAMDse0rCJBoAMPsGAEB8AwBAyKF4MiXhWmIAgNn3lCQ0gPN8tN0eazRasEEAEN8wNUz42VQAAPENo+Hf9I2jKAEQ3zBloAEcAPDT5ZSEBnAAwOwbAACz78lhal2iDAAAs+9eE95Uh64+AEB8j8bkuUQZQVcfACC+h2/Cm+rQ1QcAiO/RmPCmOnT1AUAIhOFPlxPeVIeuPgDA7BsAABDfAACIbwAAmErxffz48WeffbbPwvfff3/Tpk2bN28+deoUNigAQGiM4KfL3bt3l5eXz5s3z39hW1vbm2++WVJS4nA4tmzZsmzZsoiICGxWAIBJNPvOyMh46aWX+iy8ePHi4sWLRSKRXC5PT0+vrq7GNgUAmMjZd1FR0YEDB3z/W1BQsG7dujNnzvS5m9VqVSgUwu3IyMju7m5sUwCAiYxvvV6v1+uHfLxcLrdarcJtm82mVA56lEp8vCKMt2N4jw5vHwaI0U2l+B6mhQsX/td//ZfL5XI6nTU1NbNnzx7sniZTdxh/esJ4dGE/QLx9GN0U3TONPr7379+vVquzsrLy8/Mfe+wxnuefeeYZsViMbzQAACFA8TwfmlfC/h8DxOgwQIwuiLNvHLYDADAlIb4BABDfAACA+AYAAMQ3AADiGwAAEN8AAID4BgBAfAMAAOIbAAAQ3wAAgPgGAEB8AwAA4hsAABDfAACIbwAAQHwDAADiGwAA8Q0AAIhvAABAfAMAAOIbAADxDQAAiG8AAEB8AwAgvgEAAPENAACIbwAAxDcAAEwNohHd+/jx4x999NHLL7/sv3D37t3ffPONTCYjhBQWFsrlcmxWAIBJFN+7d+8uLy+fN29en+WVlZWvv/56dHQ0tiYAQMiMoHiSkZHx0ksv9VnI83xdXd2LL764ZcuWQ4cOYYMCAEzw7LuoqOjAgQO+/y0oKFi3bt2ZM2f63K2npyc/P3/Hjh0ej2fbtm0LFizQarXYrAAAExbfer1er9cP+XipVJqfny+RSCQSyZIlS6qqqhDfAAATGd/DdO3ataeffrq0tNTj8VRUVOTm5g52z/h4RRhvx/AeHd4+DBCjC6v43r9/v1qtzsrK2rhxY15eXkREhE6n02g0g93fZOoO409PGI8u7AeItw+jm6J7Jorn+dCsBz5AGCBGhwFidEGMbxy2AwAwJSG+AQAQ3wAAgPgGAADENwAA4hsAABDfAACA+AYAQHwDAADiGwAAEN8AAID4BgBAfAMAAOIbAAAQ3wAAiG8AAEB8AwAA4hsAAPENAACIbwAAGBehu1QxAABg9g0AgPgGAADENwAAIL4BAADxDQCA+AYAgFslvu12+89//vOtW7c+8cQTra2tYbYRrVbrz372s/z8/M2bN58/fz5cPyvHjx9/9tlnw2lEPM//7ne/27x587Zt265fvx6W79qFCxfy8/PDcmgej+e55557/PHHH3nkkZMnT4bZ6Lxe729/+9stW7Y8/vjjV69encj4fv/992+//fa///3vDz744F/+8pcw29BvvPHGvffe++abbxYUFPz+978Pyz+V3bt3/+lPfwqzQZ04ccLlcr377rvPPvtsQUFB+L1rr7322r/927+53e6w/EweOXJEpVK99dZbf/nLX/7jP/4jzEZ38uRJiqLeeeedp5566o9//OOA9xGFZlW2b98uHB/U1NQUFRUVZht6x44dYrFYmA5IJJKw/FPJyMjIycl57733wmlQFRUVy5cvJ4QsWrTo+++/D793Ta1W79u377nnngvLz+S6devWrl0rTFRFIlGYjW7VqlX3338/IaSxsXGwzByvMRcVFR04cMD3vwUFBbfffvv27duvXLny17/+dapv2QFHZzKZnnvuuRdeeCEMPjr9B7hu3bozZ86E2V+I1WpVKBS9fwkikdfrpemw+jUoJyensbGRhCmpVCq8iU899dTTTz8dfgOkafpf//VfT5w48ec//3nge/ChZTQaV61axYedqqqqDRs2fP7553z4+vrrr5955plwGlFBQcGxY8eE25mZmWH5rjU0NDz66KPh+plsamrKzc0tLi4O47+7tra2rKwsu93e/59CNNd49dVXS0tLCSGRkZEMw4TZTvLq1au//vWv//CHP9x3330EplRF6PTp04SQ8+fPa7XacB1muJ7XqK2tbefOnb/5zW90Ol34ja60tPTVV18lhEgkEpqmB/xeGKKC0aZNm55//vmioiJhyhNmG/qPf/yjy+XavXs3z/NKpXLfvn1IxqlSWygvL9+8ebNQIArXYVIUFZbjeuWVVywWS2Fh4b59+yiKeu2114SfoMLD6tWrd+3atXXrVo/H88ILLww4NJxxEABgSsJhOwAAiG8AAEB8AwAA4hsAAPENAACIbwAAQHwDACC+AQBgcvv/XcOOxrBfOx0AAAAASUVORK5CYII=" />

These vectors represent the principal axes of the data, and the length shown in Figure 5-81 is an indication of how “important” that axis is in describing the distribution of the data - more precisely, it is a measure of the variance of the data when projected onto that axis. The projection of each data point onto the principal axes are the “principal components” of the data.

If we plot these principal components beside the original data, we see the plots shown in Figure 5-82.

> **Figure 5-82: _Transformed principal axes in the data_**<br><img width="500" src="data:image/png;base64, ..." />

This transformation from data axes to principal axes is as an affine transformation, which basically means it is composed of a translation, rotation, and uniform scaling.

While this algorithm to find principal components may seem like just a mathematical curiosity, it turns out to have very far-reaching applications in the world of machine learning and data exploration.

# PCA as dimensionality reduction

Using PCA for dimensionality reduction involves zeroing out one or more of the smallest principal components, resulting in a lower-dimensional projection of the data that preserves the maximal data variance.

Here is an example of using PCA as a dimensionality reduction transform:

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("original shape:    ", X.shape)
print("transformed shape:", X_pca.shape)

Output:<br>`
original shape:    (200, 2)
transformed shape: (200, 1)
        `

The transformed data has been reduced to a single dimension. To understand the effect of this dimensionality reduction, we can perform the inverse transform of this reduced data and plot it along with the original data (Figure 5-83):

In [ ]:
%matplotlib notebook
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.8)
plt.axis('equal');

> **Figure 5-83: _Visualization of PCA as dimensionality reduction_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAeoAAAFRCAIAAAA0AnnbAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3de3wU5b0/8O/M7P2WTcgmYXPFxRRU5CfKqXjhUsSCohVNLCiBKr3X87PiKyrVeqlSrLHW9hzSo1KPVNt6LF6w1EtBSvQAVsAfUFBY2JiEZBOy2Wyy99ndmef3x8B2zWUJSQjJ8nn/tdnsbuaZ3Xzm2e888zwcY4wAAGCs4bELAAAQ3wAAgPgGAADENwAA4hsAABDfAAAwKKqR+TOJhOTzhTN1J2ZnGzK4dRnfQLx9aN2oZbOZz37vW6USMvkYmNGtw9uHBqJ1oxOKJwAAiG8AAEB8AwAA4hsAAPENAACIbwAAQHwDACC+AQAA8Q0AAIhvAABAfAMAIL4BAADxDQAAiG8AAMQ3AAAgvgEAAPENAHCOGehiaYlE4ic/+UlLS0s8Hv/+97//ta99Tbl/69attbW1KpXqlltuqaysxA4FABhd8f32229nZ2c/9dRT3d3dN910kxLfiUTiySeffOONN7Ra7ZIlS+bOnZuTk4N9CgAwAgZaPFmwYMHdd99NRLIsq1QnQt/lcpWWlppMJrVafemll+7atQs7FABgdPW+9Xo9EQWDwbvvvvuee+5R7gwGg2bziYWQjUZjIBDADgUAGF3xTUStra133XXX0qVLr7vuOuUek8kUDAaV26FQyGKxpHl6+hXvx7rMbh3ePjQQrRvD8d3R0bFixYqHH3748ssvT97pcDgaGxv9fr9Op9u1a9eKFSvSvILHE8jgT08Gty7jG4i3D60bo0emgcb3c8895/f7a2tr165dy3HcrbfeGolEKisrV61adeeddzLGKisr8/Ly8HUGAGBkcIyxkflLOP6jgWgdGojWDWPvG5ftAACMSYhvAADENwAAIL4BAADxDQCA+AYAAMQ3AAAgvgEAEN8AAID4BgAAxDcAACC+AQAQ3wAAgPgGAADENwAA4hsAABDfAACA+AYAQHwDAADiGwAAEN8AAID4BgBAfAMAAOIbAAAQ3wAAiG8AAEB8AwAA4hsAAPENAACZGN/79u2rqqpKveell15auHDhsmXLli1b1tDQgB0KADAyVAN/6Lp16zZu3Gg0GlPvPHjw4FNPPXXBBRdgVwIAjNLed2lp6dq1a3vcefDgweeee+622257/vnnsTcBAEZjfM+bN08QhB53Xn/99Y899tjvf//7PXv21NXVYYcCAIwM1RCfv3z5cpPJRESzZs367LPPZs2a1d8jbTZzBu/HzG4d3j40EK3LhPhmjCVvB4PBhQsXvvvuuzqd7uOPP66oqEjzRI8nkMGfngxuXcY3EG8fWjdGj0ynHd8cxxHRpk2bIpFIZWXlypUrq6qqtFrtjBkzZs6cia8zAAAjg0vtTZ9ROP6jgWgdGojWDWPvG5ftAACMSYhvAADENwAAIL4BAADxDQCA+AYAAMQ3AAAgvgEAEN8AAID4BgAAxDcAACC+AQAQ3wAAgPgGAADENwAA4hsAABDfAACA+AYAQHwDAADiGwAAEN8AAID4BgBAfAMAAOIbAAAQ3wAAiG8AAEB8AwAA4hsAAPENAACZGN/79u2rqqpKvWfr1q0VFRWLFy/+85//jL0JADBiVAN/6Lp16zZu3Gg0GpP3JBKJJ5988o033tBqtUuWLJk7d25OTg72KQDA6Op9l5aWrl27NvUel8tVWlpqMpnUavWll166a9cu7FAAgFEX3/PmzRMEIfWeYDBoNpuV20ajMRAIYIcCAIy6+O7NZDIFg0HldigUslgs2KEAACNDdbpPYIwlbzscjsbGRr/fr9Ppdu3atWLFijRPtNnMGbwfM7t1ePvQQLQuE+Kb4zgi2rRpUyQSqaysXLVq1Z133skYq6yszMvLS/NEjyeQwZ+eDG5dxjcQbx9aN0aPTFxqb/qMwgcIDUTr0EC0bhjjG5ftAACMSSrsAgA4xyUSUn29PxoVdDrJ4bD0GGI3aqH3DQDnuvp6fyQyjjFrJDLO5fKPlc1GfAPAuS4aFfq8jfgGABjVdDqpz9uIbwCAUc3hsOj1Xo7r0uu9DseYufwQpy4B4FwnCEJ5efaY22z0vgEAxiTENwDAmITiCQAAEZHfH6ip2e12G+32YHX1dItltE+lgt43AAARUU3N7p07r2tqmr1z5/U1NbtH/wYjvgEAiIjcbqMyJR/HcW63EfENADA22O1BZQo/xpjdHhr9G4zaNwAAEVF19fSamnfcbqPdHqquvgzxDQCZb4xO+dSDxWJ+/PE5Y2iDUTwBgKEao1M+jXXofQPAUI2SKZ/G3Mg/9L4B4CwbJVM+jbmRf4hvADjLRsmUT2Nu5N8QoXgCAEM1SqZ8stuDjY2M47ixMvIP8Q0A54r01e0xN/IP8Q0A54qamt07diyIxeRDh8jne/vXv56X+luLxfzIIzOVIYxtbQmjURqjQxgHCLVvAOhXIiE5nb79+/1Op0+Szv4yNG63MRaTZVlFpGpr62OQ4jk1hBHxDQD9Oitp2NnZdddd71VUbL/rrnd9vq7UX9ntQVkmImKM2Wzh3oMUx+iqlYOD4gkA9OuspOHDD3/86aeLOI5zu9lPf/rmf/7n/OSvqqun+3xvt7WNs9nCS5deptPFejxXp5MikX/dRnwDwFjS4xL2oTyd5+P19f5YTNBopPLy+Mhsf1ubOTn+r63tSycnLRbzr389z+VSNi/Wu3UOh8Xl8g667YhvADiblIoHEUUi5HJ5Cwqsg356a+sRjtMSEcdJymx8I6CgwO92nxj/V1AQ6PHb9IMUx+iqlYhvABiGikfqU0Ihw8SJJ9IwHu8ame1//PEZP/3pm21t5oKCwOOPX443FPENcK4YYv039elms6jcSCSk48e9TU0BItnh0JeXZ5+5MXnZ2dbUejcgvgEyn1K2Doe59vZ6my3LaKRB1H9Ty8ezZ9sbGrzRqOD1dsbjubFYNhEdPeoVBP9551nSTxJ7rk0gNXrjmzH26KOPHj58WKPRrF69uri4WLn/pZde2rBhQ05ODhH97Gc/Kysrwz4FOFuUsjXHUX5+jl7vHVwVuEf5uLxco9xwOk/EhSgK0SjXo8Le42/5/YFFi95sbFyq0bD8fD3R+2NrKu2Miu8tW7bEYrFXX3113759a9asqa2tVe4/ePDgU089dcEFF2BXApx1Z26cn04naTS8KBIRabWSTien/1s1Nbubmi6Mxy3xOB0/3n0uTCA1euN7z549V199NRFNnTr1wIEDyfsPHjz43HPPeTye2bNnf/e738UOBTiLhmXUc59L5zgcFlnuOnq0Ual9OxzZLpc/9W/1KJW43Ua1OiyKjOO4WIw7FyaQGr3xHQwGzeYTpSuVSiXLMs/zRHT99dfffvvtJpPpRz/6UV1d3axZs7BPAYbFIFYgG5ZRz31WRQRBmDRp3KRJPf9WR0fkj3/cHw5nNzYekqTb1WpLYyOrqXnHbmd5eTOJ3o/H9SUlB6urF+ENPWvxbTKZQqETx89kdhPR8uXLTSYTEc2aNeuzzz5LE982WyafuMjs1uHtOys+/7xTpyvR6YiIfL7OyZOtaYL+yJHuaFTQ6eTLLy/sHfSn1cDmZlmnO1Hr4PlEj+d2dwcee2yHy8WOHWsoKTm/qemILH9bpVIdO/ZvavWOvLxrZJkaGgx//ONlTzzxj2PHjMXFkUceWZaVZcaH86zF97Rp0/7+97/Pnz9/79695eXlyS75woUL3333XZ1O9/HHH1dUVKR5BY8nkMH//Bncuoxv4KhtndsdYexEn6mrK5Kb2+9GOp0+pb9MpOrsbOlxFvF0GxgOd0ciJ5JBr+/2eHgi8vsDDz9c99Zb7nA4RDSB55sF4XvHjvHxuKBWBwsLzSqVHIloYzGOMWYyRT//3L9q1RXKi8RiZ/DfP+M/nMMQ3/Pmzdu+ffvixYuJaM2aNZs2bYpEIpWVlStXrqyqqtJqtTNmzJg5cya+zgAMl4EXsof3jGVqBcZm4+677691dX6vNxIMVsry20T3EMmyHJLlbfH4HLU6HItxRJSfrxfFA+PHM2U2kmhUwjs4WnrfHMc99thjyR8nTJig3LjxxhtvvPFG7EeAYTfwQnb6oFcmfR14DV0QhIIClXIesqHh85aWgnD427HYR7JsJMol4og4IkZk5PlEXt4MQXilpGSS3R5atGiuIJSe3Awv3sHREt8AMMIGPn1H+qA/cqQ7zQDt3pQh201NF6rV4Wj0pljsPY4jojCRRNROxIgYkVaj+Xjy5OiUKXJ19SLlkhxJks6d6aIQ3wBwxoN+4KUVZfDf3/7W1tz8TUHIEUUi+guRiTEmCFdy3HuSFCR6gueL8/O7amoWl5VpUv/uOTVdFOIbAIbqlOMLdTop+Z/eu7Ti9weeeOLDujo/x5l1uuNEyzs79zGmisdljUZQqSzFxQ1+/wuMWR0OuuGGb2VlxQSBYrFwWZkGXWzENwAMXvqL14movDyrs7Olv5pGTc3uTZssweCtHMclEp6srJ08L/O8kbFOtVpbWvrZm29WGo0Gl8vvdAZlWSoqyhIEXq+X0dFGfAPAkHrc4bCytkEftRHlYQaDTEQXXmjs86Sl222Mx4WTyyMI0ag+L+8SWd5sMvmvvFL4+c9P1LXLy7MdDovL5Y9G/ShtI74BYBh63O3t9fn5OX3WRpSH6XRGjyfy9NNv79sX5TjzzJmqhx66Mjn/n90eVKtVosgYI52OZWfvKyyULrlEXrp0ttkspU4TiNI24hsgc3q+A7yW/bReZyCvn9rLttmy9Pq+x3s0NnoeeuittjZzPO5lTBaE7wmC5S9/6dJqP0rO/1ddPV0UP9yy5XnGrNOmaebNu95o1JeUZBMG/yG+ATK45zuQcXin+zoDef3Ugd5GI6U+Rhk90tjItbQ01dfHQqHbOC6XMY7oLaKdgvD1REJInf/PYjE/9dT1+/f7GbMSkSTJbncLx3GokCC+ATLNcF3imHxuIiE5ncFk93kgr59moHdNze6dO69zuzcHgz8QxS1E4xgLElmILIypiUilknrP/5c8HggCX15uKC9HcCO+ATLOsEzKmvo6brefsWzGLEp3W6ejU75+ahk6tbtdWOg4fLjVYAjG43qO4zguxBgj4okYz/tNJldurnvWLHV19RW9jwdOp8fliirzwUqSdObWQgPEN5yLhqvuPBRKzzcYpI6O7rw8q9PpG9yWJHvQHBcsLCxMdrcvvNB4ymsXOzu77r//gx07gsGgjuN8Wm0Zx2nC4R+0tvoZ+2ogsEOtJlFkWVnTwuG/SlK7wRC54Yacxx67vb8VywRBEAReWUUrFhtSUQgQ3wB9GK668xCPGeXl2U6nTxDOG8qWJHvQOp0UifDJ7nb6AR5KX/uvf3UfP16QSNzOmIqoKxr9WKXyCwIXj/PFxbpwuLu83NTS8lu73ZGVFf/Rj64TBPmUx5gzt1IPIL4BhjlilFxubpbD4e7SUmNjYyjZpzYYWDLv+jxmDOOWDGRqKr8/8MAD723c2BmPZxEd57jzibIZ4ziOGBOIDETNkiRzXCIWk6+5xvDEE7Pp5ESyRqPR5wud8hgzXEWh0f/9CfENcBYMLmL6i4zksOhIRFVXV5+ff15Liy8anRiNdpWWWtIn9XCF3UDiTJlD6p//VBF9m8hEFGTsP4hMRDJjAsdJKpVv3LiQ0fh8Xt5548fHb775/EEc7Ya4Us8Ac/msfH9CfAOcfYOLmP4iIzXRAgFtfj6JokBEsZhwyqQelmXJktsmSfKhQ0Gn011ebnI4LKFQePXqHVu3xny+dquVM5no2LHlRHuJsoj8RAJRGccFtNr/4LgsqzVy7bW2RYsWGI1FymtyXNcgjjFDvCpngLmMEg3iG85Rg4uY/iIjNd3MZpGItFopGiWNRjplUqfZkh79UMYoTbdU2Z7m5qAoWjmOIhGLy+V9+eW9b789t6vLLEl8MPi+SuUXhBhRiEgZRiIRNTkclvfeW5I8Fel0+vo7xvB8Qq/vPtODuAeYyyNTokF8A2R4ySU13WbPtjc0eIuKyOM5mpdn1eu9qUl93nkWJYIPH/bxPBeLqdOUCHr0QyVJPnJEG4sJGg0vy12TJo3rsW0eT+CPf9zZ1sY3NX0WjxeqVG3Z2TnxOE/EcRyn1LVVKrXVemFX11tEHkFonT27oLp6ZuowkjTHGJvNrCx4NhrqWsP1rQWIiGOMjcxfwmKJaOBwOd3TX5IkuVz9Pn4grUsuJtnU5GNMKC21EJFe/68SQXKT1Oq40xmKRnO6urqzs80GQ7csM1E8T3mYXt84f35B6sYT0Q9/+M6WLdcGg//L2PVEISILzz+t0SyPx8dJEs/z71utocLC42Vlk8aPD9588/kqVdZpnfcbmbcv/U7Gf9+gW4feN2SU0z39NfS5lpLVAFEUiITUO5XgdjrDjJkKC80NDaG2tpAoyvF4mSgGysqy29qasv/1x+X6er/Ho3nlld0ej6GgoOPnP79q3z4ukchhzEqkIpI5juO40vz814msXu9xk4m7/PKcX/zixuxsa+qh4uDB0EgG5SkPmZjQCsUTgNMIUxqp01/JyoBWKyW/ryolAuVYEo3qGDO3tHhjMU12dlZ7e5DjNDwfKCqyMWbiea8oClqt5HDoOzoi1dV/bW2drFKFm5u/VlOzgzE/Y3TyCkmJMaZWd86fX1RV9X9Orh9PHo/XbD4RoG1tnbm5pYIgjOTgDYwYQXwDDGeY0kid/nI4LIcPt9fXR2SZabWyLMeNRlJKH8rxQ6ORRJFEUdBoJI6j0lJjNGrSahOCwBcWsuef37pjR7vPRypVjkbTKopVjBXH48zne9ftNs6albVp03sqFYXDtRynNhr9CxfmVldf1tDAUo9SSoAmEtLhw2Gns7WoSE/ES1KUyDcCfXCMGEF8AwxPmI7w6S9BEFQqobi4VCkjeDyNgpDjcvkdDotyLCkqMjU3d/G8b+JEA2MsGhU++2zHL36xt7k5R5bdRCGiS4huiccpEvHxfJ1aXcRxJEkGuz1QXT1Tq93tdhvt9pLq6suSJyR1ui8NJlFC0+32M2aNx6mpiYhiEyYYIpHsEegOY8QI4htgeMJ0xL68J2u+TU1Bu90qCILb7RfF/Px8o1JGSB5LJk2SHA670gv2+wO33bbb6/0xkYaIEf2WKE6krGgjc5zFbPbHYlxp6WfKMu3J2bfTHKVcLn8kQqIo5OcburrcoZBGo4kWFo4fme4wRowgvgHGjOQ5yUTCQMSam3UtLe7p0wuVCkmyjKAcS/z+wOrVn2ze3N3a2kRk5nlTPK4mChNpiDiiPCIvESMijuOt1j0zZnB2e0jJ7gEepZQA1emCskznn1/Q3BzkOEE5WoxAdxhnJhHfAP12b0fbJBjJc5JuN0cUy8/P83ha3O4WiyWem1uWLCP4/YGVK199+20/0Q+JdhDZia6RJI7oONFOogVEjKiDqFmtfkalGjdhgv+VVxYVFOTV1/sbGphON9DKtRKgyQUny8vjjLF4vAvdYcQ3wNlMSTqTQxoGd4RInpNMJLSMqXheKC01lpRwF15odLl8jY2eVavebWrSEJmJuon+jWgcUfbJCyOJSMdxbsbWE7nHj4+99daSCRNKky+eHEjeZ6tTN1iZSyt1409eQ6TW6aTycuMgDniYNwrxDTAMRmBIQ59HiGSE2e3x7Gy+d4Qlz0m2trbFYpJKFZVlOnSo41e/OlBXFwwGPUR2ogoiC5GP6HUimShMJBPJRBzPi6Wl8rXX5lRXX9u7QpK+1akbrMyllbrxQz/gYRQg4htgGIzAkIY+szIZYeGwsbOzKXlZfLJDmjxZN2uWOhCIPvnk/+7eTaLokeWbiVxES4k+Jcom8hPJRBOJNhMliD7m+c/1eus3vmH82c8W9lfaTt/q3nNppd4/9AMeRgEivgGGwQgMaegzK3tEWO8OqSAIBkP0W9/6s8ulYcxK1EZ0K9F4ou3K5ZFE4ZOlEhXRbqJ8tfrYvfdedt11084/35q+IpG+1b3n0krd+KEf8DAK8NyKb8bYo48+evjwYY1Gs3r1amVRJSLaunVrbW2tSqW65ZZbKisrsUPhdA3XkIY09dw+s7JHhKWmeUdH5Ne/3vb6615Z9hOdT2QluopIJnqe6HtEeiIfkYmonOgtouNE3WZz/I47pk2dOq+kJJvnvYyR0+nrr7h8ytJz6gYrc2mlbvzQD3gYBXhuxfeWLVtisdirr766b9++NWvW1NbWElEikXjyySffeOMNrVa7ZMmSuXPn5uTkYJ/CCOsx60gkIvSo5/Z5hEhGmMEgFhZaXC6/xxN48cUdu3ZFOjpcicQSoiNEVxERkYnob0RXEhURGYg+J4qqVE9rNOOmTjV89avztdrxXq/bYDCLYqK/vnzqnz5l6bnHBpeXa4b3gIdRgOdWfO/Zs+fqq68moqlTpx44cEC50+VylZaWmkwmIrr00kt37dr19a9/HfsURiyyo1FBpYo1NQWDwfy2toTNZm1p6SopyR5IPTcZYTab2eVyr1v3yWuvNYTDS4h0RIxoO1EWESOSiDgiPVGC6KjB8NvS0vgf/nBDUdF4SZI++OCY0xmLx9t5Pt7RETAaI+edZzSZpPTFZZSeYUTjOxgMms0nTsKoVCpZlnmeT73TaDQGAgHsUDitKsGgJTuwLpevoUFfUGCWZa69PTR+/GlcxtLc3HrbbX89etQsSZ2M5RLlEo0nkomCRDqiEJGBqIOojegTg+HDZ5+dfdNNl6QeAObOLS4u9m3b1szzRiJRFDXt7fVTp5YpV0gmKzM9/i5KzzCi8W0ymUKhkHJbyW7lzmAwqNwZCoUslnRFtPQT1451md26QTfw8887dboSnY6IyOfrnDzZOlzb09ws63RGIjp+PGEyyRaL3mjUHD/enpcnFhaK5eWF6Q8V3d2Bn/xky7p19bHYeCIjUSXRNqLDRETEiIxE+4myiX5BlKtStV933SWLF8+YMoX13g8FBVaPhzvvvDLlR72+oaDAarOZnc5u5bjVe2NycgxpfovPJ1o3zPE9bdq0v//97/Pnz9+7d295eblyp8PhaGxs9Pv9Op1u165dK1asSPMKWK7hHGyg2x1h7MRRv6srkps7pL2U2peXZVkUVUQkioHsbD4WaxZFobCw48oriwVB6OwM935iR0dk/fo9e/dyRH69PtrWNjEW+79EKqJuop1ERiI70ftEPFGQ48T8/ERxsX3ixGk+XyQ727Zzp6ugYJzHo+u9YX5/NHKyOx2PR5V9lZurJiIivsfGKNL/Fp9PtG4gR6aBxve8efO2b9++ePFiIlqzZs2mTZsikUhlZeWqVavuvPNOxlhlZWVeXh6+zsDgqgTpyyy9T05qNO16vTcaFSZOjHOcsnpZ3OEoFgSh90vt2HFk5cq/NzXFib5DxNTqLI7byJiW44gxIhKI9EReQWjRat1GY95VV2mWLr3ZYCjascO9f39MpyvOy8sRhKyGBs9FF/Wx2bKc6Oxszs42aTRMEGL79/txNSOMACyWhuP/GWxgcgEtlSqWfonI5EXk9OVFyFJ/e+RIiDGzTuctKcmWJK/JRH3GfepLSVLjm28efeWVQ6HQV4isRDOJJI7rFIS3iPI47oZEQmasg+NeHD9e/sEPZpeUWIuL9bKsb2vrDIezjh3T/vOfUcayc3OjdjubMMG3cGFh7w2TJKmlxc9xQY0mriyk0Gcr8PlE685O7xtgEJKjO5xOXzCY3dwcjMXUDQ3H5s4t7nPB9d63U+9JLolARB0d3YLwpUvJkw9ubPQ89NDrra2CIBhzc71G43ei0RiR6eRVNhxjgkZjzc//IhZ7kch8xRWqFSsWCUIpETU2+r/4QiopsebmmnftcubnF3q9XlEUeD5QXGyZOFHX54YJglBSks1xHBExJvTXCoDhhfiGkRCNCs3NQVG0EpHfL7lc/h4Xqacvs/RYEkGvj+flWXtkaHNz6ze/+ecvvuATiXyiKNH3iLLD4dZx43aq1e2SpCKaQ/Q3IhXPfzxlSs4DDywxmcxEVFgout0R5VtoLCYkDzyFhYbiYsvEicaWFj/HceXlksNhPWV1KBiUm5uDsZhgsXReeKER9RNAfMPYptNJsZhyso60Wqn3hS3prwMsLTXW1dUHAlqzWZw9267RaJzOEyvRBIOBP/1pRySSs2PHbq93HNF3iTgiP9E2jruRSBuN6sePv66p6TVJOsrz1vz87h//eMGUKTkWSywa7VLGfnR2BpVX02gkjjtx8HA49ILgjUaFr3xFdjgK+gzi3pv9wQcNsViuVhvLzS11ubpwdQwgvmFsczgsDQ3H/H5Jq5UKCy06XVePakma6wATCamuzu3zZXd1dUtS9rZt7rlzi2027oEHNn7yCXV3H1eri+z2S7u7fUSkrGhzcvAfqVSJceP2FxezadMumD17ol6fVVpqoS8XplNnp/ryJNrZp+w7997sgoKc/HwL6ieA+IYMwRgVF5tcLh8RbzAkHA5r+gtbUtXX+32+3IMHw36/ed++rW1tvChuIQqK4rc5Lj8elzhuiyDsVKk8iQRTqttEWqIdBkPT1VcbFi684itfuYCIJElyu5s5Tu7dwWeMJEluagoTyQ6Hvr+ix0CuQhquwTYAiG8YFerr/bFYnjLRGc97Uzu8aWZNUgLun/+Mbtu24803nbJsIFpMlEskEL1EpBIEmec5xgzxOCspuf748edCoV8QjXc4ulavXmC12nU6SZKkWCzZUzaVl1v63LwjR7SimE9ER496BcHf51eBgUyT3btd/cU0Jt0GxDeMAallhGAwORsfpTm5l0hIf/zjrp//fGdnp47IQnQBkYZoPFGQyEJkJOIZI5WKI/Ll5LTPmeOvrv5R7/m1JUk65XEiGhWSJy1FUYhGuVO2or/CSO9ySn8xjZlPAPENY0BqSeH48e7m5txYTBAEqq8/Zrfn9K4eNDe3Vla+43IZiKxEPySKEOmIniOSiXgiRqRRq1/VaPRWqzR9uvbpp/tdGGEgs+vpdJJGw522e2UAABS3SURBVIsiEZFWK+l08ilbMfC5SvqLacx8AohvGNWU0kE4zLW319tsWUYjxWJ8LGYlouZmP5F+/Hhrslvq9wdWrdqycWMkFvMQLSXKJvqUSLnsnieaQLSZqJUoYTK1X3NN6be+9dXcXP3AC8f91TEcDossdx092qjUvh2OvuN+cNNk9xfTmHQbEN8wcinc53m23r/qNTmJjeMoPz9HGe/hdJ6Y5iwe59VqFgwGXnzxw1273OEwx5gcCCgr2gSIdhJdQhQm4k7O/HeASBw3LveGG/IeeuiG/rrbafRXxxAEYdKkcZMmneLpg5smu7+YxqTbgPiGM5vOvYPP6fQIAp98cO9MTL3n2LFjJ5dmOlE6KC3V7NhRHwppOzubDhxoWrNGTCRiREVEi4g+IjKdXO1XT6QlupDoN0T5Ol3rffddffXV/0ZEanV7W1uioeG0pxaJRoVEQnK7/aIo6HTBAT53iENEENOA+IYz6JSjIFKLti5XVFktT3lw79rul0/EyallBCLyer2//OXGri4NUQGRjegKojyiDUQ8Uejk8ggmogNEhy2W7quusn33uzM7OyW7vejk1kaKi0vp9Mds6HRSY6M/Gh1HRLJMLpd/IM/FEBFAfMPodcpREKkF3EQi3tTkE0VBq5WKisho7FnbTX2ww6FnrL2+PiJJ1Np6dP78PcGgQGQnWkokEBmJ3iOyEpmJGNEMjWZLIuElimZlBd56a+Hkyecrr+N0+iKR5Ibx6bc2TR3D6WzjOI1GIxUVmaJR/7DsHADENwxbreN0nXIURGoBV6eTAoFxRBSNksdz9OKLi3vUdlMfbLOpfvrTj7/4gn3+uSsYLCAyEJUSqYgEIvnkOmQ80RdEvzGbTRUV1gcf/IbBYKiv94dC3N/+1qCc8CwrMyZX7J04USeK6bY2bR3DEIkYT+u5qTtHrY6nWYMYAPENI/11/pSjIFLnFbbZsmOxrlhM0GikvDxr79quck9zc+vy5ZtdLksiEWBMisfvP1kYeZrIQiQRmYi6iQ4TfWQyjbvxxpyHH76qo4M1NLC2tmO5uWXNzUFRnCiKXrvdsm1bY0FBTnLzBj1mYxDjPVKfIkkMhRRAfMMZrHWcrlOeXquv9wcCVrfbL4rqQKBh6tSLBIEnIr3e2+ORfn+gpma3223csWNXMHhvIsFLEkf0Mscpk8tzRA6iAFEtUS6R75JLhBUrFkyaZNfppOPHE7FYHhH5/Xw0GlQupRFFwe32i2J+fr4xGZqDzs1BnEhMfcr+/X4UUgDxDWew1jHsJZdoVHC7T5z0MxrLOjoaUvvCqcG9eXPI77fk51/a3c2Iwhxn5DieMTdjjOc5WZaIPue4/JISYfXqr+Xnj29q8jEmMGaJROjYsUZlmIpWK4miRpngW6uVRFHQaKTREJq41gYQ33Bmax3DXnLR6SRRVJ+8TQUFORdfbPH7A48++qHbbbTbg9XV02tqdu/ceZ3XGxZFE9H7KpUYi3EaDS/LCb0+pNfXqFRleXkd69cvLioaf3Ihni6OCxYWFhKRJMktLeFoNKTRSDab/tAhp8mUEww2TJhQ0Nnpy80tGw2hiWttAPENZ7bWMewll5MzvgrKmA2dzkdETzzx4aZNlnhcUKtVovih12vjOE6tlmMxLh7Xl5ZO7+j4rVp9ns3Wvn79sqKi8X02QaeTIhGeiJqbg/n5NkGIiaJw6JDzZH2mSK/3XnJJscvlGw2hiUHcgPiGMfbFXxCE6dOtK1e+u21bPJHwyXKX1XphV1cD0Q9VqixRZHV16669Vt/YyAoKDK2tXXr9oYsu6vjOd26YOtWuXIfZ34CNZH+W57uLioqUXx09Ok6prdOpZgMHQHwDSi6n8Mwz/++DD+aLYh5jXUQfd3TMYEwi2qFSLeA4juPM1dXTa2recbuNF1zQuWTJTcpSZL2vw0yt23y5HK9PTvtnNoupRx28dwCI73NFn2cph9iHdbuNiYRS/haIDIzxgsBkWaXRBFQqadYstcVifvzxOUS0f7+fMXNqlaa/uk1qrGu1Ho3G43JFieSyMr1K5UmuTI83FADxfa4YyllKvz/wxBMf1tX5Oc48c6bqoYeutNnMRGS3B1WquCQRUYIozHESzxtNpj1XX80XFoarq69IU6Xpr26TGuVKWCuX40sSaTTeiy+2JBLKGU5cJgOA+D43nO5ZytTe+u9///82bbIEg7dyHPeXv3RptR/913/dSETV1dMDga2bNoVjsa5EotNs9o0b1/3oo5VlZZoeh4ceVZpEQpJl+dixY72nY+0R6703G/ONACC+R10p44w63bOU+/e7X3ih3uMx5OaG2tsT8XgWx3FElEgIbveJS8wtFvOzzy589lk6WR6xnszZrh6v1qNK43T6RNGmDOgWBK/SfGWfhELk8dTn5VkNBuZwWHovgIn5RgAQ32O1lDE4pzxLqVxi09jItbQ0FRY6jh51yvL3BUFwuxlRrVqtE0XGcZxKJdntoSEeHvqMYGWf8Dzl549Lrvvee7PT/CEs9QuIbxh1pYyhS3+W0u8PLFr0ZmPjBaJ4SJa/09wcjMe1KlXYZjNzHFdQMOGqqwJ1des4zjxrljq1qD3ww8Mps76f1S/7HVbY+w+hrgKIbzjjRtuV1jU1uxsbl4qiORbjeT4Sj6vU6lA8ThwnEcmTJ8tPPHH9UA4PA8n65D5JJKT9+1vMZodGIxUVWVwuX+orp/lDqKsA4hvOuLN4pXVyAinlunZlgTG326jRMFEkjgvLMqdSydnZlxuNfygrm2S3h6qrp4/AVwGHw3L4cHt9fcTtDolirsFgFEW+ubmrrGygQYzpRwDxPSCiKFZXV3u9XpPJ9OSTT2Zn/+u/cfXq1Z9++qnRaCSi2tpak8mE3TqUvurwBndyAqnGRlNNzTvKuGy7PZifr5ekrmh0ikbzXxdcMLWoyP/zny8a4OqRw1J0FgRBpRKKi0ujUb/bLR8/Hh4/3hSLCTpdfPQfFAHGUnz/6U9/Ki8vv+uuu955553a2toHH3ww+auDBw/+7ne/s1qt2JujRGpwi+LMeHw80fuFhdcmx5BUV08ner+lxWAw+G677RuntVg7DV/RWal4aLVSfr61vb2V4xIWS4fDUTyaD4oAYy++9+zZ853vfIeIZs6cWVtbm7yfMdbY2Pjwww97PJ6KiopbbrkF+/QsRrZSJInF4nv23HRy5r+/cNyN8bieMZYcQ5K8PHIosdv79ulSqh+FhZaWlq4JE6Ty8rjDUZw6rBCjSgAGE98bNmxYv3598sfc3FylKmI0GoPBYPL+cDhcVVV1xx13JBKJZcuWTZkypby8HLt15CmTtXIc19jIgsE/m80nZv5Tqy1qdZfF4pwxw19dfdmw/K3hKjonqx9f+YrscBQwRskLLCVJUtZtwKgSgNOO74qKioqKiuSP//7v/x4KhYgoFAqZzf+qkOr1+qqqKq1Wq9VqL7/88kOHDvUX38qF15nqrLfO67Wq1SfeTZ4PCQJfVGRuaenOynLdcEPskUeWZGUNcgsTCenzzzuVVC0vzxIEISfH4HR2n7yncChd44KCf9XcPv+8U6cr0emIiBoaGsrKjCebkzjTuzezP5z47zvXiyfTpk2rq6ubMmVKXV3dZZf9qxP3xRdf3HPPPRs3bkwkEnv27Ln55pv7ewWPJzD6d8fgvrPbbOaz3rpx43xOZ0JZkOyqq0xa7V/cbuNXvxqqrr7OYjHHYoPf/06nT6cr8flCRKrOzhalI5ybq8xdxXd2hoerCW53hLHQyVpQ1OcLnewidHs8/Bn95x8TH0408Nxs3TDE95IlS+6///7bbrtNo9H88pe/JKKXXnqptLR0zpw5N910U2VlpVqtXrRokcPhGNM7a9ReCaIs++vx5Nlsx9evv7bH0gdElJys1W4PVVfPHOAYkoGIRgWlR0xneHh1ak1m4kQdz2NUCUA6yuqxI2FMHCFTp/LguK6LLx5QcIzA8X/u3N8fOfIDpXN9/vm//eCDZSP2zSOl903Jq9vPhJPLp4306Ur0vtG6DO99nyNG7ZUgHk+eMocUx3EeT95IfvNwOCw+X2dXV+RMd4QxEBDgtCC+v2R4rwQZxtFvNtvxri6m9L5ttvYzsQH9DQcUBGHyZGtubgAfDwDE9+g1vB3AYaykr19/7fLlv/V48my29vXr552JDcA16ACIbzhFf5b6mYokjaKi8YOodyf/qCTJTmc4fTcc16ADIL7h1P3Z1KtsklORnLkNaG4OcpyJMWuabjhKzwBjC49dcOY4HBa93stxXXq9t0d/1u02Jk9FJqciOaVEQnI6ffv3+51OnyRJA98AnvcVFlr6/B4AAOh9Q9/9Wb8/UFOzt0edxG4PNjaeOBXZ53I2fTrdYnqyQ63TSZGI0Of3AABA7xv6ptRJmppm79x5fU3NbuXO6urpM2a8U1KybcaMdwY+Fcmg54pK8z0AAND7hr71WScZ3Jx/gx4cgro2AOIb+pZmJMng6iT9daIxOAQAEN/DKc1Iki/PRjKkKVvRiQYAxPcwSzOSZIhrIwAA9AmnLoeH3R5UJv8aeoUEAAC975EzjBWSgcBaYgCA+E5n4Je2j3CFZNTOSw4AIwbFk3T6HLI9GgzXYsEAgPjOTIO7tH0EpA76xlWUAIhv6GnUnpDEVZQAgNp3OiN8QnLgMAAcABDf6WDINgAgvkfOWR9Uh1F9ADACMrD2rQyqY8waiYxzufzn4AYAAHrfY1I0KgSDgVde2e3xGHJzvU89deUplyIb9g3o8zYAAHrf6eh00iuv7N63b35r66wDB24Y+fHaGNUHAIjvwXA4LJ2dAs/LPJ/Q6VQjP14bo/oAYARkYPFEEITy8pjXyw/LFNuD3QCM6gMAxPfpG7XjtQEAEN/pYLw2AGQ8XDQPAHAOxPfmzZvvvffeHne+9tprt9xyy+LFi7dt24YdCgAwMk6jeLJ69ert27dPnjw59c6Ojo6XX375zTffjEajS5YsufLKK9VqNXYrAMAo6n1Pmzbt0Ucf7XHn/v37L730UpVKZTKZysrKDh8+jH0KAHA2e98bNmxYv3598sc1a9YsWLDgk08+6fGwYDBoNp+4ptFgMAQCAexTAICzGd8VFRUVFRWnfL7JZAoGg8rtUChksfR7lYrNZs7g/ZjZrcPbhwaidWMpvgfo4osvfvbZZ2OxmCiK9fX1559/fn+P9HgCGfzpyeDWZXwD8fahdWP0yDT4+H7ppZdKS0vnzJlTVVV12223McZWrlyp0WjwjQYAYARwymJgIwDHfzQQrUMD0bph7H3jsh0AgDEJ8Q0AgPgGAADENwAAIL4BABDfAACA+AYAAMQ3AADiGwAAEN8AAID4BgAAxDcAAOIbAAAQ3wAAgPgGAEB8AwAA4hsAABDfAACIbwAAQHwDAADiGwAAEN8AAIhvAABAfAMAAOIbAADxDQAAiG8AAEB8AwAgvgEAYGxQndajN2/e/N577/3yl79MvXP16tWffvqp0WgkotraWpPJhN0KADCK4nv16tXbt2+fPHlyj/sPHjz4u9/9zmq1Ym8CAIyY0yieTJs27dFHH+1xJ2OssbHx4YcfXrJkyeuvv44dCgBwlnvfGzZsWL9+ffLHNWvWLFiw4JNPPunxsHA4XFVVdccddyQSiWXLlk2ZMqW8vBy7FQDgrMV3RUVFRUXFKZ+v1+urqqq0Wq1Wq7388ssPHTqE+AYAOJvxPUBffPHFPffcs3HjxkQisWfPnptvvrm/R9ps5gzej5ndOrx9aCBal1Hx/dJLL5WWls6ZM+emm26qrKxUq9WLFi1yOBz9Pd7jCWTwpyeDW5fxDcTbh9aN0SMTxxgbme3ABwgNROvQQLRuGOMbl+0AAIxJiG8AAMQ3AAAgvgEAAPENAID4BgAAxDcAACC+AQAQ3wAAgPgGAADENwAAIL4BABDfAACA+AYAAMQ3AADiGwAAEN8AAID4BgBAfAMAAOIbAADOiJFbqhgAAND7BgBAfAMAAOIbAAAQ3wAAgPgGAEB8AwDAuRLfkUjkhz/84dKlS++888729vYM24nBYPD73/9+VVXV4sWL9+7dm6mflc2bN997772Z1CLG2COPPLJ48eJly5YdO3YsI9+1ffv2VVVVZWTTEonEfffdd/vtt996661bt27NsNbJsvyTn/xkyZIlt99++9GjR89mfL/22msXXXTRK6+8csMNN7zwwgsZtqP/+7//+4orrnj55ZfXrFnzs5/9LCP/VVavXv2rX/0qwxq1ZcuWWCz26quv3nvvvWvWrMm8d23dunUPPfRQPB7PyM/k22+/nZ2d/Yc//OGFF154/PHHM6x1W7du5TjuT3/609133/3MM8/0+RjVyGzK8uXLleuD3G53VlZWhu3oO+64Q6PRKN0BrVabkf8q06ZNmzdv3v/8z/9kUqP27Nlz9dVXE9HUqVMPHDiQee9aaWnp2rVr77vvvoz8TC5YsGD+/PlKR1WlUmVY66655pqvfe1rRNTS0tJfZp6pNm/YsGH9+vXJH9esWXPRRRctX778yJEjL7744ljfs322zuPx3HfffQ8++GAGfHR6N3DBggWffPJJhv2HBINBs9l84j9BpZJlmecz6mzQvHnzWlpaKEPp9XrlTbz77rvvueeezGsgz/MPPPDAli1bfvOb3/T9CDayXC7XNddcwzLOoUOHFi5c+NFHH7HM9Y9//GPlypWZ1KI1a9a8++67yu1Zs2Zl5LvW3Nz8zW9+M1M/k263++abb37jjTcy+P+uo6Njzpw5kUik969GqK/x/PPPb9y4kYgMBoMgCBl2kDx69OiPf/zjp59++qqrriIYUxWhuro6Itq7d295eXmmNjNT5zXq6OhYsWJFdXX1okWLMq91GzdufP7554lIq9XyPN/n98IRKhjdcsst999//4YNG5QuT4bt6GeeeSYWi61evZoxZrFY1q5di2QcK7WF7du3L168WCkQZWozOY7LyHY999xzfr+/trZ27dq1HMetW7dOOQWVGa699tpVq1YtXbo0kUg8+OCDfTYNMw4CAIxJuGwHAADxDQAAiG8AAEB8AwAgvgEAAPENAACIbwAAxDcAAIxu/x/5dvc1GmVZiAAAAABJRU5ErkJggg==" />

The light points are the original data, while the dark points are the projected version. This makes clear what a PCA dimensionality reduction means: the information along the least important principal axis or axes is removed, leaving only the component(s) of the data with the highest variance. The fraction of variance that is cut out (proportional to the spread of points about the line formed in Figure 5-83) is roughly a measure of how much “information” is discarded in this reduction of dimensionality.

This reduced-dimension dataset is in some senses “good enough” to encode the most important relationships between the points: despite reducing the dimension of the data by 50%, the overall relationship between the data points is mostly preserved.

# PCA for visualization: Handwritten digits

The usefulness of the dimensionality reduction may not be entirely apparent in only two dimensions, but becomes much more clear when we look at high-dimensional data. To see this, let’s take a quick look at the application of PCA to the digits data we saw in “In-Depth: Decision Trees and Random Forests” on page 421.

We start by loading the data:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Output:<br>`
(1797, 64)
        `

Recall that the data consists of 8×8 pixel images, meaning that they are 64-dimensional. To gain some intuition into the relationships between these points, we can use PCA to project them to a more manageable number of dimensions, say two:

In [ ]:
pca = PCA(2) # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)

Output:<br>`
(1797, 64)
(1797, 2)
        `

We can now plot the first two principal components of each point to learn about the data (Figure 5-84):

In [ ]:
%matplotlib notebook
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

> **Figure 5-84: _PCA applied to the handwritten digits data_**<br><img width="500" src="data:image/png;base64, ..." />

Recall what these components mean: the full data is a 64-dimensional point cloud, and these points are the projection of each data point along the directions with the largest variance. Essentially, we have found the optimal stretch and rotation in 64-dimensional space that allows us to see the layout of the digits in two dimensions, and have done this in an unsupervised manner - that is, without reference to the labels.

# What do the components mean?

We can go a bit further here, and begin to ask what the reduced dimensions mean. This meaning can be understood in terms of combinations of basis vectors. For example, each image in the training set is defined by a collection of 64 pixel values, which we will call the vector x:

$$ x = \left[ x_1, x_2, x_3, \cdots, x_{64} \right] $$

One way we can think about this is in terms of a pixel basis. That is, to construct the image, we multiply each element of the vector by the pixel it describes, and then add the results together to build the image:

$$ \text{image}(x) = x_1 \dot ( \text{pixel}_1 ) + x_2 \dot ( \text{pixel}_2 ) + x_3 \dot ( \text{pixel}_3 ) + \cdots + x_{64} \dot ( \text{pixel}_{64} ) $$

One way we might imagine reducing the dimension of this data is to zero out all but a few of these basis vectors. For example, if we use only the first eight pixels, we get an eight-dimensional projection of the data (Figure 5-85), but it is not very reflective of the whole image: we’ve thrown out nearly 90% of the pixels!

> **Figure 5-85: _A naive dimensionality reduction achieved by discarding pixels_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAA2gAAACUCAIAAABpzJ2VAAAACXBIWXMAAAsSAAALEgHS3X78AAAVo0lEQVR42u3de1BU9f/H8c/KgisEEyCKQ4giBSlgGkjkJCrBeKk0zLToIjaThaWONpk1TRcrv0VXQc2pyUrBqFHLJjRvuJmpuDhGWxGgYKDCrCZCXGPd3x9nhvHHRfcgu5798Hz8ta7nuPva9+GcF2d3jzqbzSZcwRtvvGEymYQQZWVlwcHB/fv31+l0ubm5Hh4eAgAAwCna2tomTpx46623fvLJJ30wvs5VimO7xMTEzMzMkSNHsu0CAAAn27Fjx5YtW/74449NmzaFhob2tfj9XO4Z22y29rIbFRW1ZMmSqVOnms3miIiI2tpa5f722/n5+Q8++GBKSsrDDz98/PhxNvcrq6mpycrKMhqNs2bNam1tJRrpiOa0dE1NTUrApqYmBkc60mlZTk5OUlLStGnTPv/8cyFEQUHBAw88sHDhwvvuu2/OnDknT54UQqxYsWLFihVz5sxJTk5+9dVXrVbr5aXl999/N5lMc+bMmTFjxgMPPHDgwAEhRFZW1ty5c202m8ViueuuuwoKCjTdw1zIpEmTzGazcjs8PHz79u3K7YiIiAsXLlx+u6Ki4p577qmtrbXZbKWlpePHj29qarKhG42NjbNmzfrnn39sNltDQ0NdXd2PP/748ccfyxetoqJi165dmZmZ7RuSTOmqqqp27NiRlZUlQboO0ZQ7V61aJeVPXFVV1YwZM15++eVz587JN7gdO3YYjcYvvvhCvsHt37//oYceeuKJJxYsWCBfutra2v379+/fv7+oqIijpKK0tDQ6Orqurq6oqOi2226rra09cuTIyJEjCwsLbTbb5s2bU1JSbDbbCy+8cP/99yu/ED7yyCObNm26vLRcuHDhzjvvVF7V0tLSuLi4qqoqq9X6yCOPrF+/ft68eevXr9fsK9DP1Yv/7bff3t6AO/zVwYMHz507N2/evJkzZz733HN6vf7UqVP8qtSdvLy8yMhIX19fIYSnp6e3t/eoUaP+++8/+aLl5+cPHjx43rx5n332mXzpjh07duONN4aGhlZUVEgWTQhRWVlZXFws5U+cECIzM/P111/39/eXLNrhw4cHDx48YcKExx57TL7BBQUF5eTkrFq1Kj09Xb50JpNp4MCBCQkJJSUlHCUVX331VUJCgre3d1RUVFBQUG5urhAiPDx87NixQohZs2YVFxdfvHhRCJGSkmIwGNzd3WfMmPHzzz9fXlp+/fXXkJCQqKgoIURYWNjYsWMLCgr69euXkZHx6aef9uvX78knn9TsK6B39REqO9zLu2N717l06VJ8fPz777+v/LG6unrw4MFs9N1pa2sbNmyYcrukpGTo0KGyRps7d67BYDhx4sRNN90kX7qkpKQzZ84YjcalS5dKFi04OLiqqmrIkCFSbpbe3t6//PKLyWSKiooKCwuTKdoPP/wQHx9fV1dnMBji4uKk3FWaTKapU6fKt1nGxcWlpaXdfPPNEuxPekVTU9O3335rMBgSExOVk7LZ2dmRkZFubm7tPcRmsyl/vPzO9ttKaelwquvSpUttbW1CiNOnT/fv3//UqVP19fXe3t7afBH6STNOf39/s9kshNi1a5dyzx133HHw4EHl0wZGo3HGjBktLS1s992ZPn36+fPn8/Pzd+3aVV1dbTAY5I62Z8+ep556Ssp0oaGhycnJmZmZkkUrKSmJjo6WdbMcMmTI7NmzZ86c+emnn0oTbffu3WfPnnV3d4+IiEhISMjJyZHyJ66mpkY56suX7uTJk88991xgYOCGDRs4Sgohtm/f7ufn9/PPP+/du3ffvn179uxpbGw8f/78n3/+qZyUzc3NHTt27A033CCEyMvLa21tbWlp2bZt2+TJky//d0aPHl1eXv7bb78JIUpLSwsLC+Pi4urq6p5//vl33nln+vTpL774omZfBNc746jT6bq8/dJLL7322ms+Pj7jx48PCAgQQoSFhb3++uvK70lubm7r1q2TqQz1uhtuuGHZsmV9JNq+fftSU1Nramraf7eWJl1GRoby/ogEb1V3iJaXl2exWM6cOXPkyBEJTlx1SLdx48bY2Fh/f3+Xu9LFVaNVV1crofR6vZT7k4MHD0pzIrxDug8//HDJkiVxcXGrV6/mKCmE+Oqrr9LS0tr/6O3t/eijj37xxReDBg364IMPqqqqBg4c+M477yh/O2DAgNTU1Lq6uilTpqSkpFxeWnx9fT/66KOVK1c2NTW5ubmtWrVq6NChixcvnjRpUnx8fExMzOzZszdv3vzQQw9psYZJsJOCIzQ0NOTm5hYUFCxbtuzmm2+WKdru3bvXr1/v4+MTGxv79NNPSza448ePnz9//sSJE5MmTZJscEKIysrK5cuXL1iwICEhQb5oZWVlVVVVkydPDgoKkilaY2Njbm6uj49PWFjY6NGj5dtbKqU/IiJCvmjFxcW//vprYGCgl5dXTEwMR8YuFRQUrFy58vvvv7/8zhUrVtxyyy2Xt0xpUBwBAAAojhRHAAAA9B59c3Oz2WwOCAho/8oPri+r1WqxWCIjI6/6iUzXmp39uaRPx+AYHOmIxuAkm1rfKSd6s9mcmprKK6I12dnZV/1AiSvOzp5c0qdjcAyOdERjcFJOrS+UE73yBeTs7OzAwECHPlgPLoWj9r8P78FlLJKTk9Wu0v6FKQeprq5OTU1V5nJlyjJ///13hytBlJeXa3Brsz/X5ek6b5nDhw/vcvnrm1pVOrU/dF1Gdlre3hqcKk6L7NDBXffI2knX65E1Hu0aI7toOnsi6/X6oUOH2n+A02Y5UXuxszVr1qh9iOnTp6tdJSsryznHAr1yEjgwMNDRF0Nubm5Wu4pyJST7Xbp0Se1D9OACPc65arQ9J+eVZdra2joURy1f19r+Nx262zK7u16aFlLbmU7tD12XkZ2c99oHp4qTIztocBqJrIV0Doqs2Wi9Etnl0tkf2f4DnDbLiZeX17W/CFfWv39/zZYTeS4ADgAAAIeiOAIAAIDiCAAAAIojAAAAKI5Qp7y83Pb/yZ3X1o2+FrkPTpnIRCayq0fW5kU/QHEEAAAAxREAAAAURwAAAFAcAQAAQHEEAAAAxREAAACgOAIAAIDiCAAAAAfQO+2R0tPT1a6yYcMGVcuvWbNG7UO8//77alfZu3ev2lUSExPZ1AAA0BqDwaB2lTfffNOhy2scZxwBAABAcQQAAADFEQAAABRHAAAAUBwBAABAcQQAAADFEQAAAKA4AgAAgOIIAAAAiiMAAAAojgAAAKA4AgAAgOIIAAAAiiMAAABAcQQAAADFEQAAAL1J37PVamtr1a6yYcMGtassXbpU1fLp6elqH+Kff/5Ru8qhQ4fUrpKYmMimBgAAXB1nHAEAAEBxBAAAAMURAAAAFEcAAABQHAEAAEBxBAAAAMURAAAAoDgCAACA4ggAAACKIwAAACiOAAAAoDgCAACA4ggAAACKIwAAAEBxBAAAAMURAAAAvUnfs9UMBoMTntyTTz7p6Ifw8/NjIwAAALAHZxwBAABAcQQAAADFEQAAABRHAAAAUBwBAABAcQQAAADFEQAAAOjpdRyhBVarVQhRXV3tEs9WeZ7Kc+7j6RgcgyMd0RgcKI5wNovFIoRITU11reccEhLSx9MxOAZHOqIxOA7iFEc4W2RkZHZ2dkBAgJubm/afrdVqtVgskZGRpGNwDI50RGNwoDjC2QwGQ0xMjAs9YVW/YkqcjsFJP7ijR49GR0frdLqioqLr+2qwWbpiNAYHiiOujz179pSVlfXr12/QoEEzZ860Z4GrrkI65yguLv7uu++WL19uZ3YXirZly5aamhp3d/fhw4fffffdcqTrMK/ly5efOXPG39//tddek2CCnbdGybZPdpWumw4UR/Saf//9d+3atVu3bhVCzJkzJyEhwdfX98oLuLu7X3kVh9q5c+eUKVNIJ4TYsGFDYWGht7e3ndldKFpJScnWrVuzs7OFEGlpaRMmTPDw8HD1dJ3n9fTTT991113dvXuokYx2Dq5zOpfYPu3fLNlVaiodtI/L8Ujr6NGjYWFhyu3w8PAjR45cdYGrruJQZWVlpFOkpaUlJiban92Foh04cOCmm25Sbvv5+RUWFkqQrvO89Hp9YGBgd58500hGOwfXOZ1LbJ/2b5bsKjWVDtrHGUdpVVdX+/j4KLd9fHxOnTp11QV8fHyuvArptJndhaJ5enq2tbUpt1tbW0+ePBkfHy/f4IqKimw2W21tbUhISOeOJfHG6Yrp2FVKvDXCETjjKK26urr2NwHd3d0bGxuvusAVVmltbb183ZqamqysLKPROGvWrA5/RTrnZ3ehaMnJyX///bcQoqGhoby8vKGhQcrBzZ49OyUlZf78+atXr66vr3dcRqvVqizQ1NTE9snOhF0lnKCHZxz5FUT7vLy8amtrldvNzc0BAQFXXcDDw6O7VZKTkz/66KPRo0crh6iFCxd+8sknvr6+sbGxHh4e9fX1hw4dKi8vX7BggaoneeLEiW+//Van09lstmPHjrW2ttpsNp1Od/vttyckJGgk3alTp0pKSv76669JkyaNGjXKCenUTrbH0U6fPv3bb7+dOHFi4sSJzonm7++/atWqr7/+euDAgbfccoufn5/j0inL/O9//3vhhRecs1m2i4iIaD9hc+TIkQ7fAerdCS5cuDA6Onrx4sUDBgy4Xtuk4ya4c+dOT0/PioqKxx57zEHpnLkzMRqN69ev9/T01Ov1H3/8sWS7yosXLx4/flwI4efnFxUVxVGY4ggXExwcbDabldu1tbWda0HnBby9vbtbZePGje0fTcvLy4uMjFQ+Lu3p6SmE8Pb2HjVqVElJidonOWLEiGXLlim3s7KynnnmGQ2my8/PHzt2bHx8/CuvvPLee+85IZ3a7D2OduzYsYCAgNDQ0IqKClXF8VqihYWFKZ+gWrNmzeLFix2XTghRWVlZXFzstM1SsX37dqPRqGwqjY2NnT/p2LsZMzMzg4ODr+826aB0hw8fHjx48JgxYyZMmOC4dM7cmQQFBeXk5FgslrNnz8q3qzSZTIGBgaNGjdqyZQvFUWK8VS2tcePG/f7778rtP/74Q/kkWWVlpc1m626BLldp37nodDrldltb27Bhw5TbJSUlzc3NcqebN29edHR0TU1N+x7T+RwU7d577x00aFBRUVFycrJzgpw+ffq+++5TzqAEBQUpV3Rz3GZZVVU1ZMgQJw8rKCho7ty5Smu8cOHCHXfc4biMOp3u8OHD27ZtU/WFCZfYPpuamvbu3Xv27Fmj0ejQ72c4c2ei/MpkMpmio6Pl21XGxcW9+uqrL774Ym+dvYY2uS1duvTLL798/PHH2z8Ma4/z58+rfaSsrCy1q6j9bXjgwIFqH+Lo0aNOeJVjY2PtX7iurq4HE+nM3d3dy8tr3759BQUFEydOHDNmjBBC6UDKWw+dF+hylc6GDRt24MCBlpaW0tLS5ubmESNGCCHq6+uLi4vHjRvX4ydcUFBg/+pOTieE+Oabb5544gl3d3cnpNu0adP27duLi4vr6+tHjhzp4eHhuGi+vr5eXl65ubkdvqTioGgDBgy4cOFCWVnZL7/8snz5coPB4LjBHT9+fOTIkT/99FOXV4vsxXQd5jVs2LDCwsJDhw7t2rVr0aJFyu8bDsqonOyPiIjIyMhQG9POwXXeGp2zfYaFheXn5ycmJo4ZMyYjI2Pq1KkO2iydvDOpqamprKwMDw+Xb1epHAIuXrxoMpnGjx9/jcfN3joUore0T0RXWVmZmJi4d+9eVWdT/vrrL7UP2f6hH/upfY+pBz+Ka9eudcLLnZ6ebv/CVVVVPZjIdXf69Olt27ZdyztfeXl506ZN02a6ffv2jRs37ty5c+2/YcuRLiMjIyUlpbW1dd26datXr5ZscHl5eR4eHhs3bkxPT4+Li5NvsxRCbNy4MTY21t/f/91333377bdliiaEyM3NjYmJUd6lVfUpES2n27p165AhQ3r8e5qWo3344YdLliwRQqxevXrRokXX+K+56KFQYu0T4a1q9IKGhoYff/zRbDaXlpb2+B/R7DFs9+7da9euXbRo0Y4dOyRLl5SUVFFRceDAgWeffVa+wU2bNi08PPy///67lk9TaLxaTZw48ezZszt37uzBcVrj0YQQ9957708//bRlyxZ7vhnjKukaGhqu8Xramo02ZcqU3Nxco9F45513cliUGF+OQS/w8vKaP3/+/PnzpUyXlJSUlJQkZbTbbrtNCHGFizm7uuDg4JycHIl/9IKDg+3/ZozL8fT0TEtLkyzUo48+Kuu8IiIievDWIlwOZxwBAABgF844urDm5maz2dzd/4erNVar1WKxREZGKt+H6MvpGByDIx3RGBwHcYojnM1sNqemprrWc87Ozo6Jienj6RgcgyMd0RgcB3GKI5xNuZhCdnZ2YGCg9p9tdXV1ampq5/+3oA+mUxtt+PDhne8sLy+XY3D//vtv5zu7vHqwIyI7dHAtLS2d78zMzOxy4TVr1vR6ZI2k6zKaEGL69Omd77Tzwm0aH1yX0Vwu3XUfHCiO6GXKGxOBgYEudLUC+99MkTid2mhtbW2d73Tyy+K4wdXV1V33yA4aXJff5vby8nLylK97ui6jCSH69+9/jZE1O7guo7lcuus+OGgTX44BAAAAxREAAAAURwAAADgZn3EEtM5ms0mcrsv/iFaOyF1ecOTNN9/scuHu7pcgncTRGBwojvYKCQlxwpPr8lPzV9CD/1jMZDKpXWXlypVsNwAAoA/irWoAAABQHAEAAEBxBAAAAMURAAAAFEcAAABQHAEAAEBxBAAAACiOAAAAoDgCAACA4ggAAACKIwAAACiOAAAAoDgCAACA4ggAAABQHAEAAEBxBAAAQG/S92w1g8GgdpWZM2eqXeWtt95StXxoaKjah/D19VW7SlBQENsNAADogzjjCAAAAIojAAAAKI4AAACgOAIAAIDiCAAAAIojAAAAKI4AAAAAxREAAAAURwAAAFAcAQAAQHEEAAAAxREAAAAURwAAAFAcAQAAAIojAAAAKI4AAADoTXqnPdLmzZvVrvLSSy+pWv7w4cNqH+Lrr79mIwAAALAHZxwBAABAcQQAAADFEQAAABRHAAAAUBwBAABAcQQAAADFEQAAAKA4AgAAgOIIAAAAiiMAAAAojgAAAKA4AgAAgOIIAAAAiiMAAABAcQQAAICd9FarVQhRXV3t6EdqaWlRu0pjY6Oq5ZUsqvQguM1mc+gLpTwle7I4bXZOziV9OgbH4EhHNAYn2dSk1z47vcViEUKkpqb2zRfi4Ycf1uYTs1gsISEhV13G5WZnTy7p0zE4Bkc6ojE4KacmPYvFomtqajKbzQEBAW5ubrwiWmC1Wi0WS2RkpMFguPKSzc3NLjQ7+3NJn47BMTjSEY3BSTa1vlNO/g+Hz5cutVx04QAAAABJRU5ErkJggg==" />

The upper row of panels shows the individual pixels, and the lower row shows the cumulative contribution of these pixels to the construction of the image. Using only eight of the pixel-basis components, we can only construct a small portion of the 64-pixel image. Were we to continue this sequence and use all 64 pixels, we would recover the original image.

But the pixel-wise representation is not the only choice of basis. We can also use other basis functions, which each contain some predefined contribution from each pixel, and write something like:

$$ \text{image}(x) = x_1 \dot ( \text{basis}_1 ) + x_2 \dot ( \text{basis}_2 ) + x_3 \dot ( \text{basis}_3 ) + \cdots $$

PCA can be thought of as a process of choosing optimal basis functions, such that adding together just the first few of them is enough to suitably reconstruct the bulk of the elements in the dataset. The principal components, which act as the low-dimensional representation of our data, are simply the coefficients that multiply each of the elements in this series. Figure 5-86 is a similar depiction of reconstructing this digit using the mean plus the first eight PCA basis functions.

> **Figure 5-86: _A more sophisticated dimensionality reduction achieved by discarding the least important principal components (compare to Figure 5-85)_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAA2oAAACRCAIAAAA99NwbAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2deVxU1f//D6sIooKCKOIuoiwqYrilmKIm7kuamguWWma2p2ahaVmWuZtLaSkolEtaroBIbqjYwxQ3FkUB2RcBAUGY7x/395jffO5535k7wyAz+Hr+BS8OZ877nnPPfc+957yuiUKhYAAYD5mZmX/88Yenp+eGDRv27dtnaWmJWBAIYtEYS3BwsJmZmaWlZWlpaf369dEviAWBgGqhAC8GlZWVU6ZMEX7+8ssvk5KSjDGKkpKS8ePH5+XlKRSKJ0+eFBYWnjx5cuvWrXUgluTk5FOnTm3cuDEuLs6oA0lNTT1+/PimTZuMLhA+FkFctWpVHRhgqampo0eP/uKLL3JycupAvxw/fjw6Ovq3334z9ljOnDnz+uuvz549e+7cuUYdSEFBwZkzZ86cOXP9+nVccF8EzA02r125cmVsbCxjLDEx0cXFpV69eiYmJmFhYfh+oxvXrl1r1aqV8HNsbOzy5cuNMYpjx455eHjY2dkxxqytrRlj7u7u8fHxdSCWqKgob2/v3r17BwUFrVmzxngDiYyMdHBwaNeuXXJysru7u7EPsJSUlDt37tSBAZafn79x40YXF5c6EEtMTEyzZs26d+/ev39/Y4/F2dl579692dnZ6enpRh1IZGSkk5OTu7v7gQMHPD09X8CL7LNnz/z8/Dp37rxjx44XIV7DTR+XLl0q/DBo0KA1a9Z06dIFKWB1OHv2bJ8+fRhjd+/ebd++vfGen23atBF+jo+PVybEdSCWyZMnW1lZJSUltWzZ0qgD8ff3f/ToUXR09IcffmjsneLi4pKamtq8efM6MMBsbW0vXLgQGxvr6enZoUMHo47l6NGjvXv3LiwstLKy8vX1rQOTWGxs7KuvvmrUgfj6+s6aNatjx47GeOLrhfDwcDc3t5s3b967d69du3Z1Pl5Tw2+icJtU+NnT0/P9999/9dVX4+Li3NzcCgoKBF35c1RU1GuvvTZu3LgpU6Zcu3YNWaOSc+fOde7cmTEWHR3t6+t7+vRpY4wiICAgNzc3Kirq1KlTGRkZVlZWxtsjZCwRERHz5s0z9kDatWs3ZMiQjRs3GnunxMfHe3l51Y0B1rx584kTJ44ZM+bnn3823ljCw8PT09MtLCzc3NwGDBiwd+/eOnDiZ2ZmPnv2zNgDuXfv3scff+zk5LRr164X8yK7d+9ef3//4cOH//rrr4yxy5cvT5gwYf78+aNGjZo0adK9e/cYY4sXL168ePGkSZOGDBmybNmyyspK1cTm5s2bsbGxkyZNGj169IQJE86ePcsY27Rp0+TJkxUKRXZ29ssvv3z58mXDSs4MmYEDByoXUXXq1OnIkSPCz25ubvn5+ao/JycnjxgxoqCgQKFQJCQk9O3bt7S0FAsUFApFXl5e7969t27dGhUVtWvXro0bN549e7ZuhJaamrpx48a6EUtkZGRRUdH9+/eNOorVq1cnJibeunVrwYIFxt4jR48eDQ8Pnz59ekxMjLHHsnv37tu3b2dlZX366afGHktoaGhiYqJCofjwww/rwIl/4MCBCxcuGHsUa9euFX5Yv379C3iRTUhI8PLyKiwsvH79erdu3QoKCi5dutSlS5erV68qFIp9+/aNGzdOoVAsWrRo7NixpaWl5eXl06ZNCw4OVk1s8vPz+/TpIyweTUhI8PX1TU1NraysnDZt2rZt22bOnLlt2zasfdSdHj16KBNf0Z/Onz+fk5Mzc+ZM4U/m5uYPHjzo1KkTbj2eP39+4sSJc+fOrWNxPXny5OTJk3FxcQkJCR07djT2Bx/btm0LDg7u2bPn22+/bbyB+Pv7JycnJyUlLViwwNgH2PDhw1NSUioqKsrKyow9Fj8/v8TExCtXrrz33nvGHsvIkSPDwsKuXbs2ffr0ujGPCSsIjZphw4aFhYU5OTkJq6ReNEJDQwcMGGBra+vp6ens7BwWFtatW7dOnTp5e3szxsaPH79ixYrHjx8zxsaNGyc8axo9enRkZOTUqVOVic1///3XunVrYeVohw4dvL29L1++PHbs2O+//37UqFGenp5z5swxnJCNL30U1rOrZpAVFRXCr1VVVb179/7xxx+FXzMyMpo1a4bckTF27dq1MWPG1L24bGxsAgMDAwMD60As/v7+/v7+dSCQbt26McYGDRpUN8aYi4uLMT4hJQMx0n0z5FVg1qxZdWYee+ONN+pAFG5ubm5ubi/mFba0tPTPP/+0srIaNGiQsA89JCTEw8PDzMxMmasoFArhV1VR+bOQ2IhuilVVVQmrGtLS0urVq/fgwYOioiJbW1sDidrUeDusSZMmcXFxjLFTp04JSq9evc6fPy+sMIiOjh49evTTp0+ROzLGli5d6uHhgeMAAAAA6JcjR47Y29ufO3cuMjLy9OnTERERJSUlubm5t2/fFoxBwsLCvL29GzRowBg7duxYeXn506dPDx069Morr6jW07Vr1/v379+4cYMxlpCQcPXqVV9f38LCwk8//XT16tUBAQFLliwxnKiN4O6jiYkJ+fPnn3++fPnyhg0b9u3b18HBgTHWoUOHr776Stj2ZWZm9tNPPxnv1oqysrK4uDgHBwfltxODpbKyMjs728PDQ+poG0ssGgNBLBhgiAWx4MQ3nFgMhNDQUNXb4ba2tm+88cZvv/3m6Oi4du3a1NTUpk2brl69Wvhr/fr1p06dWlhYOGzYsHHjxqkmNnZ2duvXr1+xYkVpaamZmdmqVatatWq1cOHCgQMH9u7d28fHZ+LEifv27Xv99dcNIjdT4K0zBklsbKywJMJYCAkJ8fHxqQOxqAkEsWCAIRbEghPfcGIxZC5fvrxixYq//vpLVVy8eLGrq2vdWHphzoBBItxPDQkJcXJyUtWrqqr4widPniQrWbhwocyPW7lyJS8OHz6cLGxjY6P6a0ZGxrRp04QGq4llwIABqutWhS/B5GTBi+SSGilHlUePHvEiaXVpYWGh+mtJScmZM2fUBKKMZfLkyaIFKKTvRn5+Pi+Sa+Rzc3Plx0Ku6HV0dFT9taioaP/+/XJiWbBgQePGjVX1evXqyTyAqk8DlGzfvp38uHPnzvEiuURStNDi8ePH27dv1zjAJkyYIOoU8tV8vXr1IisRDQaBmJgYXmzRogV5R4esNjU1VdQvoaGhGmMJDAxs1KiRqk6+LkHU6ar5AS/a29vzImmaSI5bxtjDhw9F/fLzzz9rjGXw4MGiE9/cnLjuSNmdJiYm8iLZs0ofN1Wk1i81bNhQq3Nf+NMrr7wiikXODCmgXKOvitRqVFNTYl1ZQkICLwrOL6qBREdHyznxp0+fLjoIZGg9e/bkxbS0NF48dOgQ+XGi/EmAvH/WunVr1V8LCwuDg4PVxwKQPgIxwmMFJycn0awqmizUZCRSOQ2J6Fql5jLJGCOX7qp5DiL8ydraWjSrklMkGSBZudR7e8kLLTkzkiXVP9AR/mprays6YuS1gRRFU7b6ixz51Ia8OJHVyomlcePGosSCPLCirzFqelDq+krmmmQs5HjWOMD4TiFbQgYilT6SLWnatCkvlpaWktUWFRXpEEujRo1EH03m9FJXVvIMFX1JUHM0yPRO51isra2FJV/q6yebJzVCSLG8vFzmEJWqQYdJjBzVonjVNE9q6iabTbZZ/oQp+mvDhg1Fx5ysn/TMJ2OROvHJJ5zkZ5FjwPBXPkjx0ksv8anzqlWr6k76aETLU+oSRrSqAwAAADBAkMDUYvZiHhcXZ1zLU+oSxruqAwAAAKhdkMDUYvZiLrXGTl9oa52j7bvGtX0B15AhQ+QXVm6V0jsZGRlTp07Fqg4AAABAN4Rr6M6dO+V7PJPrDaTQ9n0BW7dulV/41q1bWlU+YsQIrcoPHDhQfmFyxQVJZmZmYGCgg4ODudQaO32h7dGXH4MAuZVEDVo9LK6hY6JEh/vtOTk5vBgUFEQWJpfFuLu78yK5RWDUqFFktbrZllZVVYk6i3wpObmGjMzj9+zZQ34Q+TrvDh061GhXkouByF0aSUlJvJiXl0dW26pVK5ndKlospdUUKYL8JkmueyPPbinD8MOHD8scz6LG6xwLef6KlucrCQsL48X9+/fzouC1IXNiEXUW2XdyIFcnFxcXk4XJVcvkfpojR47w4ptvvklWKxpjUssKNUL+o/BCDp6uXbvyop+fHy8KbnlyAtRjLPIDJHMFqesjadNLTjJ6fGhLTuyhoaG8SG6umjBhAlnt+vXrZZ74oi2SUmtw+fCbNWvm7OxcE+mj1JpmKcg1nVKQ6621nQTUQK5b1ar31R95I7YNBwAAAAAAzx+kjwAAAAAAAOkjAAAAAABA+ggAAAAAAJA+AgAAAAAApI8AAAAAAADpIwAAAAAAAHjntZFB2lCR7ztmjH3xxRe8SDplFhQUEN8t9O2FJqqQdE3/9ddfeZF0IiT9HZmE6R35WVK2eTpAGjR2796dF0mLzRMnTpDVfvDBB7xIvie3sLBQ9dfquMGJqhIgnd5IY0VXV1eyWtI5knztuMjvTedYRO/yFpCy2Jw7d67Mk6VLly682LFjR7JaUQ3k+3/lQFq+tW3bliycm5vLi6TvY0REBC+OHDmSrFbkaadzLOT7u0Wef0q8vb1ljqWSkhKZY4BVw4BTTkvIWZocNlLNS0hIkFmtaGBU58S/ePEiLx46dIgXP/30U16U8tYdPHiwzLlRX4av4PmAu48AAAAAAADpIwAAAAAAQPoIAAAAAACQPgIAAAAAAGOixrfOvPPOO1qV37Vrl1blN2/erFX5H3/8UX7hyMhIrSofNGgQhhQAAABgmGi1I2fp0qVaVX7jxg35hQMCArSq/PDhw1qV79+/v/zCtra22h5J3H0EAAAAAABaAOMeI4M00yF9bRhjd+7c4cXk5GRe7NevHy+SPhFMf/YKfn5+vOjv78+LX3/9NS9KObC89tprvGhtbc2LItMNnY1IGGNNmzblRdIc5Pbt27xIOq0wCT8g0veEdNvRjZs3b8qs39nZmRdv3bpFVpuVlSV3Vvpf4x7Rr/IhnVzu3btHFvbw8JAZNen3RLopMf2ZQ5EHQepDGzduzIvksxEyQHKKYJy3js7eN+QkJjWAyfPl4MGDvFhWVsaLpI0Xf7Lr16Rs+PDhMqc70saLSdjlkN1anVlLzrEibXfIuZecNxhja9as4UXSsevKlSt6GWDgOWUjOAQAAAAAAADpIwAAAAAAQPoIAAAAAACQPgIAAAAAAKSPAAAAAAAA6SMAAAAAAHjhgXGPseX7lMFEmzZtyMKkAceTJ094MS0tjReljHv0hZeXFy+S5hHTpk3jRV9fX7LazMxMXtSjrw19IlG+KklJSbx47NgxXnz69ClZLekmQ3rBSPnR6EB5eTkvOjg48CLpGJWdnU1W2717d5k16IucnBxeJC2WGGMzZ87kxTNnzsg81O7u7mS1Ottaibhw4QIvnjhxgizctm1bXiRdlkaMGMGLV69erdGThfSaOXv2LFmYPF9IS7IGDRrw4owZM8hqyfGsA6SLFjkhi2zCBP744w/5zSMnf305QzEJ+7YuXbrw4v79++Wf+MuWLePFhw8f4mpu9NkIDgEAAAAAAED6CAAAAAAAkD4CAAAAAACkjwAAAAAAAOkjAAAAAABA+ggAAAAAAF54dDHuKSgokF94165dWlX+4YcfalX+nXfe0ap8Xl6e/MIXL17UqvJBgwbVdIeR5hT29vZkYWtra16UbypRUlJSo7FkZGTIbEnLli150c7Ojqw2NDSUF7OysuTXoK+TgvRPIZ1cysrKyGpJ/47GjRtrrLY6fjHkYenbty8vWllZ8WJsbCxZLWkLMmrUKF5MTU3VreUiRxjSwkkK0kKoa9euvPjnn3/yYlxcHFmtq6urboGIYiFPluXLl5P/TtrxfPXVV7zYoUMHXnz06JGcw0v678jB1taWF4cPH04WJj2tVq5cqbF5aqJmjI0dO1b116qqKj3ObOTA27x5My/++OOPZA1r1qyReeKTRmxyMDU1FTkBke5m58+flzmzSSUGUVFR8me854BWnkFbtmzRqvIDBw7ILzxy5EitKr98+bJW5cmLqRQ6XDJw9xEAAAAAACB9BAAAAAAASB8BAAAAAADSRwAAAAAAgPQRAAAAAAAgfQQAAAAAAADpIwAAAAAA0AJzHALjon79+rxI+nUxxv79919etLCw4EXS6u/Zs2dktSKPNJ3t3y5dusSLf//9Ny/6+/vzYr169chqjx8/zoudOnWSGbVuZGdn86KbmxsvigznBGxsbOR3N2nLpy9PPsaYo6MjL5LWm+TAI93pmITFY8eOHXlRZM6qs/+oyNZOgPR3ZBIelqSbI+kdS/oyMsZat26t+ivpYigHFxcXXnz55ZfJwg0bNpR5vqSnp/Mi6RfLGDM3N1fzq3xIk0XSSpA/gALjx4/nxStXrvBiQkICWW15ebnqrxUVFbrFQhrxkufFzZs3edHMzIyslqyBHI2NGjXS7WSpqqoSdQQ5seTm5vIiaTY5btw48oPOnDnDi2Q7O3fuLOcCBAwE3H0EAAAAAABIHwEAAAAAANJHAAAAAACA9BEAAAAAACB9BAAAAAAASB8BAAAAAADQxXmBdLjQF3PmzKnRgO3t7Y27wyizDC8vL7LwDz/8wIukcU9QUBAvOjs7k9WK7B5IGw45kP+4Z88eXiQdKwYOHEhWS1rMkP4a+jLvYBJOFtu3b5fZ7GHDhpHVklYaMTExvCgyudC750VaWhovlpWV8eLo0aPJGkg/oKKiIo2DXL5BjImJicahThrBMAmXn99++40X79y5I380ijqisrJSZiCiWEiTqVWrVpH/TloLZWZm8mJSUpL8GV5k/WNpaanbWCJdrv755x+yMGlBdfjwYZlHo0+fPnK6m+x9OZBOTImJibxI2ni5urrKP91IpzPR5K+zM5TU4Fy0aBEvXrhwQWanMMbCwsJ4ceTIkTV3ZVEPOalKoe0gl7poyr9qqEFqtEhRUFBQo9kI7j4CAAAAAACkjwAAAAAAAOkjAAAAAABA+ggAAAAAAJA+AgAAAACAOoo5DoFhImyCy8jIEOnkZjSpDVbkllVSLC4u5sX09HSyWtHmPqGRaraUCn8qKSkR6aKNz2qaR0bNV6im2tLSUl4UbW4VyqjfGyv8ld8vTO50JjfWkZ0ldajJaskaRE0SfpUTC1+baIOtQMOGDTWOBLIl6vuFPEQ5OTmqvwr7iDUOMP5zyU+UOtTk3lty2y85RMkBxrdKiFdjLI8fPxbp5C5yMzMzshJyhJDb5PkPkirJHw3hU3Q48ckDKDWJkcOJ3FpOtkTKgkA08IRG6hALCXlUyf+V2lxMdgE5GkXnoDDm5Zz4hYWFck5ncp8yefylDo5ompU/GuVMYgDpIxAjeFtMnTq1OpW0a9dOZsmDBw/KFNU0WMoSRYglOjpaTj1kJWR2Gx4eLr958fHx1Q9EGUtoaKjOnXLq1Cle3LRpUw2NIo2xbNy40VjOCPUDbP/+/XLq2bJlSzVbQp5WpJWJzrHs3LnTiGYq9bFERETUYvP4JEng+PHjOsRy+vRpQzv+N2/e1O3E37179/Npoa2tLS+SJkSkqD4WgPQRiPHw8AgJCXFwcJC6wWA4VFZWZmdne3h4GHssGgNBLBhgiAWx4MQ3nFgA0kcgxsrKysfHx1haq/7boRHFovFrLmLBAEMsiAUnvoHEAmoRbJ0BAAAAAABIHwEAQE9cuXLl6dOn5eXlsbGxaAwAADA8vAaGw507dw4fPvzZZ5+pUVT566+/srOzr1+/7u/vHxAQQCqGEEhJScmOHTtatGhRVFQUGBjIlz9w4EBmZqaFhUXbtm0HDx5c64GI2h8REZGYmGhqauro6DhmzBi+vKiAQqHo2bOnqampsFG0b9++69atM5xhxh9tVcjGf/bZZ48ePWrSpMny5ctrvf0aGyMabwbeHUpOnz6dkZFRXl7eokWLIUOG6GVk1lYgxcXFDx8+tLOzI/c+ShW4devWuXPn5syZU4uNlzlaRB1hIBMvQPoIaiQVe56cOHFi2LBhWv3Lrl27rl69qrpHj1dUefjwYUFBQWBgYF5e3tChQ7t27VpVVSVSWrZs+fxj4Zu9cuXKd999t0WLFiNGjBg6dKizs7Nq+fj4+IMHD4aEhDDGZs2a1b9//4yMjJoIRGYsovYXFxdv2bJF2IM/adKkAQMG2NnZqZbnCxQXFwcFBXXv3t3U1DQiIqJv374GMsbIo21paalaIDU1lW/822+//fLLL9fcVgOtYtHYGNF4q6qqeg7dUZ1OYYxlZGTcv39/9uzZjLHPP/+8X79+1tbW1RyZtRJLUVHR+++/Hxsba2Fh0atXLz8/P9H5LlVAoVCsX7/e09OzdvuFHP/qO4KfivU1XwEDBw+v6yC7du3atGmTlI/acyAxMVHbf5k1a9agQYPUK6okJCT88ssvjDF7e/tWrVrFxcXxSq3EImp2SkpKVlZWixYtGGO//PKL6FrCGDt79qxytrW3t7969WoNBSIzFlH7r1y50qFDB+HnTp068c4afIF69er5+/u3bNmyQYMG5ubm7du3N5AxRh5tUQGy8ebm5k5OTjW3TVWrWNQ3hh9vz6c7qtMpjLG8vLwLFy5UVFQwxqytrUmTS21HZq3EYmtre+DAAUtLSxMTk8rKSt6pUarAyZMnfX19a71f5IwWUUfU3HwFDBzcfayDzJo1q3HjxpcvX67DMQ4YMMDFxUX4WTAG69ixo0gxhHbGxMTY2toePny4sLDQxsZm3LhxogLW1tZKZ+Py8vJ79+5NmjTJcALJyMhQGoY3bNjwwYMHGgso73OEhobOnDnToIYNf7R79+6tWsDR0ZFv/PXr1xUKRUFBQevWrdV8pXk+qG8MP97IiAyNLl26KBSK8ePHT5o0qV+/fmT6qO3IrC06duzIGIuNjfXx8SHvw/EF8vPzTU1N7ezsZNqS1xw6jBZ+KsYlGOmjJIZzooIXd+Cam7u6ujLGoqKiPDw8OnfuzBjjFWWiIHpGmZmZ+ccff3h6em7YsGHfvn2iv+qR3NzcxMTEtWvXMsamTJnSo0cP0fQ6ZMgQ4QHckydP7t+/7+npSYZWW4EUFhYq67SwsOAvb1IFHj9+nJ+fL2qPqP3PrRfUHG2ymKjxEydO7NKlC2Ns9OjRL730kvKxXa2MK6nGqB9vOnSHmZlZZWWlpaVlaWlp/fr1a7pr3nrrrR07dqxevXrJkiXVHJm1eL4L/P333+Hh4YsXL5ZZIDw8/LXXXjt06JCoWG0FQo4WraZiAznfSaReEEUihCYfqTc2kWj7hLBZs2Zala9pX0/cfXyBEJ3MVVVVb7zxhrAOLCgoaMaMGfLfUsOTlJT0559/mpiYKBSKf//9t7y8XKFQmJiY9OjRY8CAAUKZHTt28O/FGjt2LP9IVybFxcWHDh36/vvv1ShC0rB+/fquXbsqp4/58+fv2LHDzs6uZ8+eT58+PXPmzP379+fOnSszFvmB2NjYKCeg5s2bnzt3TpQ+NmnSZNWqVb///nvTpk1dXV3t7e11DiQ9PT0+Pv7u3bsDBw50d3eX3y9qsLGxUc5xZWVlDg4OMgscO3aMf/Kl2n5R4y0tLdPS0m7cuJGUlOTn56dsv7ZjTONII4+2CFHj3dzclHe5Ll26pNxwo747hHPt22+/XbRoUXViESHVGPXjTbfumD9/vpeX18KFC9Wkj1oFItUvFRUVV65c2blz54ULF5YsWeLq6tq9e3edR6bGfjlx4oS1tXVycvL06dP1PsAYYyNGjPDz8xs7duyvv/5KTm6qBXJzc728vKS+7agJ5OLFi9u2bbO2tjY3N9+6dau+BpjUaNF2KtY4wB4/fnzt2jXGmL29fQ0t+gRIH4Hebr2oTkbXrl1r1aqV8HNsbGw1d5W2b9/+o48+En7etGnTu+++S95g0G9EO3bsWLlypY2NTVpamjBN8wpjbM+ePapPkY4dO+bh4SEstBdW6Lu7u6u+1VBjLPID6dChg3KBnampKfmK2w4dOgiruDZv3rxw4UKp0DQGEhUV5e3t3bt376CgoDVr1sjvFzW4uLgoFzMVFBTwWZ1UgZiYGH4zrGr7+V74999/HRwc2rVrl5ycTKaP8mNR00Hk0Rah2vgjR45ER0cLx7OkpET1C73GcZWSknLnzh2dzxceNY1RP9506A7G2MaNG5XPJatz4mvsl507dwprHvr06fPtt99evXpVY/qoZmSq75eYmJhmzZp17969f//+eh9g0dHRP/30U2hoaIMGDZo0aXLy5EmR2QJfQLi/+88//whJXmRkpHJNgvpAnJ2d9+7dm52dTb7GvTonPjlatJ2KNQ6w2NhYJycnd3f3AwcOIH00RrB15gViz549ql9zz54926dPH8bY3bt3a3RBvb5ISUlRXYoeHBzs7+9fXl5+/fr1R48ekYrySmNiYqL89dmzZ23atBF+jo+P1+pxg7b06NFDObmnpKQI3/tVA0lLSxs1apRwt8DZ2Vm4V6RbIJMnT/by8srMzNTjzseXXnpJ+VLdW7duCSsFVdtPFmCMPXjwwMrKir/kK9vP98LIkSMdHR2vX78u5dtSfcijLRpXosY7OztPnjxZSNfy8/N79eolsztKS0tTU1ObN2+ux/aTjVFtPznedOsOExOTmJiYQ4cO6bwhRqtvKcrvb0+fPhW+4vL9onFkyumXo0ePpqenR0dH6323DWPMxMRE2AGjUCgyMjI6deokCkRUwNXVddq0aW+99dacOXM8PDy8vb1V17OqD0Q4zWNjY6VuXuqMaLSo7wiZ8xU/wHx9fZctW7ZkyRI5j0EA0kdQm4gmo3PnzgnrVKKjo319fU+fPl2LbQsODt6/f/+lS5c2bdpUXFxMKu+9997t27eF8levXv36668nTpzYr1+/SZMmtWrVilekPisgICA3NzcqKurUqVMZGRn8ZVWPgVhaWr777s44my4AABA4SURBVLvr169ft27dlClThFapBuLo6Ojv7x8SEvL777+vWLGCDE2rQCIiIubNm6ev9tevX//NN9/csmXL5s2bZ8+e3aRJE1H7yQKMscaNGyuX4ctvfLt27YYMGbJx48YaGmb80RaFwzdeSMh+/fXXtWvXrl27Vs1jXFFE8fHxer+uk41RbT853nTrjhYtWkycOHHMmDE///xzTZ/+/v7+ubm527Zt2717d35+fs+ePfl+kTMy1YcWHh6enp5uYWHh5uY2YMCAvXv36j2Q/v37N2vWbM+ePatXr543b55gfKMaiKhAv379BP348eORkZGnT58+ceKE/D7KzMxUbgXTI6LRor4jZM5XfOPv3bv38ccfOzk57dq1C1dnY8QkJSVl0KBBkZGR8u9Y3L17V/4HKFfqyETqWY8Uwtc7+WzZsqXmjuY777wjs2Rqaqq2h12rDOD48ePp6enjxo2bOXNmgwYN+DL5+fkBAQEzZszo1KlTcnJycXFxt27dlHNZNTl27Njw4cON6DRIS0s7dOgQ+XzHuGI5ffr0Sy+9lJOTo/yib0SxfP/99+PGjSsvL//pp582bNhg7GPs2LFjlpaWe/bseeedd9R7shhmLHv27OnZs2eTJk1++OGH7777rs6c+GFhYT4+PsKzXeUaDyON5eDBg82bNxe5BxhLLOvWrXv//fcZYxs2bHjvvfd0q0S4kh49elT+Avro6Gj59X/yySdatWfTpk3yC2u7S/3PP//Uqry3t7f8wsJ3NpmXy4CAgMjISKx9rINMmzZt2rRp6sucP39+4sSJys0i+sW4cscnT56cPHlScI4UPDWMNJbw8PBt27YFBwf37Nnz7bffNrp+8ff3T05OTkpKWrBgQR0YY8OHD09JSamoqNC4OsIwY/Hz80tMTLxy5Yr8S7tRnCwjR44MCwu7du2aaN+MMcby5MkTOWbphhnLsGHDwsLCnJychDVUwOhA+viCcu3aNcN5zVftYmNjExgYSL5O0Ljw9/f39/c33vZ369aNMVbrxop6xMXFpSaekD63xmvcN2OMWFtbz5o1q27E8sYbbxhv493c3LR9OAmQPoLaZ+nSpTgIAAAAAED6WHcoKyuLi4urudfs6pHKysrs7GwPDw+pPSjGEovGQBALBhhiQSw48Q0nFoD0EYiJi4ubOnWqETU4JCTEx8enDsSiJhDEggGGWBALTnzDiQUgfQRihJcohISEODk5qeqk7/TJkyfJSqRckXlWrlzJi1ILrm1sbFR/zcjImDZtGv8+Eo2xkF5iGRkZvBgUFMSLaWlp5MeR/uekDbXoS63GQJSxBAcHi2JRdURSUlhYyIvffvstL/JvsBD47LPPpNqgHvmx7N69WxSLqSnh51VeXs6L27dv58Xr16+TH0c6LQvv39PYLzNmzNA4wLZt2ybnpV65ubmkvm3bNl7My8vjxYCAAF6UsrlWvpdZICsra968eRpj+fnnn0WxkAMsPz+frGTZsmW8mJWVxYuk24CU35DSTlwgMzNzzpw5OsRCkp2dTerr1q2TGQtpfKH6Dj1V6tWrJ4rlrbfekjPG1PsfqWkeafpBOn4zxsil2GS/iE4WjQNMGcvOnTvljDFy4xf/khvG2K1bt8iPGzFiBC8OHDiQF0UmIZmZmYGBgXKmO4D0Efx/hMcKTk5OImefyspKvrDU5jv5lmCNGjXixRYtWpCFRa/ZVW2wVrGot6JVhXw7KpnlMMaaNm0qMxbR5VBjIGpiIWde8pWm5IMY8t+FD5IpkjXIjEXkeSE/fSQ9oaReZSu/X0hvRY0DrFmzZlIj9n+mPHN60iP7hYyFPN2kMiSysA6xkP0ryoHUN5v8UPK1jeQAk+puffWL1OlMxkh2Imn9KGXbTna3xlgcHR3lxFLNQLTqF50nsWbNmolOfHKMkW+IFt1BELCwsJB/cSH7RdsrC6hdYBsOAAAAAACQPgIAAAAAAKSPAAAAAACg1tFl7aO2b9rRCnK3gRo0vtFBRGxsrPzCyrfiGg45OTm8SO4sYRILJclNJDExMbw4atQoslpyhYoOVFRU8OLu3bt58fDhw7w4ZcoU+YeoJt4Mq/FQr1+/nhfJfU7z588nq01JSeFFchGh1MIjXb5TUqvQfvnlF148duwYL06YMIGs9ubNm7zo6urKi6J1afLXyIog/3H16tVkYfJdrLNnz+bFR48e8aLU5qcaZdGiRaRO7l4iX0REvulearuJ6MSXWrCrG+SON8bYxYsXeZHcjfHHH3/wYvv27clqa9QLZtWqVbx448YNXvTz8yNrOHLkCC+S+8zItY+6QXYo6RBMxkJuKZOavfv3719zVxb1SC3tJZHaaafV5CmF1O5PKcjDrobBgwfX6JHE3UcAAAAAAID0EQAAAAAAIH0EAAAAAABIHwEAAAAAANJHAAAAAACA9BEAAAAAAACkjwAAAAAAQAvwzmsjg3wDacOGDcnCX3zxBS9WVVXxIvl2Zq0srHSgpKSEFyMjI3lx1qxZvDh16lSyWtL3kXxlsx7Jy8vjRdL38YMPPuBFKd/Hs2fPyvwsqXcu6wBpdUbaJX7yySe8+NZbb5HVkmZ42dnZvCh6ebHO/oIPHz7kxb1795KFDx48yItDhw6VGQjZKYx7t6/OsZBmk2FhYWThqKgoXiSd9kjz1PT09BqNJTU1lReDg4PJwqTh6Pjx42UOPPK4MQnzVB0grfv279/Pixs2bOBFKW/dhQsX8mJGRkbNBSJ1vmzZsoUXDxw4wIsjR44kq718+TIvFhUV8aJ+nURBTYO7jwAAAAAAAOkjAAAAAABA+ggAAAAAAJA+AgAAAAAApI8AAAAAAADpIwAAAAAAADDuMbZ8nzLTcXJyIgvfuXOHF5OTk3mxX79+vEiaBDHOXkFntwXSIIY0j/Dx8eFF0g+CMWZjY8OLpEmQvgJhEl4k+fn5vNi9e3deJM2GGGO3b9/mxVatWvGiHo174uPjZfaLl5cXL5JWJlKxkGNAX/2SkJAgcyQwxhwcHHiRtLC5f/8+L3bs2JGs1sPDQy+xkOcyacLFGGvbti0vkh5JpHj37l2yWm9vb70MMLJfpI5Mr169eLGyspIXzczMZH4WY6xr16566ZfExEReLCsr48X27dvz4tOnT+kLs7m5zBPT09NTX5MYaXJkaWnJi87Ozrz45MkTslpXV1deJK3iQN1PH62srOQXHjNmjFaVf/PNN1qVb9eunVbl7ezs5BcmTxIAAAAAGCOtW7eWX5i8saKGiIgI+YXJb0FqaNSokVblya+RUsi3eVaWxMNrAAAAAACgBUgfAQAAAAAA0kcAAAAAAID0EQAAAAAAIH0EAAAAAADGBIx7jC3fp7ZHtWnThixMevSQ9gqk2YqUcY++ePz4MS9aWFjw4pUrV3hRyoGlW7duvBgQEFCjsZAePaRVx7Nnz3gxNjaWrPbvv//mxR49etRoLKSLUOfOnXmRNPW4ceMGWW10dDQvTp06teYCKS4u5kUph528vDyZIunV4ufnV6OdQhonkVY1jLGbN2/yIumYQRpOSbnJ6AvyQx0dHcnCFRUVvEjavpCjUf5+Ut0gZ05bW1teVCgU8g+1VM/WKOSET9rukM5EUl48pKdYrQQI9JyN4BAAAAAAAACkjwAAAAAAAOkjAAAAAABA+ggAAAAAAJA+AgAAAAAApI8AAAAAAOCFB8Y9RkaDBg140d7enixsbW3Ni6TLT1FRES9KOePoi6ZNm/Kim5sbL5KWHKQRBpN4r7yJiUmNxuLi4sKLTZo04cXGjRvzYv369clq5bss6RHSb4i0QyJjIccSY8zGxoYXSZ8mfUGOJW9vb7Iw2QWkcQ/ZraSoRzw8PGQef6lhQ54CZF+Tg1mPkF4wUhY2GRkZvHj//n1ezMrKkjkGavpkqaqq4sXy8nJeTElJIavNzs6Wedz0iJOTEy/m5ubyImmHJDUvkU5egwcPxtXc2MHdRwAAAAAAoAU1fvdx3759WpX//PPPtSofExOjVfnff/8dvQ4AAAC8gEg96iHZunVrzSUw5Msm1LBo0SKtypMPfKSorKyUmzWa/7+8EXcfAQAAAACAFiB9BAAAAAAASB8BAAAAAADSRwAAAAAAgPQRAAAAAAAYE/B9NDLIXWOFhYVk4X///ZcXSac9Ozs7Xnz27BlZrcjVTMp/USOkzRjpB/b333/zotTnktWSpnf6CoQx1qpVK17s168fL549e5YXpUzvyO4mA6xO40W0a9eOF4cOHSpzgP33339ktb169eLFGrWyIwPp0aMHWfjChQu8+PjxY17s3Lmz/EBEA09n/1HS93H8+PFk4ejoaF60tbXlxS5duvBi165d6ZsN/+v2R5r/ycHd3Z0XBw0aRBYODw/nRdL3kXS0JT+L7widYyEP4MCBA3nx/PnzvEi6WkrFQnqvijAzM9P5fGndurXMSSwiIoIXpbbukka8bdu2rbkBBp4P6B4AAAAAAID0EQAAAAAAIH0EAAAAAABIHwEAAAAAANJHAAAAAACA9BEAAAAAAAAY9xhbh5kTXebl5UUW/uGHH3iRNO4JCgriRWdnZ7Jakd+N6Ff5kC0ZMWIELx49epQXHz58SFZLel7Y29vXaL/Uq1ePF7/88ktenD9/Pi/u27ePrPaLL77gRTn+HdXB0tKSFydNmsSLS5Ys4cUTJ06Q1a5cuZIXXVxciC+1evLvIAMZPXq0/ENNDtGPPvqIFx0dHclqRYZKOsdiZWXFi4sXLyYLz549mxdJ456lS5fyIukMxaOzCVGDBg14cdWqVWThjz/+mBdJj5hPPvmEF21sbMhqRVZZOvcLeVS/+eYbXly+fLnG4aHku+++k/lZpaWlqr9Wx7iHtAnbunUrL37++ee8mJ+fT1a7aNEimf0i6lbyYgcMB9x9BAAAAAAAWmAu5PtS5qXVR8oPWYqSkhKtyktZlUqhVaR6tGImm6Ft4wEAAACgmgBkZmbK/xet7pqXlZVp1R7RzWD9ZkdZWVlalU9NTZVfWP5TRGX2Yp6dnc0Ymzp16gsy2qZMmWI4jcnOziaN/gEAAACg8RrKGAsMDHwRgl2wYIFBHXlzDw+PkJAQBweH6qyZADp8Z8rOziZfRAYAAAAAjSCBqcXsxdzKysrHxwdH5PmD+44AAACAziCBqcXsBTubDDfBZ9RKTXKBQkFBAVkJuXONFIuLi3kxPT2drFa0YkPjOk6pWMilpeTi1IqKCl6UWplKbgB89OgRL4p2gMpZkCoVC7mepqioSOPRU9MpUv1CxiIqWZ1YyF2o5eXlvEiuVNaqX9LS0jQeDZkDTOb6p5ycHDmjWs3pRq5AIncT80dDaKQOsZADLDc3l6zk2bNnMntQePYnp1N4dI6FRGpdF9lssl/IDyJ33/PVyoxF5uIzshgZiNTJQsZC7r4XLc7TGIi2Y4xc/Eeu8JNayUceDXJZnqhbsUPAwDGpud0hoDrExsYa14LUkJAQqW+BxhWLmkAQCwYYYkEsOPENJxaA9BEQ3/ni4uKMYkmHciUE+eXYiGLRGAhiwQBDLIgFJ77hxAJqkf8DDjzsQyteM3UAAAAASUVORK5CYII=" />

Unlike the pixel basis, the PCA basis allows us to recover the salient features of the input image with just a mean plus eight components! The amount of each pixel in each component is the corollary of the orientation of the vector in our two-dimensional example. This is the sense in which PCA provides a low-dimensional representation of the data: it discovers a set of basis functions that are more efficient than the native pixel-basis of the input data.

# Choosing the number of components

A vital part of using PCA in practice is the ability to estimate how many components are needed to describe the data. We can determine this by looking at the cumulative explained variance ratio as a function of the number of components (Figure 5-87):

In [ ]:
%matplotlib notebook
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

> **Figure 5-87: _The cumulative explained variance, which measures how well PCA preserves the content of the data_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfYAAAFgCAIAAADGtsy1AAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3deUBU5f4/8OfMPsMMDPsuErsbJi6UkqKimEuRG5not6x79ap5/VqpmaKlgXXr+lPzfrO8lVtaKZpmi4hW7oiBIouIKJvswwyzMOvz++N4yatCmhZzhvfreu89HGeGz3nO8c3DM888h6GUEgAAcEQ8NAEAACIeAAAQ8QAAgIgHAABEPAAAIOIBAOAWgk6vwGKxqlR67ragq6sM9aN+ND7q//N5eio40IsXCPjc/iGJ+lE/Gh/12ysM1AAAIOIBAAARDwAAiHgAAEDEAwAAIh4AABDxAACIeAAAQMQDAAAiHgAAEPEAAICIBwAARDwAACIeAAAQ8QAAgIgHAABEPAAAIOIBAAARDwCAiAcAAEQ8AAAg4gEAABEPAACIeAAA6LSIz8vLS0lJuXVPVlbWpEmTkpOTv/zySzQ0AMCfT/BQXuXjjz/ev3+/k5NT2x6LxZKenr53716xWPzss8+OGDHCzc0NzQ0AwL1efFBQ0AcffHDrntLS0qCgILlcLhQKY2JisrOz0dYAAJzsxSckJFRVVd26R6vVKhQKdtvJyamlpQVtDWDnKKVWGzVbbBarzWqjNhu12KjVarPaqNVKbZTaKKWU2GyUUmqjpFrVqlLrCW37D6G07ZUIIYTe/C/7l23/9+vDbtm+o5g//nidqzUaTevv6dT6KLyU0i4U8XeSy+VarZbd1ul0zs7O+PcD8MexWG2tJmur0dJqsraarAYTu2ExmqxGs7XVZDWarK1mq9FkNZmtJovNZLGazTajxWa2WE1mm8XK/qFoyXsR4ue8bEb/Lhfx9JYfxyEhIdevX9doNBKJJDs7e9asWR080dNTwenzjfpR/0MvnlKqa7Vo9SaNzqTVm7UGU4verDWYtHoz+6W+1aI3WvQGs95o0bdaTGbrfX1fHo8RC/liEV8k5MskQqGAJxTwhAK+kM8TCHhCAU/A5/F5DJ/PsBsCPo/HY3gMw+MxDEN4PIbPMIRhGIYwhBCGMIRhGELIzf9jCGG/JL/u/+8vb35Fbj771wf8irHXsxYV7M6Vq+5hRjx76g4ePGgwGCZPnrx06dIXXniBUjp58mQvL68Onlhf38Lpf6KoH/XfF6PJqtYZNTozEfCrazVaA5vav/7RtZp1BouN/kafWiTkSUUCiVjgKhdJRAKJiC8RCaRi/n+2+WIRXyoSiEV8sfDml2IhXyzkswmOi+dB6reHA7+XHzMMpZ3/qxkiBvU7Uv2UUoPRotKamrXG5hZjs9bYrDU1a41qrUmjM6l1JmP7PW4ew8ilAiep0EkqlEuETlKBk4TdFsj+86VMInCSCCUi/kOJaVw8jv37q4AAwP2zUdqiNzdpWlUtxlv+tKq0JlVLq8lsu2t8K2RCb1eps5PIxUnk7CTy9VIwNptcKpTLhAqpUC4VSsUChmHQvPCwIOIBOhxUMVubNK0N6tZGdWuDurVBbWgLdKvtLr8BK2RCH1eZUiF2VYiVcrFSLlLKxUq5WKkQK6RCHo9xpF4kIOIBOEPXaq5TGWpV+jqV4T9/9Bq9+baHMQxRysXdfRSuCrGrQuKqELf9UcrFQgEWBQFEPECnMpmtdSpDTZO+VqWvadLXNOlrmwxaw3+lOY9hPFwkAV5yd2eJh4vEw0Xq7iJxd5YoFSI+DzkOiHgA+6DRm2oa9dWNuppG/Y1G/Y1GXaO6lf53mnsqJY/4OXu5Sr1dZV6uUm9XqZuzpNPfzwRAxAP8ilKqajFWN+qqG/Q3GnXVDbobjfrbuucuTqKIbkofN5m3m8zbTebjJvNwQZoDIh7A/gK9SdN6qaypqkFXVa+tbtBVNehaTb9OSWQY4qWUhvq7+HrIfN2cfN1lvu4ymUSIpgNEPIDdMZqt1Q26ijptZZ22ok5bWa/VtVra/pbPY3zcZL4eTn7uMj8PJz93J283qVDAR7sBIh7AHrWaLNdrWsputFyr0Vyv1dY16dvG0BlCPF2lfcI83RXiAE8nfw8nbzcZxlsAEPFgv2yUVtfrSiqbr1ZrrtW0VDfq2j5/LRMLwgKVgZ7yQG95gKfc38NJLOJjajkAIh7smtliLbvRUlLZXFKpLqlUG4w3x17EQn5YgDLYVxHs69zdR+GplOKTnwCIeOAAo9laWqUuLm8uLlddvaFpW8bWy1XaL9wjLEAZ4u/i6ya77eOgAICIB7vtrduuVDYXlquKypvLqjXsAgAMIYHe8vAAZXigMjTARSkXo6EAEPHADZTSWpUh/2pjfllTUbmKXZ+LYUiQtyKimzKim2t4gAtmMQIg4oFLWk2Wwmuqi1cb88uaGtQ3753m6y7rGezWs7tbWIBSJsG1B4CIB0512Ksb9RdLGy9ebbxc0cyOw8jEgv4Rnj2D3XoFu7u7SNBKAIh44BIbpaVV6uzCul9KGhr/c7PjIG9F7xD3Po+4B/spsGgXACIeuJfsV6s02UV154rrVC1GtsM+INKrT4h7r2A3F7xrCoCIB86hlF6raTlTUJtd9GuyD+ntOyDKKyrIFR8uBUDEAydVNejOFNSeLaytUxkIIVKxYHBvnwGRXj26uyHZARDxwElNmtbTBbVnCmor6rSEEJGQNzDKa1AP717B7rjDEQAiHjjJaLLmXK47cbGm6LqKEsLnMX1DPQb28Ho01FMswtqNAIh44CAbpcXlzScv3jhXXG80WwkhoQEuj/fy6R/hJZfiA0oAiHjgpga14cTFmuMXbrATHz1cJKN7BT7Wy8fbVYbGAUDEAyeZzNYzBbU/X6guvKaihIiF/CG9fQf39gkLVPKwrCMAIh446kajLiun6nRhrc5gJoSEBbgM6eM7INJLIsJVAYCIB26yUXqxtDEzp/JSWRMhxFUhfiK225Devr7uTmgcAEQ8cJW+1XL84o2snMq6ZgMhJDxQOTImYNTjwU1NOjQOACIeuKqmSZ95ruLExRqj2SoU8OL6+I6ICejmrSCE8PGpJQBEPHARpfTStabMc5UXShsJIW7O4vGDuz8R7YfpjwCIeOAwo9l6Kr8mM6eyukFHCAkNcEnoH9gv3AOLPgIg4oHDDEZL1vnK789WaA1mPo95rKf3yP6Bwb7OaBkARDxwmL7VnJlTeTi7QtdqkYoFYx8LGt4vwFWBRX0BEPHAZVqD+XB2RWZOpcFocZIIkuKCR8QE4IaoAICI5zaL1XYkp3L/8bJWk1UhE04aFhL/qL9UjNMKAIh4jiu6rtpx+HJVg85JIpg6PHRYX3+sAQkAiHjOU7UYvzx65XRBLUPI0L5+E4eGYB4kACDiOc9itWXlVO47XtZqsnb3UUwfFfGIH2bLAAAinvvKbmg+/baook7rJBHMGB3xRLQfj4fFIAEAEc9xrSbL3p+uHsmppJQM6e07OT5EIROhWQAAEc95F0obtn1f3KgxertKZyRGRgW5ok0AABHPeWqd6fPMy2cL6/g8Ztzj3cc/HiQUYM4MACDiOY5SerawbvsPxbpWS4if88wxkQGecjQLACDiOU+jN23/vvhccb1IyHsuITy+nz/usQcAiHhHcP5y/dbvijR6c1iAy6yxUV64OzYAIOIdgK7VvPPw5VOXagV83tThoQn9AzEnEgAQ8Y7gUlnTlm8KmrWmYF/FrLE9/DxwJ1UA+CMjvrKy8sqVK3FxcdXV1YGBgWisP4jZYv3q2NXD5yr4PCYpLvjJx4Jw4w4A+GMj/tChQ//6178MBsPu3buTk5Nfe+21p556Cu310FXWaTcfuFRZr/Nxk/1lQo/uPliNAAAemnZ7ix999NHnn38ul8vd3d0zMjI2b96Mxnq4bJT+cLb8zc+yK+t1wx71T31+APIdAP6kXjyPx5PLb07E9vLy4mHo4KFStRj//U3BpWsqhUz4/JNRfUM90CYA8OdFfFhY2Pbt2y0WS2Fh4c6dOyMjI9FYD8u1Gs3/++qCWmvqE+L+/JNRLk5YbQYA/hDt9s1XrFhRW1srFouXLVsml8tTU1PRWA/F+cv16TvOa7SmKfGhCyb1Qb4DQCf04sVicd++fRctWtTU1JSVleXkhDl8D4pS+v3Zii+PXhEKefMm9n40zBNtAgCd04t/4403fvjhB3b7zJkzHffiKaWpqanJyckzZsyoqKho2//1118/88wzkydP/vzzz7t4Q1ustm3fF39x9IqLXLT0uRjkOwB0Zi8+Pz//wIEDhBA3N7d33313/PjxHbxKZmamyWTatWtXXl5eWlrapk2b2P3vvPPOt99+K5FIxo4dO27cOIVC0TVbWd9q+de+i5euqbp5yV+e1MfNWYIrDwA6M+JtNltdXZ2XlxchpLGxseMZNTk5OXFxcYSQ6Ojo/Pz8tv2RkZFqtZphGEII01UX0mpQG9Z9eaG6Qdc31OMvE3pIRPhEMQB0dsTPnj07KSkpJiaGUnrhwoVly5Z18Cparbathy4QCGw2G/sjISwsbOLEiTKZLCEhoW0KZpdyvaZl3Zd5ap1pZP+A5OFhWHMGAOwi4sePHz9w4MDc3FyBQLB8+XK2O98euVyu0+nauv9svhcXFx87diwrK0smk73yyivff//96NGj7/p0T09uD+C0V39OUe3aneeNZuuLT/V66okQztXP9fZH8ajf4ev//RGv0WgyMzObm5sppYWFhYSQefPmtffgfv36HT16NDExMTc3Nzw8nN2pUCikUqlIJGIYxs3NTaPRtPf0+voWTl8id63/p7zqrd8V8/nMnKd69Y/0sttjbK9+rrc/ikf9XaH+3x/xCxYsUCgUYWFh9zKGnpCQcOLEieTkZEJIWlrawYMHDQbD5MmTp0yZMm3aNJFI1K1bt6SkpC7ymxGldN/PZQdOXpNLhS9P7BMa4ILfFgGgUzCU0rv+xfjx49kZNX8CR+oIWKy2T78tOplf46WULpwS7e0m41b96IiheNTvSL34dufJREVFFRUV4WfgfbHZ6OavL53Mr3nEz/n1lBj7z3cAcGztDtSUlJQkJSW5u7uLxWJKKcMwR44cQXt1gFK6/Yfic8X1EYHKv0+JFgv5aBMAsNOI37hxI1rnvuz7uexYbnU3L/n8iX2Q7wBg1xHv6en5448/slMhrVZrZWXlggUL0F7tyTxXceDkNS+ldOHUvjIJPtwEAPYd8fPmzTMYDOXl5f3798/Ozu7bty8aqz2nC2p2Zpa4OIn+N7kvVo4EAPvR7tutZWVlW7duTUhIePHFF7/88su6ujo01l2dL6rbcrBQKuYvnBLtpZSiQQCAAxHv7u7OMExwcHBxcbG3t7fJZEJj3am0Wp322VmGYV6e2KebtwINAgB2paO7Pr311lvPPvvsK6+8UldXZzab0Vi3qVXp/9+XF0xm69yk3hHdXNEgAMCZXvzKlSvHjBkTGhr68ssv19XVvffee2isW2n0pn9+kac1mP82KfrRcCz+DgAcifhLly4RQs6fP08pzc7OVigUo0ePVqvVaKw2RrN1w1cX6lSGcY8HjY7tjgYBAPt0l4GaXbt2vfXWW+vXr791J8MwW7duRXsRQmw2+tGBgtJqzWM9vZPiHkGDAACXIv6tt94ihIwZM2batGlooLv8CMwqOX+5PrKb8vkno7rsfU4AgBPaHYvfuXMnWudOP2RXZJ6r9PNwmvdMbwGfhwYBAI714lk+Pj4zZsyIjo4Wi8Xsng7Wi+8izhXV7T5S4iIXLZwcLZMIcfUAAFcjHh9nvU15bctHBwtEIv7fJ0W7u+D+2gDA5Yi/tc9OKa2srOzKzWSx2j4+WGi22F6e2CfIBx9xAgCOR/z27dvff/99g8HAfhkQEHD48OEu20xfn7hWWa99Itq3b5gHLhoA4Ip23zD897//vX///ieffPLw4cNr1qzp06dPl22jshuaQ6euuzuLpw4PwxUDAI4Q8e7u7oGBgREREZcvX37mmWfKysq6ZgOZLdaPDxbYKH3+ySipGKsEA4BDRLxUKj19+nRERMTRo0fr6+s1Gk3XbKB9P5fdaNTH9/Pv0d0NlwsAOEjEL1++PCsrKy4urrm5ecyYMdOnT++CrXOlSv3d2XJPpWTysBBcKwDAOe2OPFy/fv3VV1/l8XgbNmzomk1jNFu3HCwglMwa20MiwhANADhQL/7rr78eMWLEihUrzp071zWbZs+PpbUqQ8KAwPBAJS4UAHCoiF+/fv2hQ4f69ev30UcfJSYmrlu3rku1S3G5KvNcpbeb7JknsNAYAHBVR+MPcrk8Jiampqbmxo0bubm5XadRzBbbp98VMwx5cWyUSMjHVQIAjhbx//73v7/55huTyTRhwoTNmzf7+Ph0nUY5dPp6bZN+RExAiL8LLhEAcMCIr6urW716dVRUVFdrkdom/TenrinlIgzRAIDDRvySJUu6YHNQSrd+X2yx0mkjw/FBJwDgOqx4/l9OF9QWXlf1fsQ9JgK3YwUARLwD0bWadx8pEQl400eF43ZOAOAA7jIWsW/fvrs+9Omnn3bstthzrFSjN08c+oinUoorAwAcM+LPnDlDCCkvL79+/frQoUP5fP7x48dDQ0MdO+KvVKmP5Vb7eTiNHtgNlwUAOGzEp6WlEUJSUlK+/vprNzc3QoharZ47d64Dt4LFatv6XTEhZMboCNyRFQAcRrtxVldXp1Te/OC+VCqtr6934FbIPFdZWa+N6+OLtQoAwMF78axhw4Y9//zzo0aNstls33333ZgxYxy1CVQtxv3Hy+RS4eT4UFwQANAlIn7p0qXff//92bNnGYZ54YUXRowY4ahNsPenUqPZmjwiVC4V4oIAgC4R8YQQDw+P0NDQZ5555sKFC456/NdqNCcv1gR4yuP6+OFqAAAH0+5Y/GeffbZu3bpPP/3UYDCsWLFiy5YtjnfwlNJdmSWUkGdHhPJ4mAgPAF0m4jMyMrZs2SKVSpVK5VdffbVnzx7HO/ic4vrLleq+oR5RuGkfAHSpiOfxeCKRiN0Wi8V8vqOtqWu22L48doXPY6YMx7usAOCY2h2LHzhw4Nq1aw0GQ2Zm5u7du2NjYx3syDNzKuqbW0cNCPRxk+E6AICu1Yt/7bXXgoKCIiIi9u3bN3To0MWLFzvSYWt0pgMnrsmlwvGDu+MiAIAu14vn8Xjjxo0bOnQopZQQUldX5+fnOHNO9v18tdVkfS4hxEmCiZIA0PUi/v/+7/82b96sVCoZhqGUMgxz5MgRxzjmyjrtj3nVvu6yoX0xURIAumTEf/XVV5mZmewaNY6EUrorq4RSMnV4GJajAQDH1m7G+fr6urg44J1L88uaCq6pegW79Qlxx+kHgC7ai+/evfu0adMGDRrUNnVy3rx5DtCF//pEGSEEy9EAQJeOeG9vb29vbwc72qLy5tIqTd9Qj0AvOc49AHTdiHeAPvudDp68RggZ93h3nHgA6KIRn5SUlJGRERkZ2Xb/UnZGTWFhIacPtbRKXXhd1bO76yN+zjjxANBFIz4jI4MQUlRUhC48AICjRTyrsbHxwIEDOp2OUmqz2SorK9955x3uHmd5bUteaWNogAvu6wQAXUe7kybnzZtXWFj49ddfGwyGrKwsHo/bU8i/OXWdEDL+8e5to08AAF034lUq1dq1a4cPHz5q1Kht27aVlJR08CqU0tTU1OTk5BkzZlRUVLTtv3DhwnPPPffcc88tWLDAZDJ11kHeaNSdK6oL8lH0CsaiwQCAiCeE/dxTcHBwUVGRQqGwWCwdvEpmZqbJZNq1a9eiRYvS0tLa9q9YsSI9PX3Hjh1xcXHV1dWddZCHTl2nhIx7LAhdeADoUtodi4+NjX355ZcXL178wgsvXLp0SSwWd/AqOTk5cXFxhJDo6Oj8/Hx2Z1lZmVKp/OSTT0pKSoYNG9a9e/dOOcKGZsOpS7V+Hk6PhnvifAMAIp4QQhYuXFheXu7v7//+++9nZ2d3PE1eq9UqFIqbrygQ2Gw2Ho+nUqlyc3NTU1MDAwP/+te/9urVa9CgQX/+EX57ptxG6djHgnjowgMAIn7fvn1t2+fPnyeEKJXKkydPPv300+29ilwu1+l07Dab7+yzunXrFhwcTAiJi4vLz89vL+I9PRV/0OE1qg0/X7jh4y4bGxfC/8MWHfvj6v9zoH4Uj/q7UMSfOXPmrg/tIOL79et39OjRxMTE3Nzc8PBwdmdgYKBer6+oqAgMDMzJyZk0aVJ7T6+vb/mDDm93VonFahs9ILCpSffHXSJ/XP1/ziWO+lE86nfUn093ifi290stFktxcTGfz4+IiOj4jcqEhIQTJ04kJyezTz948KDBYJg8efKaNWv+93//lxDy6KOPDh069E8+/laT5ae8ahe56PFevvh9DQC6oHbH4k+ePPnaa695eXnZbDaNRrNu3bo+ffq092CGYVatWtX2JTs4QwgZNGjQl19+2VnHdiq/xmC0jh7QTSjAuvAAgIi/xdtvv/3xxx9HRkYSQi5evJiamrp3714OHRil9Mj5Kj6Pwa2dAKDLard7KxKJ2HwnhPTu3ZtzB1ZU3lzdoBsQ6eUiF+M0AwB68f+lT58+y5YtmzJlCp/P/+abb/z9/bOzswkhAwYM4MSBZeVUEkKGxwTgHAMAIv52paWlhJB//OMfbXvWr1/PMMzWrVvt/6ga1a3nS+qDvBUhWDcYABDxd/rwww9lMlnbl1VVVf7+/lw5qmO5VZSS4TH+WLEAALqydsfik5KScnNz2e2dO3dOnTqVK4dktlh/zK2WS4WDorxxggEAvfi7WLNmzdKlS4cPH15QUCCRSL744guuHNLZwjqtwTwmtptIyMcJBgD04u+if//+06dP37lz55UrV+bOnevnx5mph1nnKxmGxPf1x9kFAPTi72769Ol8Pv/AgQNVVVWLFi2Kj49fsmSJ/R/P1WpN2Y2WR8M8PJRSnF0AQC/+7kaPHv3ZZ58FBAQMGjRo7969RqORE8dzBHMlAQB+M+JTUlJycnI+//xzk8lUUFCQmppq/wej0Zmyi2p93WU9glxxagEA2o34zz77bN26dZ9++qlOp1uxYsWWLVvs/2B+zKu2WOnwfgGYKwkA0FHEZ2RkbNmyRSqVurq6fvXVV3v27LHzI7HabMd+qZKI+I/38sF5BQDoKOJ5PJ5IJGK3xWIxn2/vExAvlalULcbHevpIxQKcVwAA0sGMmoEDB65du9ZgMGRmZu7evTs2NtbOj+T0pRpCyOO90YUHAPitXvxrr70WFBQUERGxb9++oUOHLl682J4Pw2C0nL9c7+UqfcQXi9IAAPxWL57H4yUnJ7M3crJ/v5TUmyy2x3r64I1WAIDf7sVzy6lLtYSQ2J5YlAYAwLEivllrLLjWFOLn7O0qwxkFALiniK+srDx27JjVaq2oqLDnYzhbUEspie2JN1oBAO4t4g8dOjRnzpzVq1c3NzcnJyfv37/fbo/h1KVaPo8ZEOWF0wkAcE8R/9FHH33++edyudzd3T0jI2Pz5s32eQBVDbrrtS29gt2cZSKcTgCAe4p4Ho8nl8vZbS8vLx7PTkft2enwj+ETrQAAd2h30mRYWNj27dstFkthYeHOnTsjIyPtsHobpacv1UpE/OhQD5xLAIB77cWvWLGitrZWLBa//vrrcrncPleavFKpbtS0xkR4inGDJwCAe+/Ff/HFFzNnzly0aJE9V3+KHaXBXBoAgPvqxdfW1k6ZMmXWrFn79+83GAx2WLrZYssurFPKRZHdsDo8AMD9RPzixYuzsrLmzJmTl5f39NNPv/rqq/ZW+oXSRr3REtvDh8fDogUAAPcT8YQQSqnZbDabzQzDtC0sbD/YuTRYtAAAoD3tjsW/9dZbmZmZUVFREyZMeOONN8RisV3VrWs155U2+Hs6BXrJcRYBAO4v4rt3756RkeHm5mafdecU11usFEtLAgDcX8Tv3r176tSparV6586dt+6fN2+e/dSdf7WRENIv3BOnEACgPXcZi6eU2nnRlNLLFc1KucjbVYpTCABwH7149jYg/v7+SUlJbTt37NhhP0XfaNRr9OZBPbwxSgMAcH8R/+mnn2q12l27dlVVVbF7rFbrgQMHnnvuOTspuriimRASEajE+QMA6MBdBmqCgoJu2yMSidLT0+2n6OJyFSEkohsiHgDgPnvx8fHx8fHxY8aMCQkJadvZ2tpqJxVTSovLm52dRD5uuMcTAMB9RjzrypUrCxcu1Ov1lFKbzWYwGE6fPm0PFdeqDGqdaUCkFwbiAQB+Z8S/++67q1ev/uSTT2bPnn38+HGVSmUnFWOUBgDgHrW7gIGzs3NsbGx0dHRLS8v8+fNzc3PtJeLxXisAwANGvEQiKSsrCwkJOXv2rMlkamlpsYdy2YF4uVTo5+GEkwcA8Dsj/u9///u6devi4+NPnTo1ePDgkSNH2kO59c0GVYsxopsSA/EAAL+p3bH4gQMHDhw4kBCyZ88etVrt4uJiD+UWl2OUBgDgASI+JSXlrn3krVu3dn7EVzQTQnAPEACA3xnx8+fPt9tyi8tVThKBnycG4gEAfttdxuLZIRrmDp1ea0OzoVFjDA9U8jAQDwDw+3rxrPXr17MbFouluLi4f//+AwYM6OQuPDtdEqM0AAAPGPHbtm1r266oqEhLS+v0Wtn3WiPxoScAgHvDu5cHBQYGXr16tdNrLSpXycSCAE/cyQ8A4MF68UuXLm3bLi0tDQ8P79xCmzStDerWvqEePB4G4gEAHizi2UnxhBCGYRITEx977LHOLZQdpQnHjHgAgAeP+KSkJK1Wq9Fo2C8bGhr8/Pw6M+IrVISQyCBEPADAA0f82rVrv/jiC6VSSQihlDIMc+TIkc7txUvF/G5eCpwzAIAHjfgjR4789NNPTk528SEjVYuxVmXoE+KOgXgAgHvX7oyaiIgIk8l0j69CKU1NTU1OTp4xY0ZFRcVtf7tixYr333//gbrwFSqCpWkAAB5WL/6pp54aNWpUeHg4n89n93SwRk1mZqbJZNq1a1deXl5aWtqmTZva/mrXrl2XL19ue/P297lcjg89AQA8vFAgWXAAABE/SURBVIh/++23ly1bdo9vsebk5MTFxRFCoqOj8/Pz2/b/8ssvFy9eTE5OfsBp9cUVzWIRP8gHM+IBAB5GxCsUiqeffvoeX0Wr1SoUN98IFQgENpuNx+PV19dv3Lhx06ZNhw4depASTWZrTZM+zN+Fz+PhhAEAPISIj4mJmT9//hNPPCEUCtk9HSS+XC7X6XTsNpvvhJDvvvuuubn5pZdeqq+vNxqNjzzySHuv4OnZ0TyZKxXNlJLQbq4dP6wT2W1hqB/Fo37Hrv/3R7zBYJDL5efPn2/b00HE9+vX7+jRo4mJibm5uW2fg01JSUlJSSGEZGRklJWVdfD0+vqO7hp48XIdIcRdIe74YZ14idhnYagfxaN+h6//90f8fa07lpCQcOLEieTkZPaJBw8eNBgMkydPfiiHUVmvJYQEYI14AICH1YsfPnz4bWvEd/DRJ4ZhVq1a1fZlcHDwrX+blJT0ICVW1WsJIf4eeK8VAOAhRXzbYsIWi+Xw4cP3Pkf+oaus17k7i2USAc4WAMB9aXeOiv9/BAUFvfjii5mZmZ1Sn0ZvUutM/lhAGADgIfbis7Oz2Q1KaUlJidFo7JT6quq0hJBAL0Q8AMDDi/i2G/sxDOPq6pqent4p9VXW6wgh/nivFQDgIUb8tm3bGhsb3d3dDQZDXV1dUFBQJ0U8O50GvXgAgPvG6yDiX3zxRUJIU1PT7Nmzd+/e3Vm9eD6P8XGT4VQBADy0iN+9e/eOHTsIIf7+/nv37t2+ffufX5yN0uoGna+7TMDH0gUAAA8v4s1ms0gkYrfb1jD4kzU0G4xmK0ZpAAB+n3bH4keOHDlz5swxY8YQQn744YcRI0b8+cXhvVYAgD8k4l999dXvvvsuOztbIBDMmDFj5MiRnRHxeK8VAOAPiHhCSGJiYmJiYicWx/biEfEAAL+PXb+NWVWvlYoFbs5inCcAAIeKeLPFWtOkD/B0um01NAAA4HzEVzfoKcUoDQCAI0Y8lokHAHDwiMcakwAADhnxOvTiAQActhfv5iyWSYQ4SQAADhXxWoNZrTXhvVYAAAeM+Mo6diAeozQAAI4X8Vi6AADAcSNeRwgJRMQDADhexFfVa/k8xscddwIBAHCsiLdRWlmv88GdQAAAHC/iG9StuBMIAIBjRnxVHZYuAABw0IjH0gUAAA4c8Vi6AADAcXvxUjHf3VmC0wMA4FARb7bYapsM/h5y3AkEAMDRIv5Go85GKUZpAAAcMOKrG3WEED8PRDwAgMNFfE2jnhCCz7UCADhixDfpCSE+boh4AABHjHiRgOeG6TQAAA4W8ZTS2iaDl6uMh+k0AAAOFvGqFqPRbMVAPACAA0b8DQzEAwA4asSz02l8EfEAAA4Y8U2YMQkA4NgRj148AIADRnyj3kUukooFODEAAA4V8SaztUnTioF4AAAHjPhalYFilAYAwCEjHgPxAACOG/GNOoLpNAAA6MUDAADHIl7AZzxcpDgrAAAOFfGU0pomvZerjMfDAmQAAI4V8RqdyWC0YpQGAMABIx4D8QAADhvxWGMSAMBxe/G4ZSsAwMP2cFaDoZSuXLmyuLhYJBKtWbMmMDCQ3X/w4MGtW7cKBILw8PCVK1d2FPHoxQMA2GcvPjMz02Qy7dq1a9GiRWlpaexOo9G4fv367du379y5s6Wl5ejRox334uVSoVwqxCkBALCviM/JyYmLiyOEREdH5+fnsztFItGuXbtEIhEhxGKxiMXi9p5uttjq1QaM0gAA2GPEa7VahULBbgsEApvNRghhGMbNzY0Qsm3bNoPB8Pjjj7f39LpmA6UYpQEAeMgezli8XC7X6XTsts1m4/Fu/uSglL7zzjvXr1/fuHFjB083WGyEkLBurp6eCi42IkfLRv0oHvVzvf4/KeL79et39OjRxMTE3Nzc8PDwtv3Lly+XSCSbNm3q+OnFZY2EELmYX1/fwsVLhItlo34Uj/odoP4/KeITEhJOnDiRnJxMCElLSzt48KDBYOjZs+fevXtjYmJSUlIYhpkxY8bIkSPv+nRMpwEAsN9ePMMwq1atavsyODiY3SgoKLiXp9c06fk8xlOJBcgAAB4mu/joU02j3kMpFfB5OB8AAA4V8WqtUddqwS1bAQAcMOKr6rUEA/EAAI4Z8XVagtVpAADQiwcAAC5FfGUdIh4AwHF78TKxQCHDAmQAAA4X8TcadD7uMobBLVsBABwu4q02ilEaAADHjHiCgXgAAEQ8AABwL+J9MSkeAMAhI37Gk1F+Hk44EwAADhjxk0eEYzoNAIBjRjwAACDiAQAAEQ8AAIh4AABEPAAAIOIBAAARDwAAiHgAAEDEAwAAIh4AABEPAACIeAAAQMQDAMCfjKGUohUAANCLBwAARDwAACDiAQAAEQ8AAIh4AABAxAMAODxBJ35vSunKlSuLi4tFItGaNWsCAwM51HB5eXn/+Mc/tm3bVl5evmTJEh6PFxYWlpqaaudlWyyW119/vaqqymw2z549OzQ0lEPFE0JsNtsbb7xRVlbG4/FWrVolEom4VT+rsbFx4sSJn3zyCZ/P51z9zzzzjFwuJ4QEBATMnj2bc/Vv3rw5KyvLbDZPmzZtwIAB3Ko/IyNj7969DMMYjcaioqIdO3a8/fbbv1E/7Tw//PDDkiVLKKW5ublz5syh3PHRRx+NGzdu6tSplNLZs2dnZ2dTSlesWHH48GE7r3zPnj1vv/02pVStVg8bNoxbxVNKDx8+/Prrr1NKz5w5M2fOHM7VTyk1m81z584dPXr01atXOVe/0WhMSkpq+5Jz9Z85c2b27NmUUp1Ot2HDBi5eP6xVq1Z98cUX91J/Zw7U5OTkxMXFEUKio6Pz8/M51IUPCgr64IMP2O1Lly7179+fEPLEE0+cOnXKzisfM2bMggULCCFWq5XP5xcUFHCoeELIyJEj33rrLUJIdXW1i4sL5+onhKxdu/bZZ5/18vKilHKu/qKiIr1eP2vWrP/5n//Jy8vjXP3Hjx8PDw//29/+NmfOnGHDhnHx+iGEXLx48cqVK5MnT76X8OnMiNdqtQqF4uaAkUBgs9m40sQJCQl8Pr/t1yB2w8nJqaWlxc4rl0qlMplMq9UuWLBg4cKF3Cr+5iXL4y1ZsmT16tXjxo3jXP179+51d3cfPHgwW3nbNc+V+iUSyaxZs7Zs2bJy5cpXXnmFc+2vUqny8/PXr1/P1s+59m8ba5o/f/6tezqovzPH4uVyuU6naxtj5fE4+d5vW9k6nc7Z2dn+C75x48a8efOmT58+duzYd999l1vFs9LT0xsbGydNmmQ0GrlVPzuQeuLEieLi4sWLF6tUKm7V371796CgIHZDqVQWFBRwq36lUhkSEiIQCIKDg8VicW1tLeeu/5aWlmvXrg0YMOAew6czU7Vfv34//vgjISQ3Nzc8PJxwU48ePbKzswkhP/30U0xMjJ1X29DQMGvWrFdffTUpKYkQEhUVxaHiCSH79+/fvHkzIUQsFvN4vF69ep09e5ZD9W/fvn3btm3btm2LjIx855134uLiuNX+e/bsSU9PJ4TU1tZqtdrBgwdzq/1jYmJ+/vlntn6DwRAbG8ut+gkh2dnZsbGx7Pa9/PvtzF58QkLCiRMnkpOTCSFpaWkcjfjFixcvX77cbDaHhIQkJibaebUffvihRqPZtGnTBx98wDDMsmXLVq9ezZXiCSGjRo1aunTp9OnTLRbLG2+88cgjj7zxxhscqp/TFw8hZNKkSUuXLp02bRqPx0tPT1cqldxq/2HDhp07d27SpEnsdD5/f3/OXT9lZWVtkw/v5frBSpMAAA4LH30CAEDEAwAAIh4AABDxAACAiAcAAEQ8AAAg4oHzUlJS2A99PDitVjtx4sSkpKTr16/b/4Fv2LAhJycHFwAg4gHuSWFhoUgkysjIYD+ab+fOnj3LodWcoNPho0/QOTn14YcfSiSS0tLSiIiI9957r7a2NiUlJSsrixCyceNGQsi8efOGDBkSHx9/7tw5T0/PadOmbdu2rba2Nj09vX///ikpKV5eXmVlZYSQJUuWDBw4UK/Xv/nmmyUlJTab7aWXXnryySczMjIyMjKam5vj4+MXLlzIfuvGxsZly5ZVV1cLBIKFCxf27NkzOTm5oaEhNjZ206ZN7GNMJtOqVatycnKEQuGcOXOefPLJ3Nzct99+22Qyubq6vvnmm4GBgSkpKT169Dh58qTJZFq2bNm2bdtKS0tnzpw5c+bMjRs3lpWVVVRUqNXqKVOmzJo1i1K6Zs2a06dPMwwzYcKEl1566c4WEAgE+/bt27p1K6W0Z8+eK1asEIlEQ4YMSUxMzMnJEQgE69aty87OXrVqlZeX18aNG48fP75v3z4+n9+7d+9Vq1bhooK7owCdsWz3o48+WltbSymdNGnS0aNHKysrhw8fzv7thg0bNmzYQCmNiIjIysqilKakpCxatIhSmpGRMW/ePErp9OnTly9fTiktKioaOnSoyWRi79BCKW1paRk3blxFRcXevXtHjRpls9lu/dYLFiz45JNPKKXl5eVDhgxpbGw8c+ZMSkrKrY/5+OOP2WU46+vrx40bZzKZ4uPj8/PzKaXffvvtxIkT2QLS0tLYakeNGmU0GquqqgYMGMDumTBhgsFgaGlpSUhIKCgo2LFjB1u2wWCYNGnSsWPH2lrAZrOxLVBSUjJt2jSj0Ugpfe+99/71r3+xLXDkyBFKaXp6enp6Ovt9s7OzLRZLbGysxWKx2WwrV65kWxLgTgL8kINOER4e7uXlRQgJCQlpbm5u72HsHQX8/f3ZVZb8/PzUajX7V5MmTSKEREREuLm5lZaWnjx50mg0fvXVV4SQ1tbWK1euEEJ69uzJMMytL3j69OnVq1cTQgIDA/v27ZuXl+fk5HTbN83Ozp46dSohxMPD48CBAyUlJUqlsmfPnoSQxMTE1NRUrVZLCHniiSfY2qKjo0UikZ+fX9uCrmPHjpVIJISQESNGnDp1Kjc3l133TSKRjB8//vTp0/Hx8be1QFVV1fXr19n7zFgsFvbbEUKGDBlCCAkLCzt37lxbt4zP5/fr12/ixIkjRox47rnn2NcBuBMiHjqHSCS6OVbIMOz/to0Zms1moVB48wIVCG7b+PXa/c8eSqlQKLTZbO+++25UVBQ7GuPi4nLgwAGxWHznr61t2zabzWq13uVfxS3fq7y8nP094NZXYEfD24psu3nAna9gtVpvq5xN8DtbwGq1jhkzZtmyZYQQg8HAFsYwDPuwW9uH9cEHH+Tl5f3000+zZs1677332FtDANwGb7eCXXB2dtZoNCqVymQyscu9/qYDBw4QQi5evKjT6bp37x4bG7tz505CSF1d3YQJE27cuHHXZ8XGxrI9/YqKil9++aVv3753PqZ///7ffvst+6MiJSXF399frVazNyY7dOiQn59fe2tzt6Xw4cOHzWazWq0+duzY4MGDBw0atG/fPpvNZjAYDhw4MGjQoDufO3DgwMzMzKamJkppamrqp59+etsPpLYfHhaLpampacyYMeHh4fPnzx88eHBxcTEuIUAvHuyXXC5/4YUXJk6c6OfnFx0dfWv39q4YhtHpdElJSXw+/7333uPz+XPnzl21atX48eNtNttrr70WGBjYNrJxq2XLlq1YsWLPnj08Hm/NmjUeHh5Xr1697THTpk1bvXr1hAkTGIZZvny5XC7/5z//+eabbxoMBqVSuW7duvZqa9spkUimTZum0+n++te/hoSEBAUFlZWVPfXUUxaL5amnnho5ciS7TPmtIiMj586dO3PmTEppVFTUX/7yl7t+l7i4uJUrV65duzY5OXnixIlSqdTPz48dBQK4yzWJGTUAD1fbjCA0BWCgBgAA0IsHAAD04gEAABEPAICIBwAARDwAACDiAQAAEQ8AAA/g/wMM5d0V3U0LLgAAAABJRU5ErkJggg==" />

This curve quantifies how much of the total, 64-dimensional variance is contained within the first N components. For example, we see that with the digits the first 10 components contain approximately 75% of the variance, while you need around 50 components to describe close to 100% of the variance.

Here we see that our two-dimensional projection loses a lot of information (as measured by the explained variance) and that we’d need about 20 components to retain 90% of the variance. Looking at this plot for a high-dimensional dataset can help you understand the level of redundancy present in multiple observations.

# PCA as Noise Filtering

PCA can also be used as a filtering approach for noisy data. The idea is this: any components with variance much larger than the effect of the noise should be relatively unaffected by the noise. So if you reconstruct the data using just the largest subset of principal components, you should be preferentially keeping the signal and throwing out the noise.

Let’s see how this looks with the digits data. First we will plot several of the input noise-free data (Figure 5-88):

In [ ]:
%matplotlib notebook
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

> **Figure 5-88: _Digits without noise_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAjwAAADsCAIAAAD/zOvLAAAACXBIWXMAAAsSAAALEgHS3X78AAATgklEQVR42u3d0XHc2LUF0OYr/cuKgKMIZEXAUQQ0I9BMBDQjoBUBrQhkRqBhBB5GoGEEsjKwIqB/3+u9/XQMAi1yvNYnCt19cXGBU121cXB0f3+/A4Cn4H9MAQCKFgAoWgAoWgCgaAHAOp495MO//PLL3pa//OUvuduPP/442e0Pf/jDIY88R7Xb7f75z39OhvqnP/3pYOP89ddfJwP44x//OPzsKv76179OJuqHH36YDOnApz7P8k8//TRZ3gdekDl7u93ub3/722O7iQwvpd9+++2Qo8olWkdVz/Ld3V1ufP78+d6Wf/zjH6ss5j//+c+TUeUqzQ9uejXlbSfnc9Pbjn9aADwxihYAihYAKFoAKFoA8Ng9KD2YgbEap8mEyTAWtWlIr2Zsbm9v97YMw3tryYTVmzdvcrdhkGkVNRaY6aYa1sp0U42Q1fjZdnKl1ezlgeUZzNW42+2ur6/3thwfH0++bS156us4Ly8vH+H9Lq/6um6HycO1cnrDXGWu23p3WiW8V9fPzc3N5LNHR0d7W169erX4qP3TAuAJU7QAULQAQNECQNECgMdumh6sSY9MmNTdMitY03f52RVDevnlw4zNgXNlGc2qwZucmXfv3m00pNqXL2OBNQGYp/7AQcHaGy1TWLWH2zCAV6OwC2QU7cuXL7lbBkeHrf/WirrVNOlkiR5YPafDw6mnfruWenmTGear6znNcS646OqFk05OTiZXxLpT558WAE+GogWAogUAihYAihYAKFoAsJJp5L0mIIdJzckHV1SbXWaw9evXr5NvO3BEO3O6dUpzt9PT042GVAeQmeCaEs7ZqwtpuxeE15fT51CHsf461GEEfME815e+57qtV9N2U5pnsD6VceBnRTJUPYxZ19tFlY+j1GWzQH7P69evJ+u2nuVVnsEYfknOya497TAM0PunBcDvjaIFgKIFAIoWAIoWAChaALCSB0XeF8fBN80916RyhkpfvHixeKirqN+cAdwaKk01272dTMTWyHuGX2vz7zzGZYshv+fi4iJ3e/v27eTb3r9/nxs/fPiw0ZTm4GtoO99XUI9xeF2ssm5rPDpXcj31a/XIz++pb5wY5uDrRbfd0y/Dm8zt7e3els+fP280pfUCzGcb6l30/Px8ci7yjjEcuX9aADwZihYAihYAKFoAKFoA8NhN04M1TFIzISmzMfWDNVz03dWhrtIMtDZarYm19PHjx8kJOqQ6gExh1QxbJs2WdaHNMTx//jx3u76+XraSD7xKHxJXq2HOVWTEK1Nt9aqvKcdPnz6tcnHlqGoC8Ojo6JuX0m7LoGBdaW/evNnbcnl5OTmnwyzuWhHNHPxDbo95Kxgmpf3TAuDJULQAULQAQNECQNECAEULAFYyjbzX0GTmHWtmcRhkXKuh51ORPXx3raHn3d1d7nZ2dra35fT0dPITayW2M5JeU8KZe64dS9caVY6htiLNdVsHX/vqbvdowbBr8PBhgO2i+bmoapZ92FI5j3qV50n+3f0kH4HYLt0+v5HmqOrgcwJfv36du2Xv7GUPkEzUk5WDr+28h3XBPy0AnjBFCwBFCwAULQAULQB47B6UHsxQSo2pZMJk+NLrFWUQq8btbm5uJkOtwb//VA3eDFtS5jznyOtZWytUlvM5DH/WAWTD3AMvhq9fv250ludypQ27J9eU43ahuJyWGgvMwFgd0nYpx3rl5qgO3Gm6/lzOTH2NfYYM601suxh2fnO9Ow0zw4tjov5pAfBkKFoAKFoAoGgBoGgBgKIFACs5ur+/NwsA+KcFAIoWAIoWAChaALCCZw/5cPaYqr0Hh13IFr/IckXZrK/2CstWWmt1MMtJqH35hm+5XUXtLJejqi8nzVHVXnPZzm6t19dWuUrrJNcDX2We61LPMdTXLtc3WafPnz9/c21vN5/1cLabz+HdqY6qnvq6SusKX8Vw/Q9vpNt18hy+E3k4Tv+0APj9U7QAULQAQNECQNECgMfuQenBDL3U9+deXl7ubak5nNy46atja4Lry5cv39yya8GktUJQecj1m3OutntdaQ19ZX6yDiAnqr6NN49xrfRgjZDl7M2Tdauc+rr+b29v97bkm2rr1VSjWdtlBSeLoU7Lpu8Izlfo1rtHLuZhPHhTOYb6RuDcrS6kvBLXWgx5z6y3R+lBAFC0AFC0AEDRAkDRAoAnYpoerBGyzAq+ffs2d8v+VDXTVQMz2xnG7U5OTnLjdtGs/OYaZMreaNulB2vyJ09WDTLlqa+JuNrqbbuznMuvRknrWc7ZWNAzs2Yjc0rrbnlEm6byJuPM3ONut7u6ujrkqPIGNZzkYchwU7n+a//AYWfU7e5OOaX1pn19ff3N+8BDxumfFgBPhqIFgKIFAIoWAIoWAChaALCSaeR9GKsdtrjdNKRb8/QZFK6tHg+pxmozVFrn6sCR3Ilh8rtmZNcK6WZQONO3u5bGrgP4+vXr5AStZdisOQdw4MUwfDRlu8cYhj9Xz2mu0rpu6+Bzntdat3lOa+fxfKCoPmeynbyL1gdyclrqkycLnhXxTwuAJ0bRAkDRAgBFCwBFCwAULQBYyTTyfuAW7A9RE8C58fj4OHfLkPF2Eeeal63tkCfjrEH/Q/b/rn2pc/bWDb9OTn3KoHAdfLXKeqiTMMxP//zzz9/3+qorLb18+XJvy6tXryYLfq2s/OvXrxd/tj4pkaurBr4XyEOuc5UPFB24wX/+3HAG6jkdvtbAPy0AnjBFCwBFCwAULQAULQB47I7u7+8n+9XI0IsXL/a2fPz4MXf78ccf97bUvrqZI9ouubf7N4m1s7OzvS3Pnz8fzsZGak/MjJ8dckhzGbiq5zTPRa6ZZat0GFasvXFrvvSQ3WmHS/TTp0+523bXTkbI6uydn58vO8ZlM5ynviZCM+1Wf67eoDICt9Yk52oftvFdK774XVZy3tmGKWL/tAB4MhQtABQtAFC0AFC0AEDRAoCVTBvm1s6MJycne1tqzDSDjPXbNg24D49o8W5ryYj2+/fvc7cM4tdsdw6+Znn//1atNUyfWdu6W66HGo9eK0eex1sfGMih5sMbu6Wx+4nhlNaTla1UD3zh5DiHLW7rEs3lXRfDN7sJ56mvvadzSuu5GPatXuvU59HV3Q75uMVw8MNG6nXkNzc3y069f1oAPBmKFgCKFgAoWgAoWgDw2D17yIczFlgDQpkwqZmuA6uZq4xm3d3d5W6ZolkrZJjpppqoycHXXpM5qpqIW5AenL+ffs/p6enkqDeVq7S2Rd5uVDVzlT9Xk5bDpqKHvHDqVZ8BvJqDzfXwzaDgQ+Ri3i4jWtUbxXBUiy+6teR9++LiYvLBvK/WUz+8i/qnBcCToWgBoGgBgKIFgKIFAIoWAKzk6P7+3iwA4J8WAChaAChaAKBoAYCiBcB/kwc1zM2+n8O3udd2kLXt5nZqI9qXL19OPvv58+dvHuMy2Wb03bt3udvHjx/3tgxfeb7AsGFuvoW9rpDaEzMbcR64jWn9udrWebt2rjmG4STUKd3uahp2d83daqvf2rd6O3l9zZt35wpfazHkbbNedHlOD3yZ5ADqVZ+Hs+5q9E8LgCdD0QJA0QIARQsARQsAHrsHpQczilNfTp8bb25ucrfMv2364u2aHjykGhDKhFV9P/3Z2dnelu16SNaJylhgDTLlxho3yoVUd1tLBsbqMQ5f/r1Azt5ut7u9vf3mlroeDhwhy+BoverzDevbzefcMMlcg44Zgau7raKu/+EtKz+71sznuq2n/uLi4pv39ofc3v3TAuDJULQAULQAQNECQNECgMdumh6saZYMAZ6fn+dumQ3btOFYTeVl7iVHVZ2cnOTGVWKNNdKTM1N7o+VuNZC2yjzXLxnmpjLvVD9YW1auoq7bn3/+eW/L1dVV7pYxufmyWXDqj4+PJ+f0kBm8GlerzTBTrttNw8BDudKG7VI3nfnFmcY6zmGydxWZEd21SOGwRaF/WgD83ihaAChaAKBoAaBoAYCiBQArebbu19WUcPry5ct2h1Qz4tnD8XHKppw1wZ+h0u+eJ67x6IzL1wD9dpH3+p7vfDCj7nZ0dJQbc54XDL5m2VMNCtfeoxupay9t91jIQ5ZfvQ/kzNdjrDeo7Y5o+FBQZu7rkLYLuOdSz6dHhge4E3kH4L+BogWAogUAihYAihYAPHbT9OAwkVKjOBl6qXGjTPssa05ak2A5/hp0vL6+3tsyfMX1WjI1VA8nJ/C7v8u8BplyYz2ctQafS6i+DjwneR7JWyXoWLNhee2cnZ3lbhl9HEZ2Fxgu/rpbRh83zT3mqR829p1bZZXWO2Quqrpu0zCGupZhu+FcDy9fvpwMftjg2z8tAJ4MRQsARQsAFC0AFC0AULQAYCVH9/f3iz+cGdCagMxIbo02Zhh6uz6qu3+Tp8+YbE3n106mq8gYaH3YIAO+h+yjOpfntE7ddsndX375ZbKxDqDGjh9yvfynapQ808OfP3/O3Vbp7lqn5fXr1yse44cPHw551ad64dRVmqdjQQi+Rt7zZNUB5FVfv612Cn5s94E6+OHI/dMC4MlQtABQtABA0QJA0QKAx+7ZQz6cOZ8a1ko19HLg/NswXnV7e5sbM0e01qu4h5OQsa4a9MpvG7ak/KZMhNZzmuth+Ab3tdT5zI01tjR8lfhaMjA2DKnWkOEqC7J+yfHx8d6W+nL6oZz5TdODGWO7ubnJ3a6urnLjKg1z65fkxno5f/fMcI6qLtG86usSzWVTE925CP3TAuDJULQAULQAQNECQNECAEULAFbyoMh75kdrUjNjkTVhvEqidK42os32uPWItou855TWpwiGYej8bP3ggpkfRt5zkvODj0FdkJeXl4ccQ85MXXvn5+eTlbyKujByUdV2qDn4+m3fPbT96tWr3O3ATXvz52r4OzceeJx59xg+41Rvj/n4zfAu6p8WAE+GogWAogUAihYAihYAPHZHh3x9OAD4pwWAogUAihYAKFoAKFoA8Ag8qPdgdr6qre2yx1TtrLXWm+Cr7BY4fBF7HepGQ6oDuLu7W/wTp6ene1uGvcL+t9qXLxvl1Ymq3fNSNq/btBHlsHteXcyrNJms05ItBOuohkParitdrod66vNwhi9TX0u9n+TG2gzzwH1Qhw05c+brzeHvf//7N8/FN9VWonkG379/n7tlO8e6GofL2z8tAJ4wRQsARQsAFC0AFC0AeOym6cGapLq+vt7bUl8Dmom4mtwbvup0mRx/Dd7kxu2iWTVClgN4+/btZErrXK0SyKwpxxzn2dnZ4p/ISV4r/DYMQdWJ2i7YVkf19evXvS3v3r2bfFu96DIwtt3LtYfLu05yXpjLFm0mQusFnjNfw7GLg23L1DFMRlU/mDO/ID1YvzknOZOK9bM1tCw9CMDvn6IFgKIFAIoWAIoWADx2z9b9upoSydhS3S0zJytmeDJc9/z588VDXSXbViNkaRhs265z4zB1dn5+Phl8/bYF6aahmnrNXNmClozrrsb5lOZFsV3KsQZHM+VYA655OdclmrsNG/Etm9K8cuvPHTg9mDNT7zA5V/WoV7k71W/OW1a9cDJVnk1Q/dMC4L+CogWAogUAihYAihYAKFoAsJIHNcxNw/RtzVNul9zdtQ6z9eXfFxcXe1tq6ncVw1fR55CqDx8+5Mbt3rme6ou387mCYW/QtdTnCnJU9VxstyCH+ew6pTmB9dpc5RGI4Thr/+vht631tEOerOPj49xt2IO4rtLtrqYc/Js3b3K3fLRgu0c16sHmZVJvj1dXV3tblj3G4J8WAE+eogWAogUAihYAihYAPHZH9/f3k/1q1i6jOMNvq7msTOxs10d1rg41QzsLwlo1sZbfXCch0zg1QjbsybtA/lz9rcwgDd+5vkyOoZ6+bPlauyfXmR82LV0gz2n95mEj2nXzWv/nlnF0tLfl06dPuVuOqo4zW9OuFdKrCzIvuhrAqwsyNy449fWqz/jlly9fcrfh3fURqhOVUzHM6/qnBcCToWgBoGgBgKIFgKIFAIoWAKzk2QF+I7OnNdC5SovP1WUkd9ceAFjQubIeb26syd3tsuxDGQevWd4Ml296loeR99xtGESup36tcHmutHqWcwDbNfYdthuuXVOH/aCHzXYXGPbnrQ821FWa1/iCdH69UeT3DJ+H2W72tr5d7NojBMP59E8LgCdD0QJA0QIARQsARQsAHrtperDGP7Jh7rBrau1Pulbj0aGaz8nx1xzRxcXF3pYaoFoQ68pETY0v3t3d7W358OHDRhNVz2lG5mosKk/0du8srxNeo335LvP6evUDRzRzAuupz1HVZtarqJdkJkLr7OX1Ve8h2131dUHmqOoFXoea87xgMQ+7MNdbRw5+u/RgvT3m8dYB5JQOv80/LQB+bxQtABQtAFC0AFC0AEDRAoCVPCjyfnp6urflxYsXudvJycnelgx2by0j6fWIMmlac8+vXr3aYki7lsaujwdcXl7ubdkuSl7jqvm0Qx1nnugDt0WuZ/njx497W87OznK3ekSrzPPwyZAaB89lc+BnRa6urva25BMg9eawVmfhoeGTAHVUGetf66qv6z/Xw/X1de623WMtw3Hm1VSfdsgFWS/DutE/LQB+VxQtABQtAFC0AFC0AOCxO7q/vzcLAPinBQCKFgCKFgAoWgCgaAGgaAHAI/Qv/fKQ7KOdJd0AAAAASUVORK5CYII=" />

Now let’s add some random noise to create a noisy dataset, and replot it (Figure 5-89):

In [ ]:
%matplotlib notebook
np.random.seed(42)
noisy = np.random.normal(digits.data, 4)
plot_digits(noisy)

> **Figure 5-89: _Digits with Gaussian random noise added_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAjwAAADsCAIAAAD/zOvLAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2ddXRU5/r9X9w9SJHgUNzdCRR39wKF4hQJVkqB9uJwgQC5uBYrbsWKBIq7hOABgrs79/sHa81v+uyHM2/S3N8irP35a9izZ3LmnDPnYdbaZ7+R/vvf/xpCCCEkIhCZu4AQQgiHFiGEEMKhRQghhEOLEEII4dAihBBCwoeoqnrlyhWhpE2bFm3Dhw8XyoABA9A2aNAgoQwePBhtM2bMEErBggXRli9fPvd/Hjp0SBguXbqEr2rYsKFQIkWKhLYUKVIIZefOnWhLkCCBUNavX4+2Nm3auB6/efNGPBs9enSbI7R161YU/f39hXLkyBG0Xb58WSgzZ85EW9u2bV2Pnz59Kp6NFy8evuTXX38Vyk8//YS2Dh06eDwZjDHJkiUTSpQoUZx3y8qVK1GsU6eOUEJCQtB2/vx5oRw+fBhtvXv3DpfvGB764OBgtE2fPl0oY8aMQZtl3HfDhg0ez21jTP78+R3e5OXLlyg+e/bM4+FTsdxyvPh86vrjzKRJk1BcsWKFUHLnzo22CRMmoJg3b16hHD16NFzOENzUrl27oq1Vq1ZCmTNnjs35FhgY6PxxHj9+bHMVxfO2bt26aLtz545QkiZNiragoCCPlzX1WPCXFiGEkAgDhxYhhBAOLUIIIYRDixBCCIcWIYQQ8rmjpwcxq6OGSfbu3SuUXLlyoe3EiRNCUWNRvr6+YfgAmDA8duyYzQtTpUqF4vXr14WydOlStPXt21co7kFBFcusIEYo165di7Y1a9aE7Xg3btzY2YCBsU6dOqFtwYIFQmnevLnNBqhJs2vXrtm8Nl26dK7HGBRUWbduHYr169e3Ob1fvXqF4rJly0L7wTHDqebo8J2LFi2KNky9VqtWDW19+vQRinNQUEUN4J07d04o3t7eaMNknfpVwliv+v0NQ3rQ/WxxgRHNdu3aoa1GjRo224BRW/OJtK3zmaam+xDM+Kmnd/Xq1W0uy38bA1HlIIgcWfk90717d6EULlzY5jvevn17tE2ePFkoamiTv7QIIYREYDi0CCGEcGgRQgghHFqEEEI4tAghhJDPnaiWPjX08u7dO6FgUNAYM3/+fKGofXoDBw4UChbc2VCzZk0Ue/bsKRS1Dit58uRCKVWqFNoOHjwolMyZM6PN/U88evTIZpdiGPLWrVtoS5gwoVDmzZuHNvyj2EtmjIkTJ47r8YsXL8SzGBRU90ChQoVsjk63bt1QzJYtm1A8RkCxW88YU7VqVaFg+aExJnHixEIZOnQo2mLGjImimuhzJmvWrDa258+fC+XevXtoq1ChglCiRYuGtjDE7RA10IsVfFevXkUbJiTz5Mlj80dr1aoVLpc29SuJeb9vv/0WbWpk18/PL7TbgF/ADBky2FxdDxw4gDb8irVs2RJtmOhLkiSJ83bev39fKNgfaLSUr4+PD9pKlCjh8f2NFllUr3XYmclfWoQQQiIMHFqEEEI4tAghhBAOLUIIIRxahBBCCIcWIYQQEk7YRt7fv3+PYtmyZYWiZhZbtGghFEyCGmO8vLyEMmrUKLRhE6hg2LBhKE6dOlUoah1qzpw5hYIZdPP3jLglGFLPlCkT2jDljDFuY0yBAgWEgkFk9Y+q1ZzuBZdYM2q5SnqsWLFQxPpdTLcbY06dOmXzcdxRs7b46bAb12g3RYSEhKBNzenieu1qda+at3bx5MkTFHGt8eXLl9vsqyZNmqAtbty4QlFvinDPTL9+/Vo8e/PmTXzJ8ePHhYL90UaLWQcEBKANbz9Q70XxSMeOHT0eKWNM//79hdKjRw/LP4FVxRs3bkRbpUqVHL6AKpUrVxZK06ZN0YZ13q1atULb6dOnPb7Q/L3o+cGDB+LZihUr2mx5sWLFUMQ+9PPnz6Nt7ty5QsF0O39pEUIIidhwaBFCCOHQIoQQQji0CCGEcGgRQgghHFqEEEJIOBHJMtD87NkzFBs3biyUdevWoQ2bld++fYs2jMt7e3ujTWR5MawZObLVJMZst9Hy5RMmTEBb2FK5gm3btqHYr18/oajd2DNmzBAK5lyNVtm+ZcsWtGEO2ONLypUrJxTsbDZap/uiRYvQdvfuXY/njDEmXrx4rscPHz5Ew+3bt4WCjddGq/wfPHiw8t2AiLMxZuTIkULxeA8GNnarW2XJxIkTbTa+U6dOQvG4ZsLMmTOF4p6KdoHl99jqbbRkfNu2bdE2fvx4oajJ+NKlS4d2R/3xxx8oVqtWTShlypRB2/bt2/9/XoXx66wG8fEirF698eqqrgPgDObRjTHt27cXCi70oZ426uISWCS/fv16m/ONv7QIIYREGDi0CCGEcGgRQgghHFqEEEI4tAghhJDPHb0wF5M/auUilnKquIe+PtK9e3e0YYxH7esUTalYuKm+CkOAjx8/Rhtm29S+WjVVFVquXbuGYpQoUYRy7NgxtK1du1YoaqBRTfQ58+HDB6Go1ZnYkdq6dWu0YdRNPTq///67UNKmTYs299BdUFAQGm7cuCEUNYU4ZMgQjy80xmTIkAHFIkWKCOXs2bNoy5o1q8OhmTVrFr6kTZs2NvsKw7ExYsSw2RUeKV68uFBWrVqFtq5duwrFz8/P5v3VNCbiMSg4adIkFLHYukqVKmjDuF3nzp0t98/s2bM9XrLEabNhwwbxbOzYsfEl7dq1E8q0adPQhhfh6tWro+3HH38Uilpr6w52JefLlw9t2HsbHBxscwTr1q2LNoxPr1mzBm0NGzbkLy1CCCERFQ4tQgghHFqEEEIIhxYhhBAOLUIIIYRDixBCCAkn9Eh0ypQphaLWtmIYcf/+/WjbtWuXUJYuXYq22rVrC8Wm5/H9+/dCwcSnMWb69OlCweZTo6VyK1WqhDbs4vzqq6/QljdvXoctx/sKjNYOuWPHDrTVqFFDKMOGDUMb9tUmS5bMw/9i7OqGsZp29erVaMucObNQrly5graQkBChNGjQwHkD1BQvJndXrFiBNjymceLEQVuXLl1QxJTz5s2b0eYeeU+VKpV4FtPtxpiVK1cK5cmTJ2jDxLmKWqDsTLZs2YSi1qEOHz48bNeaV69eoYiF13gTi81xQZYsWYJihw4dPH7jPvVtGjBggFAuXLjgvA1FixYVyqBBg2w2HkPwRqs8PnLkiOVXw5l58+bZbACidprjKYrnttFuP8CbCvhLixBCSMSGQ4sQQgiHFiGEEMKhRQghhEOLEEII+dyJpC7YbMno0aOFoi49joEZbCwNM5ifyZ8/P9qwCReraY0xjRo1svmj/2SnuTh58iSKCRIkEIoaIcPC3HPnzqEtZ86cQilVqhTa3ItocflzdZ/g+tnp0qWz+dS5cuVC8cSJE6Hde4cOHbKxqWFFPEPUblz1AKVIkUIoXl5eztuAkVo8Lkarn1bPZDxv69Spg7b+/fuHdpfeunXL44dVUb/19+/fFwqmiI1WtqvmdcMAZpuN1uCsNk2rF4eHDx8KJVGiROGyqbVq1RKKWh2LtGzZEkWs81Y7r8MAJoQxr6teMQ4ePIg27DjmLy1CCCFfGhxahBBCOLQIIYQQDi1CCCEcWoQQQgiHFiGEEBJORAqX9DYhhBDCX1qEEEIIhxYhhBAOLUIIIYRDixBCCPnEysXIlClTUOzUqZNQ1NV4sVmrfPnyaJs1a1a4fCS1hS9LlixCwZI9oy3s26xZM7RdvXpVKGnSpAntdo4dOxbFFy9eCCU4OBhtuK4olgEaY5InTy4U9xV1/wk9e/YUyr///W+bF6prmOL+xDWXbXj58qVQtm7dijZcnxpXgjafaJhEcdWqVWhz7wM8c+aMePaXX37Bl5w9e1YoR48eVb6xUeV39u3btzbbqZ5vvXr1cj3GXr569erhS3CRbnWt8BEjRtjsz/8dz58/R3HcuHFCUZcSxvo+Y4yvr69QcufOHS6biickXjONMfHixRPKggUL0LZz506hlC5dOrSbhEWLxpibN28KJUeOHDbnHhZRGmPu3r0rlPjx46MtZcqU/KVFCCEkosKhRQghhEOLEEII4dAihBDCoUUIIYR87tjWOJ06dQpFdRVaZMKECUI5ffo02qZNmyaUd+/eoU1dVNQdjLgYLfKE0UdjjL+/v80nwpRazJgxw+V44Dqq9+7dQxvGzwYOHIg2XIv28ePHaHOOFE6aNAnFkJAQoTx69AhtHz58EMr06dPR1rVrV6E8e/YMbe75UtUQO3Zs+Z+yyMp/yzCsFT16dLS1atXK5ixVl5ZWc1Auli1bhmKDBg2EUqJECZt3njNnDtowB+sR/OKoxzRbtmwe96fR8p94HVD3nrrKrY+Pj+uxmpaMFi2azVZZgklOdT8PGzYstO+sxmIxoqxufJEiRWz+xPDhw4VSv359tGXKlCm0G1+zZk2hbNmyxeZiW7t2bbRhkpO/tAghhHxpcGgRQgjh0CKEEEI4tAghhHBoEUIIIZ87tt2DzmkoFx07dkQRAzN+fn5o69Gjh1A8BgWN1mqF2RUVy6BgtWrVUIwRI4ZQ7ty5gzb3BNfFixfFs5cuXcKXYFbwxIkTaMPqPzU9+P79e6FgkZdHunTpYmPbvXs3iv/6179sXovZS+zB8/jRjFZfqUbRkPnz56Oo1k7iocewlkfUNBfuK0wqqp9RTdOFgeXLl9sc+tGjRwtFjbphtHjfvn1oS5AggVDcg4Iq27dvR7FYsWJCKVq0KNr27t0rFOxINJ/I02LQ+unTp2hzbwjEHlS11XDPnj1CUSs6mzRpYnPo+/Xr989PhqZNm6LYvn17oaxduxZtly9fFooaFLxw4YJQ1KbQDh068JcWIYSQiAqHFiGEEA4tQgghhEOLEEIIhxYhhBDCoUUIIYSEE7aFuWqkG4Oh6irRWDRZvHhxtGELrbrceNjAhcPVItqAgAChqN2miRMnFkrGjBnR5u3t7XqMy1cnSpQIX7Jw4UKhJE+eHG3t2rUTypo1a9CmLoatnASO1aJXr151/mgf+frrr9GGK2qrC2/PnDlTKGqxr3vQX03HYktsnjx50IavVUuWt23bhmL58uWFUqFCBbSlSJHCYZeqQfzChQvbHCxc4v3YsWNow0JhXKxdMG/ePKGoPcLp06cXihou37Fjh82xSJgwYWi/y/v377fZ4Tdu3EAb3kCCiWpjTJs2bVDEe3Ju376NNo+RfRsqVaqEIt53hGejup/V6224MGTIEBRxtxQqVAhtWEit1lvzlxYhhJAIDIcWIYQQDi1CCCGEQ4sQQgiHFiGEEMKhRQghhIQTti3vDx48QLFKlSpC8VjO/ZHAwEAUMRapVh3XqVPH/Z+PHj0SBkzuGmNWrFhhs2G+vr5CSZo0KdpKly4tFI93Dpw+fVooJUuWRFvBggWFkiVLFrS9efNGKDlz5kTbkydPhGLZ1u8OptvV80EtSscYtxqvt7zvwp3g4GAUN23aJJQMGTKgDUPqake+esMAnm/O6XYVNQGMn+j48eNow7tKgoKC0JYtWzahYKm2MSZTpkyuxy1bthTPrlu3Dl/y22+/CUW9K6Ns2bJCUTPoffv2FcrUqVOd9556OwTeXFGjRg2bS5aKegsElqyfPHkytIdeXQxhy5YtQnn16hXaGjZsKJS2bdvaXBw8YnmhwJ1ctWpVm9MbO/iNMdeuXRPKoEGD+EuLEELIFwWHFiGEEA4tQgghhEOLEEIIhxYhhBDyuaOnB7FWEms6jTF79+4VyuzZs9HWvXt3oWDhrDFm+vTpQkmXLp3HD4BFtN26dUMbxmzUwlzsqcTiUaOlB507Zz/15xDMCv75559ou379us12Hjp0SChqw2YYwCOItchGiwWqOwp7ijHOZIypXr266/F3332Hhl27dglFjczhxmO203wivKoGX51ZtGiRUNKkSYM2TFi9fv3a5v3VICWmHNWCZmfcd7gLDOKqvbFjx44VSq9evdCG7bceC5q/+eYbNJw/f14onTt3RhvmS1XUROiUKVM8nm/GmFKlSjm8s9orjR+5QYMGaMMrG0Y0jRZ4Vs/kRo0auR5j4rpu3br4kpQpU3r8W8aYixcv2hz6oUOHCoXpQUIIIV8aHFqEEEI4tAghhBAOLUIIIRxahBBCCIcWIYQQEk7okXfMJYeEhKCtSJEiQvnhhx/QhrFItWHz7du3YfgA6dOnF4qaEffx8RFKxowZ0Ybh3fv376MNw6+nTp1Cm3uJbZkyZcSz+/btw5dg9tTf3x9tefPmFYraDIsFly9fvkRbzJgxXY9btWolnp07dy6+5NixYx4VY0y0aNFsDuLGjRuFMmzYMOeXjBo1CsU+ffoIxTl//P/+7xZZ+d9b7dq1UcQi2jt37qAtWbJk6uOPqF3JzZs3Fwpm5VXUEtgJEyYIRS1Xdefdu3fy6hBVuT7kyZNHKHhuGy2Yrt6LgmeI+r1wj7yrNb6Yzlf/HN6lo94Eoqbq8Q4cj13J2Hvr5eWFtrVr19q8M8b68eqnnszu6XaVNm3aCCV27Ng2H0f94iRJkkQokydPRhvWDW/evNnmWPCXFiGEkAgDhxYhhBAOLUIIIYRDixBCCIcWIYQQ8rkTSV3pHNtd1dALxngwBma0ZbZLlCiBNrVUMQyoNaPYrzpt2jS0YWYmV65caPvrr7+EMmPGDLT16NHDYTvV9ekx3eQe7XOBGTY1aYa1m2rPrHuCCxNK27dvx5f07NnT5gy5cuWKw99ygZlJjKipn1qwYMECoWAkzxjz1VdfCeXWrVs2W2W0Wlv1AKVOndphOy9fvowipqTUtt9t27YJpVy5cjbfC1zY3hgzcuTIf/6Nixs3LorPnz+3eS0u664W5iZIkCBcLg7Lly+3OUPUkG24gKlO9dPhdqqvVauK8RRdvXo12mrVqhXajcekpXpxQDBAa/4erv6IGvzmLy1CCCERGA4tQgghHFqEEEIIhxYhhBAOLUIIIYRDixBCCAkn9Mg7IYQQwl9ahBBCCIcWIYQQDi1CCCGEQ4sQQgjh0CKEEPLlElVVcVlltRUUK19btmyJtqFDhwpFXfk7ZcqUQgkODkabWPR69+7dwqCuZY5guarRKoD/+OMPtD1+/Ph/dDyyZ88ulDNnzqBt4sSJQrl27RracE163F0e99ijR49Q3LVrl1DUKt5IkSIJ5cGDB2h7+vSpUGLEiOG8o2zODXvUNdfVruSsWbMKpVOnTqH9c0uWLEFx586dQqlatSra7t69KxS1XDhHjhxCmTNnDtq+//5712Msaa1Xr57Nx6lTpw6Kq1atEopaAlu3bt3Q7j21JRbbjX19fdHm7+/v8TtijEmcODGKAwYMCNv+cScwMBDFgQMH2hxT/HPYOWuM2bp1q1B8fHycv5tYt92kSRN8CV7eAwIC0Na/f3+hFCxY0ObikD9/fv7SIoQQ8kXBoUUIIYRDixBCCOHQIoQQwqFFCCGEfO78o+7BokWLCmX//v1hfrdz584JJXPmzB5fdenSJaE8efIEbXnz5rXZhgwZMghFXRkdd5q6uHicOHFcj9+/fy+eVZcVxw1Qc0THjx8XypYtW9BWoUKF0B6FEydOCEVdKhtXrJ86dSragoKChOLn54e2KVOmCKVdu3ZhOIVwl3p7e6Nt7NixQunduzfa5s6di6Kaj3Xm3r17NmdLvnz5hKKG09auXSsUzJ4ZY7p06SKUAgUKoO3w4cOh/Th46N+9e2fzqQcNGoQ2zG2WKVMmXC5talC2W7duQilUqJDNTjZatk0NjrpnPocPHy6exWSdMWbcuHFCUcPAeATVa12VKlXkVR5yeubvUVvL4Kj6PkjFihWFsn79erRFixaNv7QIIYR84XBoEUII4dAihBBCOLQIIYRwaBFCCCGfO3p6cM+ePUKJFy8e2nLnzi2UZs2aoQ1b/tQUCpaqeXl5heEjHTx4EEWsvMPPaLSaRIziGGM2bNjwvzoesGfUA4QJK+yBNH+PL35EDUOqzWAusETOGFO7dm2htGjRAm24VWoiNFasWEJRK+kGDx4c2v159OhRFDEG1qBBA7SpQcH79+8LpVWrVs7bgC9JkiQJ2rABsnPnzmjDPaP2NOLpXbx4ceftvHnzplDevn2LtkaNGgll3759aMPzFos9jTHVq1cXyrNnz9AWN25c543HOlBM2BotTapeZNTLHX6iU6dOoU3tA3Sxd+9eFDFku3jxYptD36FDB7ThB8cLdXh9cVQwAj169Gibj8NfWoQQQr40OLQIIYRwaBFCCCEcWoQQQji0CCGEEA4tQgghJJyIqqopUqQQSurUqW3eTl3DXu1yRV68eCGUQ4cOoc05n22M+fDhA4rY/6iWlmKcF7PdRlvb22OPKlYJFylSBG3YGoyNwEZLtaoZWfw4Hksqf/rpJ6EULlwYbdu3b7fZA5jgb9OmDdpmzpz5Pzq/sadY3fM//vgj2sqVK4cirs6u3kWQPn161+MZM2aIZ69cuWKzrzZu3Ig2TOcfOHAAbalSpQrtvsLo/Js3b9CGAXf10G/atEkomG43xrx69Uoo48ePR5taCuxOggQJhDJr1iy0YQly5cqVLfcPNgWr7c/OkfdixYrZiGpeH2/5yJEjB9rCEHDftWuXUAIDA9GGNw6pF7Hz588LpXXr1mjDyPv169dtzmT+0iKEEBJh4NAihBDCoUUIIYRwaBFCCOHQIoQQQj539PQg5pHU2lZLcPVltSoRKz4bN24chj+nxgKxjTd27NhoW7JkiVBu3bqFNjVQ5wxWcKr9oZhYO3LkCNqSJ08ulD59+thshpoHc+eXX36xeR+sCn348CHasmXLZmMLLwICAoSirt2OFcBdu3a1OW+NMZUqVRJKmjRpnLeqb9++Qpk9ezbatm7dKpQLFy6gbc2aNUJRE1z+/v5C+fXXX523M3HixELBtJiK2h9dokQJoaiZyZcvXwqlUKFCzn/uP//5D4oYVlSLntu3b+/xK/+pbTh27JhQ6tev77ypmIhWw8B43k6YMAFt0aNHF8qNGzfQdvLkSaGkTZsWbfHjx3c9LlWqlHgWFXXjMe1sjHn37p1Q1L5jDChmz56dv7QIIYR8UXBoEUII4dAihBBCOLQIIYRwaBFCCCEcWoQQQkg4oUfesZTz6tWraPPy8hJKnTp10Pb7778LpXnz5mgLQ8+j0Zol8+TJgzZM5aoVwCEhIULJly8f2tSEtDNYIlmzZk20bdu2TSgVKlRA25w5c4TStGlTtA0dOlQomTNnDpfzxrkV1AVGctXIL4K9tMYi1v/s2TOhqG22eLOB2tSJLaJGS0irnbxRokRxPcbbRbCa1mjp4d27d6Pt559/FkqyZMnQhgH3nTt3os39lgA8WFGjRrU5WPfu3UOxe/fuQlFvFME61IULFzr/ObV9Fe9PWL58OdqmTJkilESJEqFt9OjRKKoH2pnDhw97/LxGK+BWu5KxUrlZs2Zo87gDbZg2bRqKPj4+Nq8NCgoSyqBBg5zPvY/gjTRGu72Hv7QIIYREGDi0CCGEcGgRQgghHFqEEEI4tAghhJDPHT0d5J59+sjbt2/Rhu2rakklZtjUVla1/NEjK1euFEq/fv3QVq9ePaHcvHkTbRgYU/tqV69eLZQuXbo4byd2Vt6+fRttalYQwRpfX19f5b8kkUP9nxLsY8XSW6MF29RSzq+//loo+/fvR9uIESOEoibi3FHzft7e3kJJmjQp2vBglS1bFm25cuWy2WP4ZRGsX79eKGpXMhY9q2uuY7500qRJaMOGaLU72J2UKVMKZfPmzWjDTmc15fjq1SuhYC2s+UR4zxk1e5kxY0ab18aKFUsoai7az8/P5t2wQtcYkzdvXtdjtXYWiRMnjs1Wde7cWSgzZsywuSSqoW5nqlatiiImrs+ePYs2LC9W+38xdNqxY0e0rVixgr+0CCGERFQ4tAghhHBoEUIIIRxahBBCOLQIIYQQDi1CCCEknIhq6cuSJQuKd+7cEYoa8Vy8eLFQtm/fjrYPHz4IRU0GV69e3f2fgwcPFga1kjUwMFAoGI82xixdulQoahgaM9PYYGuM+fbbb12P3YOwH8EKXaMV2mJLptGaTOvXr4823BVY3ipQa3yRkiVLCgXzuOp2qkF/jwF3S54+fSoUDBOrYN3qp8AdqB4gd8qXLy8UjIMbLcQ/efJktGHjqnpPQhgIDg4WyjfffIM2vFfkzJkzaCtQoIBQMPpvjJk7d65QsGRWvJt6+020aNFsPiNWsqpBfIz1q3jc8/jmajT/+++/F0qjRo3QduHCBaGoifPYsWOH9tBjxW3MmDFtXjhy5EgUEyZMKBQ1c//kyROhWFb98pcWIYSQCAOHFiGEEA4tQgghhEOLEEIIhxYhhBDyuRPJY5yMEEII4S8tQgghhEOLEEIIhxYhhBDCoUUIIYRwaBFCCPmysO0eVFf+PnnypFC6du2KtkePHgklHCOLu3btEkqaNGnQduLECaG0aNECbdOnTxdKiRIl0Iar3atrrmPfoDtqY1v27NmFUqRIEbRhbZfa6YcLsTdp0sR5f2ID5LVr19CWLl06oahLj+Pa8Pny5UObWvAYBnDF+lGjRqENt+Gnn35C24EDB1DE9dqXLFmCNvfuuDdv3ohnV69ejS9p2LChUEJCQtBWsGBBoQwZMgRt7du3D+3ew/fZuHEj2rCXb82aNWhLnDixUDp16oS2xo0bC2XcuHFo69mzp/PG4y4dMWIE2qpVqyaUNm3aoA0PmdGK/u7evYu25s2bh3bPDx06VChq1ysuWr9v3z60YalppkyZwuX7Vbp0aY+XX6MVfqollitWrOAvLUIIIV84HFqEEEI4tAghhBAOLUIIIRxahBBCyOeOnh7cvHmzUNQoTrx48Wz+xp9//imUrFmzog0DaZs2bfL45o8fPxYKrptptKU5MYBntAVD8YVGW5U1DMybN8/GhivVGm0x3Fq1aqEtQYIEod2qDh06CAWjkipqAhCDoyqYwlKXr61UqZLz+2CQT03W4d4rU6YM2tTwHi4tXbx4ceetivL46pQAAAgoSURBVB49ulDURatxrdjUqVOjDcNp/v7+aMP0oJovTZ48uetxjx49xLPuS287fE8tUSN5YiFyYxEUVMHzv2PHjmgbM2aMUKpWrYq2/Pnzo4grAntccRvzfu473AUuft2sWTO0YRa3e/fuNhcWTCd6ZPfu3ShiVvCHH35AG2YFL168iLarV68KRV2TGtd65i8tQgghEQYOLUIIIRxahBBCCIcWIYQQDi1CCCHkc0dPDxYuXFgorVu3RlvLli2FgvVcxpgkSZIIxcfHB21JkyYNwwfA9JEKRgqnTp2KtuHDhwtFzSBhjMcjmKOrUaMG2rAl7N27d2i7cOGCUNTgDaby1APknrzC9sjZs2fbfEA8GYwWnFNL1by8vITiMSj4888/o4hZwQULFqAN85BqvMqysQ3bGgVYENeuXTu0pU+fXihbt25FG34TMV5ltC4+NV/qDuZpjxw5EuYrC/aL4slgjAkICBCKGufzuMOfPn0qFCyiNMYsX75cKGoSGAPPRgudekRN9yFYeYofR323mjVrog3De3PnzkVbq1atXI8x3Xf//n18SYoUKTxeYdQjeOnSJbR5e3vzlxYhhJAvHA4tQgghHFqEEEIIhxYhhBAOLUIIIYRDixBCCAkn9Mg7dmuq+VFEDTTjot2vX79GG+aVV61ahbbatWu7/zMwMFAYHjx4gK8qWbKkUGbNmoU27H9US0sPHjwolEKFCjnvGVyCeubMmWj766+/hKJmT+/duycUtZIyDAtaY5cxflij5acfPnyItsWLFwtl0aJFaMOzSw36u/f/xo8f3+bjqKHtokWLCiV79uxou3btGoo7duwQSuXKlZ23IW7cuELBYl+jxYLVUxQ/0W+//Ya2YsWKhfbQ410ZeNIaYxIlSiQUy+rnOnXqoIiRd/Ua4n77QeTIkZ3PjY+o1bQY/lZ3lHoXAcbQY8WKpVxSo0Z12AlY8G2M2bNnj1Cw79hoLbr16tWzOUB49RNglv27775DG5Ziq3ugQYMGQsmQIQParl+/LpRnz57ZzBT+0iKEEBJh4NAihBDCoUUIIYRwaBFCCOHQIoQQQj539KALRsiOHz+ONsx1qLEoXMO+SpUqaEuZMqVQfH19PX4ANfeFBAUFCUVdYf3QoUNCSZw4MdowlecxPRgtWjShqJ2wAwYMEIoaZOrVq5dQME8YNrDVVM0RubdtfuTGjRto27Rpk1Dev3+PNkypTZkyBW2dOnVy2ANGi2apmcY5c+YIRS0F7tKlC4p+fn42f8KdnDlz2py0eIr27t0bbXny5BFKSEiIzZFVI7tp0qRxPcaUo1psjZ9XPZNPnTollO3bt9t8fy17igUYQD137hzaTpw4IRS1MDddunQoYgWwGqx1vxTs379fPHv06FF8CeY2Vfr16+fx1DJazzL2gBtj/P39XY9HjBhhc7CuXr0qFLX0FpPMaiwQG9JxBPCXFiGEkIgNhxYhhBAOLUIIIYRDixBCCIcWIYQQwqFFCCGEhBNRLX27d+9GEUOluXPnRtvhw4dt/sTgwYOFEiVKlDB8JLXbd/369UI5e/Ys2sqXLy8UbNg0xnTt2lUoaqjUmTdv3qCIvZ9qE+7ChQuFgl2Wxpht27Z5/IACzPWqPaeIem8AdpuqQWS8hUBNUXsEk7WYbjdaavzYsWNoc0/YuyhXrpxQ1HCwh/8qapWvWJhbvXp1tGXLlk0omFc2xowZM0YoHm8OwZz30qVL0YbfcbXTGe8iKFiwINowtK1+Md1vwlGvJ5hcx9tXjBZ5f/v2reWhx9s8xo8f77xLixQpIpSECROi7fTp0x6PhTHm7t27Qpk/fz7aWrRoIRTswxVUrFhRKNjha4wpXry4UNQ7WPAuHbzryRizdu1aodStW5e/tAghhHxRcGgRQgjh0CKEEEI4tAghhHBoEUIIIZ87kTAtpoJdjcaYkSNHCkV9N6zfvXDhAtrUIIpHsMPxw4cPaMMMXtu2bdGGIUAMzBgt2+NxNwYGBgrFsuo3UqRINjY1c4WNxhs3bkSb84LxV65cQRGjnmpOD7E838LGsmXLhLJ582a07dy502bvqXt++vTpQsmfPz/aVNHFggULUJw5c6ZQduzYgTZc4PzFixc2O+fy5csopk+f3vUYK27VPlbcLWr39KtXr4SSNm1atGHFttq87F6lquZp48eP7/FkMMaUKlVKKFOnTkUbFj2re16tEXdGDcitXLlSKGpoGUOnmTNnRhtGFj3Srl07odSqVQttmHJMlSoV2rBHeOLEieF4KeAvLUIIIREGDi1CCCEcWoQQQgiHFiGEEA4tQgghhEOLEEIICSf0wlxMfKrdnbFjxxaKmhIeNGiQUIYMGWKzcfv370dRBDq9vb1t3gpD6jdv3kQbNsw2bdoUbdi4qubC3TO+GHDv27cvviQgIMDm40yePDlsx9s53a6ibhIG3L28vNCGRbQvX75EG4aJBwwYgLZhw4Y5byoGcKdNm4a2oUOH2py3NWrUQBGjwLdv33beKsxwP336FG0YcMf7CswnYug2JEqUyNlg+c6PHz8WCrY8G+0mmaRJk9q8v3u6XQX7WI0xwcHBQvH19UUb3n5z5MgRtKmbqmbxncFUN3ZDG62TevTo0Wjz8/MTyvnz5z1eIc0nuoPd+4szZcrk8KwLLKRu0qQJ2oKCgjyeM0a78enWrVtoK1myJH9pEUIIiahwaBFCCOHQIoQQQji0CCGEcGgRQgghnzuR/qcFpoQQQgh/aRFCCOHQIoQQQji0CCGEEA4tQgghHFqEEEIIhxYhhBASZv4P9LO0hCu68hwAAAAASUVORK5CYII=" />

It’s clear by eye that the images are noisy, and contain spurious pixels. Let’s train a PCA on the noisy data, requesting that the projection preserve 50% of the variance:

In [ ]:
pca = PCA(0.50).fit(noisy)
pca.n_components_

Output:<br>`
12
    `

Here 50% of the variance amounts to 12 principal components. Now we compute these components, and then use the inverse of the transform to reconstruct the filtered digits (Figure 5-90):

In [ ]:
%matplotlib notebook
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

> **Figure 5-90: _Digits “denoised” using PCA_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAjwAAADsCAIAAAD/zOvLAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2dedSN5f7G74bjOJnnKSFTJEOZZcyQKVNKouOkQdJc/EpRFEdIOqc6UupQVITqVEodQ5KZaECZhSgzSUf9/mgta/teX3vf79vbWl7r8/mrc51rv/vZz76f52uvdT3Xfcavv/4aAAAAMgNncgoAAIChBQAAwNACAACGFgAAAEMLAAAgYzjbVY8dO2aUw4cPq61nz55GmTx5stpat25tlIEDB6qtYsWKRjnnnHPUdsYZZyT+T00/GsNv/Pjjj0aZNWuW2h5++GGj3HjjjWrr0aOHUf70pz8lP9E///xzWl/yG3feeaeKo0ePNsqwYcPU1rdv39+/RPTUuaflgw8+UFvdunWNMnjwYLVddNFFaT2q//3vfzEn+ZtvvlHbmjVrjDJ16lS1TZw4MUOusV9++cUohw4dUtu1115rlLffflttI0eONMrdd98dcwkfOXJEbdmyZUvykrPOOktforcCd4nq2z333HNqy5o1a8ov0Vwsej7dq37mzJlq0xvUlClT1JYnT56YM9+2bVvnlnr22UkWw/79+1W8/fbbjfLvf/9bbc8++6xRevXqlb4VGEI488wzk1xN7l109uzZRunfv7/aihYtGnN9xSxadxHySwsAADINDC0AAGBoAQAAMLQAAIChBQAAcKrjB100OrJ27Vq1aRTnqquuUluxYsWM8uqrr6rtvvvuswfn5XD+/Oc/J/5Pzb189913+iq19evXT22ff/65UXr37q02N42TnMisoJ6Zd999N+aF7se5+uqrjVKiRInkf0fTTXPmzFHb+vXrjdKuXTu1vfbaa0Zx40bK0aNHVcySJcvx/3YDeBo3dXOJkyZNMoobMozEbe9MvIISY1q/sW/fPn3JvHnzjJI7d261Pfroo0YpV66c2jS36R5nYnrQzQoq//rXv4wyduxYtT322GNG2bJli9qKFy9uFM0TGtw464YNG4yyaNEitVWpUsUo2bNnj7kM3feNPGMpF/aMGTOM0rhxY7Wdd955RhkxYoTabrjhhpiFlJw9e/aoeNtttxmldOnSatNrc8yYMWq7+eabU84dfmkBAEDmhqEFAAAMLQAAAIYWAAAwtAAAAE51/PSg5p3cvJxmBZ9++mm1aZjkySefVJvmfwoXLpzyA2jgZPPmzWqbNm2aUTQoGLy8osaNgiQYQ0RhmkaP3DpHPVeaAAxeashtlkuZFVQ0gJc/f361aajS7R48cOCAUapWrRpzGIlBQZdcuXKle9FrWKt8+fJqc0/p999/bxQNx6akYMGCKlarVs0obr+intL58+errVKlSkYpWbJkWo/Tzfvdc889RmnevLnatBZv48aNatu9e3fMVZ94R3LXhqbyGjVqFLO8d+7cqbayZcuqWKpUKaP89NNPakuefnSvJj2BFSpUUNv27dtjLrpWrVoZJUeOHGpLjD7qXfTLL7/Ul3z11VdGad++vdr01uoWPOqdLTLlyC8tAADINDC0AACAoQUAAMDQAgAAhhYAAABDCwAAIIM4O9Lnhju1jfQvf/mL2jTl6aZCV6xYYZQ6deqkPDDNJWuqNYSwePHimI+pn8gNQysp+3A1E58vXz61XXLJJUZxs9362pYtW6pNWy/dfcRPWBAS+r/gggvUtmnTJqNMmDBBbfpFu7lnN2GcDpYsWWKUAQMGqK1mzZpG6dq1q9rGjx+vonak9uzZM0MOXjPi7gGsXr3aKLoJevCeSXB7ZhMvWE3Y63sFLz/dp08ftWkpsO7C7q4Qfd7G4Naqau/tf/7zH7XpudLq55OteV26MTeoGJo2bWqU0aNHx9xkrr32WrXlzJnTKO6zE4krWVe125CuTJ06VcU1a9bEvFYbotu0acMvLQAAOK1gaAEAAEMLAACAoQUAAAwtAAAAhhYAAEAG4UfetdN927ZtatNEdbZs2WLe1bVpwXkMGnXVMubg9Sjv2LFDbb/++qtR3AC94la2J75Wc6h79+7Vl2g03w2VKs2aNVNRU87aT5/yM7op+Vq1ahnlnXfeUZumkx966CG1/fWvfzXK5ZdfHvPXDBMnTjTKe++9p7bbb789ZjHozgAhhI4dO6Z1ierjIu4Hadu2rVE6d+6stsGDBxtlyJAhatPqd92Twd4L5GkH96rXRXXdddfFXM5vvPGG2ho0aGAUt1w/MRmvx+muUvfvaFG6u9K0KD14z8OkRO8nupdFCGH69OlGWbZsmdoKFSpkFPeBHH0mwb3qk+M+lVS9enWjuFX6devWNcqLL76oNn1Ahcg7AACcbjC0AACAoQUAAMDQAgAAhhYAAMCpjp8e1M7KIkWKqG358uVG0VrY4PU8uiWVmpXav3+/2kwM79ixY8bgphA1IBfZ6qifMYRQrVq1lH/fcPToUaNs3rxZbZoN27Jli9patGhhlAIFCqhNU2op20gV9zv97rvvjFK6dGm1abrJjeSNHTvWKG7/byIHDhxQ8bzzzov5RMWKFTPK0qVL1eY2zJYpUybGllhEq9ktLZN1vyw3waWVyhpRC+nK4mqt6vnnnx9ji3yvuXPnqnjxxRcnOXXxC1IvwMaNG8dczm7u94orrsiQ26vendwv6+OPPzaKWz+ttwK3UrlkyZIxpzRLlizH/1vj4lWrVtWX3HfffUapV6+e2jSi6V717pnnlxYAAJxWMLQAAIChBQAAwNACAACGFgAAAEMLAAAggzg70ucGmmfNmmWUYcOGqW3nzp1G2bRpU8xbaMmsctZZZxlFK3RDCDVr1jRKjRo1Yg7VLbjMnTu3UQoWLKi2xF7gxIDpb1SsWFFf0qlTJ6M8+eSTatPXrl27Vm16AvPmzZv8fGq76/fff6+2xx9/3ChuT7H2abrVmd26dTNKymcntBU0hNC+fXujuAljrY7t2rWr2vTBhuAVvOoKNOjTDu4DEjfffLNR3DrU//u//4u5YPPnz28UjV+bg9fMff369fUlWjf82muvqU37anVphRBWrVpllEsvvVRtiZePPkITvPJuTX4H74mFG2644Q/8TSBPmOitw73E3FJgjby/+eabarvpppuMUq5cubQep3t32rhxY8xtZ+bMmUbRh0xCRIMzv7QAACDTw9ACAACGFgAAAEMLAAAYWgAAAKc6fnpQg0YaRgreFu/uZuraXDl8+HC1pYy4ROK2Q+pW2Y888ojaevfubZS3335bbfny5Ys5PylOvRcQGjVqlFHcqmLNXGmNafDKVVNuvK1/x83p6Qp555131PbEE08YpWzZsmrT4FzKSJ6LhpQ07Be8yNPChQvV5u4in44D0+Com43U0tLIoOAzzzyjYp06dYzyww8/qC0x9Rr50UaPHm0UN2GrS9RN2OqCdFOOKUlM6ia5crVX2s0iZthvAknl6a0jeAnGa665Jubv/+1vf1NRL7EdO3aorXDhwmk68uBlj90bvq6HQYMGqa169er80gIAgNMchhYAADC0AAAAGFoAAMDQAgAAYGgBAABkEGe4jaIAAAD80gIAAGBoAQAAQwsAAIChBQAAkGb87kGtQXOrqHTDTd3VNISwbNkyozRs2FBtnTt3Nor2pwUpGdMaN7fTb9++fUbZtWuX2ipXrmyUqlWrqu3ll182iruzc3IOHz6sonZ5vfHGG2p78MEHjeJuwKpbnRYqVEhtieV4Gsxxj1N3Lp44caLa9u7daxQtgQwhDB061Ci603Q4cWtddxvclM2Kv/Hee+8ZRTsSQwgjR46MWSEp0avp4MGDamvbtq1RVq5cqTZt8HP3Qb711luN0rFjR7Wl3Mk6hn79+qmoPZDuzeHQoUPJL3Dlxx9/TL6GdbUcp0WLFkb58MMP1eauhz59+qT8Zs0i1CZP3cY6hNC/f3+j7N+/X22TJk0yihujmzx5slHatGmT1u/UbYDUu9O8efPUpg2uXbp0SfeU4ZcWAABkYhhaAADA0AIAAGBoAQAAQwsAAOBU53elBx944AGjaL4lhHD//fcbZcmSJWrTSFXMHqaaz3FflTt3bqM89dRTajty5IhRNP8WQti0aZNR0pEenDNnjopz585N+V4hhPnz5xvFTQ/mzJkz5uO4W8oeZ/v27SouWLDAKPXr14+xLVq0SG1bt241Sq1atZKfvcig4PLly1UcMmSIUfLkyaO2d999V0Xd7zX52XOXqJu50v2U3S9L93vVVFvwtiHOmjVr8uPUIK67q++2bduMolHSEMKwYcNiviDNCrrRyuzZsx//b3drckUzbCGE9evXG+Xcc89Vm7u1eocOHYxSsmTJ5Meg9yI3EapBx2rVqqltxYoVRtFgdgjhm2++Seu9SDOc7oWjC7Jo0aIxc8FND+pM+fbbb9WmMVR+aQEAQKaBoQUAAAwtAAAAhhYAADC0AAAATnX89KBWWq1Zs0ZtY8eONYo2ngUvrOUGe7TjTsNvSmSUSJNRo0aNinnhZZddpmKlSpWM4vahJT+2AwcOqKhJMzcb5kaelFy5chklZSBTg0xlypRRm8ZEV69erbYdO3YYxe1ybNmyZYasZj17AwcOdBa9tFM2aNBAbW7QUZ0aTA0nVuFpLZ77LWi6zz0tTZo0MUq7du1iToXbyZkOtNrOzV66ubIYIq9og5b1udfXuHHjjDJmzBi1vfXWWxlyYJpxda/cu+66yyjTp09XW4ECBYyiUdIQQr169dJ6d9KYqNsA2bhxY6OMGDEi5qrfsGGD2kqVKmUUDQrySwsAADI3DC0AAGBoAQAAMLQAAIChBQAAwNACAADIIPwUrFYZuoFmpWHDhjE27WcMXlltDBpszZEjh9qefvppo7htpBredfd913RyyjisdhC7G88PHjzYKO7+3KtWrTKKPjAQvI3Y3f25E9Ft7N1qWt2mXfewDyHMnj3bKKNHj445TneFJMZw3Q8ya9Yso8yYMUNtWuip9bLhJEHhnTt3GkUT7SlxU/IahnZPqfbV3nvvvc6FLQF3feojnJh1Trk2TrbS3AcGtMv1yiuvjLmU3O8iJdoS67b96sMGbse324Srda5uV3LiQyO6jN3Iu17OI0eOVNsFF1xglFtuuSXm5KS8O+karly5crqv0xIlShhlwoQJarvnnnuSXOD80gIAgNMBhhYAADC0AAAAGFoAAMDQAgAAYGgBAABkEH7kXfPZLrVq1TJK586dY16oNcAhhM8//9wo2lQdJMurAffdu3frqzTqWqVKFbXt27fPKG6AXuvn3d7uxPCutqdXqFBBX6Lif//7X7VpCXTbtm3VdvHFFxslZazZDbjH4LZ6a2r80UcfVVuHDh2Mkj9//uRvp+czhLBw4UKjuGHfFStWGGX9+vVqc9PJ5513XlrPjF5N9evXV5tWfevjB8HLdn/44Ydqa9q0aVqPMzK7v2vXLqPMmTNHbW+++aZRsmfPrrbrrrvOKMWLF0/Hsty0aZNRNEceTrJpg7Jx40YV9Zkc9z6ZeNVHZrj18ZtWrVrFHHynTp3+oMHgPnhQvnz5lDft4D2G1KNHD7VNmzbNKN26deOXFgAAnFYwtAAAgKEFAADA0AIAAIYWAADAqY6fHtSuyUKFCqltzZo1Rlm3bp3aSpcubRQ3mqJJG7ex1MT5NA6nvZbuJ+rYsaPaBg4caJRx48apTdNuKVs+3bRbDK+++mrMuXL/vgao3Jbe5LiBw82bNxulZ8+eatNIYfv27dX2wQcfGKVr167Jj8qtf61YsaJRqlevrjYNVWrULZykUvnCCy9M8z8Mz4z6p6EuyJUrV6rt2WefTfldRF7UKS83t6z5iy++iDlRzz33XMrTHkIYMGCAUSZOnJj8ON1Qpa4096vX7KV78G3atFGxUqVK6ftmE/nhhx9UXLt2rVGqVq2qtueffz5mNWqv7h+He3PQxHWLFi3UNn78eKOQHgQAgNMNhhYAADC0AAAAGFoAAMDQAgAAYGgBAABkEGdH+sqVK6diwYIFjdK9e3e1XX755SlfGEJo3LixUVLmyIMX9XZ7Tps0aWIUN6w5atQooxQoUCBDTrS2bbodoO+8845RNBgaQihcuLBR3GT8li1bjDJ8+HC1FStW7Ph/a+7Z7Zx96aWXYk7URx99FHNytO3Ufdoh8UjcAHe7du2MUqRIEbVpEbM+6hBCqF27torpKBTWtzOlz7+xdOlSo7z22mtq27lzp1FKlCgRcxiHDx9W8Zxzzkl+RSgtW7Y0ij6xELzEubsYtPrZfWQlcYm6X4GWd7vJb+32LVmypNrcJy60JjsdHDp0SMV58+YZpWbNmmrTNe9+p+lAnyJwO83HjBkT89dq1KhhlKFDh8ZcSnv27FFbnjx5+KUFAACZFYYWAAAwtAAAABhaAADA0AIAADjV8dODGkrJmzev2jSK49ahaizqjjvuUJsG/NJHYiDqOLly5TLKvn37Yv6aG3RUjh07pmJi+lENbvJHO1LdelDNTC5evFhtWbNmNcrWrVvVlhjN0qygG+TTENSIESPUpsG5OnXqqE03F3dDVm6O8WQnPMnbaSWrVnyGEK644oqYr14/YzgxH6hZwf379+tLbr31VqOsXr1abaNHjzZK5BbybmQx+RedL18+td15551G0UBjCKFevXoxC6lZs2Ypv8T04YYMNVLo3nauueaaP+iGq1XXwWvRXbBggdqqVKmS5MpNwi+//OL8Xklo+9XLTW+Y7q3VDY66WUGlV69eMdcF6UEAAMjEMLQAAIChBQAAwNACAACGFgAAAEMLAAAggzgjsiUTAACAX1oAAAAMLQAAYGgBAAAwtAAAABhaAABwuuB3aP78889GcXc31wZMd4vrCy+80ChTpkxRm7YxptxzPXhFtJGFmxs3blSxVKlSRtEd3IO3R3g6cNshe/fubZRXXnlFbd26dTPK888/r7Z07A2vX/2RI0fUNmjQIKNo1W8IYdGiRUZxm4VfeOGFmNOeJUuW5H8n3V2rlSpVUrFRo0Yq3nvvvUZx92tPRBt1Dx48qLZbbrnFKNu3b1db0aJFjdKlSxe1nX/++UYpU6aM2hIrlTVIrL3MLtdff72Kb7/9tlG06jecZGP75Bw9ejT52kjCuHHjjPLII4/E3AdCCHfffbdR3KpitxL3OG4T9IABA1JeEa7YqVMntemm9e59wC0WT877779vlCFDhqjtyiuvNMptt92mtr179ya/t5/s4PmlBQAAmQaGFgAAMLQAAAAYWgAAwNACAAA41fHTg25WUKlcubJRNm/e7AzGM+1ozJ49u9q++eYbo8RsJq1pt/Xr16tNIz1fffWVczpkS/Jly5b9Qaf+ww8/VFGzgm44TaOPI0eOVNsDDzyQ1qPSr/7rr79W28cff2yU+vXrq00P/rnnnlOb7ivfuXPn5MfpZhqTB7eOoyksN3uWM2dOFXWFp0wP6mbnn332mdo0V+b2gurbtW7dWm07duwwipu3TCQyK6iBsRdffFFtDRs2jFlI6SDyOGfPnq3i4sWLjXLttdeqzb2avvzyS6O0atUqrQfvLtENGzYYxY2zamxy+PDhaqtatWrK7yLl1bRv3z61TZw4Meb2qGvvpptuUlvu3Lljxsd5553HLy0AAMisMLQAAIChBQAAwNACAACGFgAAwKmOnx7UMMmWLVvUpnm/q666Sm3a2VW7du2Yv6b9aYpGszS7Erz+t5kzZ8bYqlSpEnMe3aBXYsxJj9MtyuvevbtRHn744ZhTqinKjMJNx918881GqVevnto++eQToyxZskRtmm5yP05isjEyKOjSp08fo7Rp0yb52yU5MK1QCycmozSaqMvMveiqV6+utvbt28csJE1/FSlSJPlp0aPSvFwIoX///kYpUaKE2gYOHGiUmDBwDJHZZjdFXK5cuZgLfOjQoSrq9+iuUk0gp0TL+pYuXaq2nTt3GsXN4j7++OMxx5n4cTTgrYHGEMLChQuNkiNHDrXpStPQZgihVq1aRtGgIL+0AAAgc8PQAgAAhhYAAABDCwAAGFoAAAAMLQAAgAzCT2cmbsL9G7t371abNjOOGjVKbZpBd7d/1kBn+tCDDyH069fPKLNmzYr5a40bN1bxp59+ivlEyWnSpImKus28duMGL0DfsWPHmDfVF4YTA69ayuluy123bl2jaNVvCOHTTz81irtDeYUKFYzihsITs8779+9Xgx68u8+3RsndRtG5c+eqWK1aNaNo72dKNm3apKLGgt2HDXSludXP27ZtM0rFihWTH9WPP/6Y8nyGEPLmzZty0YYQChQoYBSNm2cgeveYMmVKzMFPnjw58i20cdjdHj4dNGjQwCgTJkxQm/bVuitEP2PKimFN87ud5jVr1jTKggUL1KZ5fX3+IYTw5ptvJrnA+aUFAACnAwwtAABgaAEAADC0AACAoQUAAHCq46cHdedvN9pXo0YNoxQtWlRtKl5//fVq++KLL4ziRshMH6Vm2/Lnz6+v0rSbdrkGLy61detWtWmCK2XBq1ZSul2TmuAaM2aM2jTMWbx4cbVpytEtV008NnfjeaV06dJGyZUrl9pWrlxplHPPPVdtmjRLSc6cOVV8/fXXjXLw4EG1aT2u20/qZvw0r5WyK1lxT3LZsmWN4jbM6qb17qlw10NydEG63a+aK5s2bZraFi1aZJTx48errVSpUjFfWcqIpn6Dhw8fVptGi/fs2RN5fiLrXBPRsK77dmPHjjXKjBkz1KZhxVtuuUVt5cuXN4obAE6+hitVqqQ29xtUNPp43XXXqU3bfrUHnF9aAACQuWFoAQAAQwsAAIChBQAADC0AAACGFgAAQAbhR96zZctmFDfQrElNN6Tbu3dvo2j1ZAihRIkSRtm7d6/aTKI9suWzfv36RtmyZYvaVq1aZRRt+w0hrFu3zigaAU/5kd1UtNbOvvzyy2rr0aOHUdzGXj1495mE5LjnU0Pbblz1/fffN8qkSZPUNm7cuAw5Kj2lmiMPIdSpU8cozz77rNo2b96sosaOU7aRKto0HULo2bOnUdwsuz4C0alTJ7U1b97cKCm7khV9ZOJkp0XRBue+ffuqrWvXrkZp3bp1Om5khQoVMkr79u3Vpi3VWqUdTvKwjd5DUqLn3F23etsZPHiw2ubPnx/z9RUuXDjmfpv49Es61nCS22P37t2NMnv2bLVpMp7IOwAAnG4wtAAAgKEFAADA0AIAAIYWAADAqc7Zkb5atWrF2B5++GEVddtytzWyS5cuRnHTUwZNc7m9lloi6e5UrTtAu3uZux2pacXt2NVsmNvYe+DAAaOMHDlSbZrxu/XWW9WWWOyrH81td50+fbpR3N3cP/vsM6O4NcGaUnPfNDHj5BoaNmxolM6dO8d8F24fa4UKFVTUOuaU0SzFzXBq0uz7779X26effmqU/v37R/379MwU/0LVYJs2Ygev3XjXrl1q06ibZuRCCJdccolRmjZtqraUla+VK1dOecmHEPbt22cUjR2GkxS8piN5q43D7rrVm4wWfIcQ8ubNa5Tt27erTRN9midMeQG6WVMtQR42bJjaNKK5bNmymJOjPeDup+aXFgAAZBoYWgAAwNACAABgaAEAAEMLAACAoQUAAJBBxEbe3T5WDUpmzZpVbYcPHzbKQw89pLZq1aoZxY2HpkRD8C5uInbQoEFGcQP0+fLl+/2n3g2VauPwkiVL1KZh6MTY+nHy5MmT8oXhxH5krc50W1M/+ugjo9x2221qK1WqlFHuueeemNXlpqg1bG0oWLBg+r4LjeaHdDXMRuJeJjNnzjTK6tWr1aYNs40aNUr3ekv8OBrTdz/szp07jeJW0+rTI7Vr11abHrwu2qgbmeSnVQleM7J7Lbdo0eIPuuG6j/Hogxn33Xef2po1a2YUN8t+8OBBo7iPzZx77rnH/1trx93j1Np09y/ff//9Sd7rOH369DFK5KNE/NICAIBMA0MLAAAYWgAAAAwtAABgaAEAAJzqnJEh3a8AAAD80gIAAGBoAQAAQwsAAIChBQAAwNACAIDMg989qFVU//vf/9Smey2/9NJLasuePbtRnn/+ebVdeumlRknHRuYn4/PPPzeKu6O21mFdf/31atPdo93N5hMr5n7++Wfz/x46dEhfoi1kX3/9dfK//Bvu3vDdunUziruXeWKlmDYNusfZunVro+hW3MHbBL1Xr15qu+qqq1J+wBDdKpmSBx54wCjLly9X25NPPqmiW1mZHL12tB0uhNCgQQOjbNq0SW3nn39+zFd/zTXXGMWtKMyRI8fx/z5y5EjMtzB16lSj3HXXXWrTzekfffRRtV188cX2luR1BiYep4sWnPbt21dtkyZNMsrf//53td14440x36wbvU5s71SDe08bMmSIUZYuXao2bad0v6CJEyca5fLLL0/rotXzGULo0KFDykNy7+R6gYcQrr76aqO4tZO6HvilBQAAmQaGFgAAMLQAAAAYWgAAwNACAAA41fHTgxrT2rFjh9oee+wxo2hmKXjJuieeeEJtNWrUMIq7XbJBg45uxmzhwoVGcQNjemC6n3LwEozffvut2kqXLn38vzVCNmfOHH3JV199ZZSSJUuqbf369UbROF/wdvJ1w0vJz/n48ePVtmDBAqNceOGFaluxYoVR3D2Cu3TpYv8xlWqDYI26BS9M5e6sOmPGjJiz524GnQ40OOpmww4cOGAU3cY6eGHO6tWrx1zCKU+pG0VT7rjjjpiTrOtWN7QNITzzzDNG0cSpwc3sffzxx0Z56623Ym4y7l7ekezfv1/FxCyuZvC2bNmiL9m+fbtRqlatqrY6deoYZejQoWpLxyfSJaoZ0RDCBx98YBTNJwcvqq33q+BlBfVmHrzgKL+0AAAg08DQAgAAhhYAAABDCwAAGFoAAACnOmdH+nbt2hVjGz16tIoFChQwSv369dU2e/Zso7Ro0SLlO0ZGvCpVqmSUNm3aqK148eJGcTN+zZs3N0piUNBF01xu/k1TQ+65Wr16tVEuu+wytWkeLG/evGldIhpodOnZs6eKI0eONIpW0oUTA1eRxCRLw0mSVPYhYXIAAAZISURBVIUKFTKKm1grVapUhlxj+tXnzp07ZlG5rY/6WrfYTbOC2bJlS+uRb968WUXNCro1nqNGjTJKvXr11KaxWzcMmY77QJkyZdTWsGFDo3z33XdqS1kqGLluzznnHKO49XrNmjUzihsy1ECm2+SpOV4NB6a8c7p5P0X7A0MIR48eNYqbg9UlmrJhkl9aAACQyWBoAQAAQwsAAIChBQAADC0AAACGFgAAQAYRG3n/5JNPYmxur67mPjU8GkKYN2+eUWIi79pd61KrVi2j3H777WqbMGGCUbTQM4TQpEkTo0RmZBNxI6eaS541a5ba9u7da5TXX39dbZ06dfr9x1m2bFkVGzdubBS3CVf/shu81tixptINGqsNXg5+2rRpasuZM6dR3KbO5Kfl9+B2OuvTCG7lq+5l7l50JUqU+P1f/ZdffqmiPkfh1mTr2+nzJOEknbPJcY9ZP2+DBg3U9v777xvFjfW3bNlSRb0daQt2OHF7eF2luvaC91zNU089pba7777bKD169FCbHlXK54LUcMEFF6hNo/nuM056ORcpUkRtGuvXL5FfWgAAkLlhaAEAAEMLAACAoQUAAAwtAACAUx0/PajJn0WLFsX8uZ9++knFLFmyGOWLL75QW/qKMvVQI0NfmoQJIWTPnt0okydPVtukSZOMknKPcMWtuNXQzrFjx2IOYP78+Wpzd5FPzi+//GIUt4+1e/fuRlm3bp3atEe4du3aatMWUfdTJyZF3cJc3cbe3Y9c+0OXL1+uNm1wDiHccMMNRtEd3MOJcSw9pW69sq5/N0ylC143aw/RQaxEIveG1yWqLbTBS0i6ramaNNPTFbxy1ZSn9K677opZaXophRAefPBBFQsWLGiUatWqJT8qXaW7d+9Wmy4/N++n5/miiy5SmxusTetd1L3qXVHZtm1byqvG/dSkBwEA4HSDoQUAAAwtAAAAhhYAADC0AAAAGFoAAAAZhB9519R4o0aN1PbGG28YxY2tawLSzX1qC61bpmkaJzUL6xaDLly40Chz585VW9++fY3i5p7vvffemDdNHr7PlStXjLh27Vq1rV692ihVq1ZVW6VKldL8rxg5n/ny5Yt54cqVK1V84YUXjOI+G6APRaTk0KFDKu7bt88omm4PXmh77NixavvnP/+p4ksvvRRz5hMjy5FLVM/M+vXr1abNyG5ta9u2bdN6SrXY2o0gr1ixwihuV7J2OrtPZfTq1Svmqs+dO3daP457fXXp0sUo7dq1U1u/fv1UHD58uFEmTpyY1qPS5wqCl/t3M/d6SletWqU2zcG7fdCJzyRkbDd00aJFjeL2X2/dupVfWgAAcJrD0AIAAIYWAAAAQwsAABhaAAAApzpnR/o6duyo4kMPPWQUd+dvTey4tGnTJh0fQNshNUIWvJ3L3SrVunXrGmXx4sVq09yLm4Rx9xc/jhshU9wMm+4iryksl5QbhEeiocqnn35abZs2bTLKnj17MmbhesesacmmTZuqTbc8d7+LDRs2qOh23f7+RRtCeOWVV4ziZsOmTp1qlAkTJsS8aTqKaN0mXMWtQ9VSbLfdtVatWmk9e24WTtGYpfvaxBbm5K/V0KkbYc2WLVuSozpy5EjMVen2Qeu6jUTLi1PifrQXX3zRKG5DusbFNXEavIBrZAabX1oAAJBpYGgBAABDCwAAgKEFAAAMLQAAAIYWAABABhGbdc6ePbuKnTp1Mkr58uXVVqxYMaNEpr1j0O5dNxiqifDp06erbcCAAUZxc8+a03Ujqsk/8oEDB9TWqlUroyxatEht//jHP4zSvHnzqH+knJnmf6a4CeM77rjDKG5h7v3332+UHj16xLypdoOGE1tTjx07poaCBQsaZebMmWp7/PHHjdKgQYPIszFw4ECjaM9sStyD1zzx5s2b1TZo0CCjdOvW7Q/66t1VPWXKFKNceeWVatOHDdzvIh242X3tXF63bp3aRowYEbO8XTp06GAUzXaHEMqWLZvkjxQuXFjFevXqGUULvoP3pEQ6GrFdtFfafaRk8uTJRnFrx2vXrm2UPn36xHxq91EQXYT80gIAgEwDQwsAABhaAAAADC0AAGBoAQAAnOqckYFBPgAAAH5pAQAAMLQAAIChBQAAwNACAACGFgAAAEMLAAAgA/l/kUDy6c3otekAAAAASUVORK5CYII=" />

This signal preserving/noise filtering property makes PCA a very useful feature selection routine - for example, rather than training a classifier on very high-dimensional data, you might instead train the classifier on the lower-dimensional representation, which will automatically serve to filter out random noise in the inputs.

# Example: Eigenfaces

Earlier we explored an example of using a PCA projection as a feature selector for facial recognition with a support vector machine (“In-Depth: Support Vector Machines” on page 405). Here we will take a look back and explore a bit more of what went into that. Recall that we were using the Labeled Faces in the Wild dataset made available through Scikit-Learn:

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Output:<br>`
['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush' 'Gerhard Schroeder' 'Hugo Chavez' 'Junichiro Koizumi' 'Tony Blair']
(1348, 62, 47)
    `

Let’s take a look at the principal axes that span this dataset. Because this is a large dataset, we will use RandomizedPCA - it contains a randomized method to approximate the first N principal components much more quickly than the standard PCA estimator, and thus is very useful for high-dimensional data (here, a dimensionality of nearly 3,000). We will take a look at the first 150 components:

In [ ]:
from sklearn.decomposition import RandomizedPCA
pca = RandomizedPCA(150)
pca.fit(faces.data)

Output:<br>`
RandomizedPCA(copy=True, iterated_power=3, n_components=150,
              random_state=None, whiten=False)
    `

In this case, it can be interesting to visualize the images associated with the first several principal components (these components are technically known as “eigenvectors,” so these types of images are often called “eigenfaces”). As you can see in Figure 5-91, they are as creepy as they sound:

In [ ]:
%matplotlib notebook
fig, axes = plt.subplots(3, 8, figsize=(9, 4),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap='bone')

> **Figure 5-91: _A visualization of eigenfaces learned from the LFW dataset_**<br><img width="500" src="data:image/png;base64, ..." />

The results are very interesting, and give us insight into how the images vary: for example, the first few eigenfaces (from the top left) seem to be associated with the angle of lighting on the face, and later principal vectors seem to be picking out certain features, such as eyes, noses, and lips. Let’s take a look at the cumulative variance of these components to see how much of the data information the projection is preserving (Figure 5-92):

In [ ]:
%matplotlib notebook
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

> **Figure 5-92: _Cumulative explained variance for the LFW data_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfkAAAFkCAIAAACsLNUuAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3deVgUV9o28FO9r9DsCCKgAipRFNcsxBWD0ahEUVzQGU3eiaOOYzBqxqiYSNBkzPgZY95oEhO3oFFxYTSOiJpXjRFJWsUFFxgFUXbohequ7q76/ijTw6gQY6Dphvt35cpVFN3F06fg7uPpU6cojuMIAAC0agI0AQAAsh4AAJD1AACArAcAAGQ9AAAg6wEAwHmy/sKFC0lJSfX3ZGdnjx8/PjEx8dtvv0VDAwC0IFGTHOXzzz/fv3+/Uqm077FaratWrdq7d69UKp00adLQoUM9PT3R3AAALtyvDw4O/uSTT+rvuXXrVnBwsEqlEovFvXv3zsnJQVsDALh21sfGxgqFwvp7DAaDWq3mt5VKpV6vR1sDALh21j9KpVIZDAZ+22g0urm5NfRILNIAANDcRE14rPqp3alTp9u3b+t0OplMlpOTM3PmzIaeRVFUebkL9Pp9fNSos63VicZEnU5eZ8tkPUVRhJDMzEyaphMSEt5+++0ZM2ZwHJeQkODr64v3VQCAlkI5wxAK3upRJ4pEnaizWfv1uJYKAKD1Q9YDACDrAQAAWQ8AAMh6AABA1gMAALIeAACQ9QAAgKwHAABkPQAAsh4AAJD1AACArAcAAGcgQhMAAPxOHMdZbSxttpkY64P/MzYTY+3Yzs3XQ4GsBwBwLlYbW2swl9XQJrPVxNTLbrPNxFhNjI02W02MjWasJsbGP4bfY2Mfsz58txCPBYm9kPUAAM2I5TgzY6PN1jqTtc5srTNb+W3abK2rt1F/p8XKPuHBBRQlkwjlUqFGJZVJhTKJSC4RyiSiB9tSoUwi6hbs4SRNgawHAJfBcRxjYY0mi4G2GE1WI21pKLIfbDPWJ7kbk0hIKaQiuVTk6SaVS0XuapmAcHKJSCYVyiUimUQok4pkEuFDeyQiAX8zPpeArAeAlu9902arkY9vk8VI8/9/kOZGk9VgstQ92LZYbb8e3nKp8JfgViqkIrlMxEe5QiaSS0UKqci+UyEVKWQisUhY/+mucl8qZD0AOAWrjdXXWQy0xVDH6GkLyS+/V24w1FkMJkv9TK8zWRvPb4oiSplYKRN5ucuUMrFSLuK/VMrECplIYY9yqUghE8kkIoGAQuMj6wGgCXridSarvo4x0BZDnUVPW/5722KgGQNtoc22xkdOlHKxRiUN9FYq7AkuF6tkIqVcXC/TxTKpUEAhvpH1ANCkzIxNV8fojMyD/xsZXZ1FZ2T0dUytkTHQFgNtaWQcXCig1Aqxl5tcrRCrFWKVXKySi9UKSaCfG2u18l8q5WLXGu9G1gOAK+E4zmiy6usYnZGpNTL6OkutkdH/J9MZndFitjTYH1fKRGqFxM9ToZbzOS5RycW/ZLpEpRCr5WKZRPjYEG+V4+DIegBoGVYby+d4jcFca2BqDOYaA1NrMNcYmVqDWV9neexMcHt/3M9D7qaUuCklbgqJm1KiVojdlRL1L9siIS62R9YDgEPYWLbWwFTrzTUGpsZgtnCkpEzPJzuf5o/NcrFI4K6UhPir+eB2U0rcFGJ7prspJQqZCMPiyHoAcHT3vEZvrtKbq/Xmar25Sm+q/mW7xmB+7HC5VCzUqCT+XkqNSqJRSd1VEo1SqlFJ3FVSjUoil4owPo6sB4AWYLHaqvTmat1/pTmf7zoj8+jjhQLKQy3tHOjuoZZ6qKUalVSjkga31xCrzV0pkUvxFw3NmfUcx6WkpOTn50skktTU1KCgIH7/vn37vvzySzc3t7Fjx44fPx7NDW1WnclaqTNV1pr4/1fw27W0rs7y6IPFIoGnWhrgpfFQSz3UMg+11FMt9XCTeqhlaoX40WEWfOYJDsr6rKwshmHS09MvXLiQlpa2YcMGQkh1dfW6dev279+vUqn+8Ic/PPfccwEBAWhxaMU4jtPTlspaU2WtqeKXWOf/X2e2Pvy3J6Q83WSBPipPN6mnWubhJvX8JdmVMgy2gFNmfW5ubkxMDCEkKioqLy+P31lUVNS1a1e1Wk0I6d69u1arRdZDq2G1sRW1pvIauryGLqum+Y3yGtOjsxWlYqGXu6xze3cvN5mXu8z+f3eVBB+EgotlvcFg4DOdECISiViWFQgEISEhN2/erKqqksvlP/zwQ2hoKJobXLGrbjRZH8l0ukpnfujDUalE6Osh93aXebvL+TT3dpd5ucvQSYfWk/UqlcpoNPLbfNATQtzc3BYvXjx37lyNRhMZGenh0eDanj4+apdoLNTZ6us00JaScsPdcsPdckNJufFuuaG00mg0PTz84uUu69bRy99L0c5L6eelbOel8PdSuiklLZjpOOlts05HZ310dPTx48fj4uK0Wm14eDi/02azXb58efv27QzDzJw5880332zo6S7xsZKrfPyFOp+QxWorq6bvV9H3q4ylVfT96rrSqjr9f39SKhEJvDXysPYab43MVyP30cj5nvtDyyISQhiaqaAZnHTU6bRvSE2T9bGxsadPn05MTCSEpKWlZWZm0jSdkJBACImPj5dKpTNmzNBoNPhnFLTUOEy13lxSabxfWWeP9cpaU/1BGIoiPu7yEH83P095O0+Fn6fC31MRFupdWWlAA0IrQHFPspJ/M8NbPepsQizHVelMJRXGkoq6kgpjSaWxpMJoYv7rI1N3lcTf40Gg+3nK/T0VPhr5o1f/ozFRJ/r1AE6B5biKWj7ZH/x3r7Ku/mQYoYDy91S081YGeCn8vRTtPJW+HnJccwRtDX7jwcUYaMvdckNRmaG43FBUZrxbYWAs/7lBqEhI+XsqA7wVAd7KAC9lgLfS10OO5boAkPXg1Kw2trSqrqjcUFxmLC43FJUZqvXm+n32dl7K9j7KAO8H//loZEIBkh0AWQ/Orc5kuVNquF2qLyozFJcZSiqN9e8v6qGWdu/o1d5XGeSjau+r8vdUoM8OgKwHF6CvY/hw//d9/Z37+rIa2v4tiUgQ5Ktq76Nq76viw10lF6PFAJD14BrhXnhPV6EtuXyr4k6pvlL3nzEZpUzULcQj2F8d7Kfu4Kf21chxk2gAZD24BquNLSozFJTobpXUFpToyqr/03N3U4i7d/QK9lcF+7kF+6u83GRYTgAAWQ+ugeO4Sp2poETH5/vt+war7cFUGYVU9EyoZ8cAt6gIP41cpFFJEO4AyHpwGSzLFZcbrhfVXC+quVFcW/vLfTYEFNXeV9kpwL1jgFvHADc/TwW/uCOWXAdA1oNrsNrYf9/TXy9+kO/0L0u0u6skvSN8Oga4dQpwD/ZXS8VCtBUAsh5ciZmx3Sqp5fvvBSU6xvpgcMbXQ947wie8vSa8g8bHHcPuAMh6cDUmxnq9qPbanerrRTW37+ttLEcIoQgJ9FFFBGnCgtzDgzQalRQNBYCsBxdjsbIFJbVXb1dfuV1dWKLj810ooEL81eFBmrAgTVh7d6UMs90BkPXgaliOKyo1XP531dV/V90oruXHZyiKhLZz6xrs0SXYo3OAu1SCwXcAZD24IANtuVxYlVdQeamwSvfL/JlAH2XXYI+uwR4RQR4KGX5PAJD14Jpd+Nv39ZcKKi8VVBaU6Pi7GLgpJc8/4x/Z0bNrsKe7UoJWAkDWg6t24fMKKy/dqrpcWKmrsxBCKIp0CnTv3tGrR0evID+VAPNnAJD14KLKa+ifb1Rob5RfL6plOY4Q4q6UPN/dv3tHr8hQT3zECoCsB1fFcVxxufH8tbKfb1QUlz+4aWqnALeozt49OnkF+aowBR4AWQ8u7G65IedaWc61snuVdYQQkVDQo5NXrzDvnp293TELHgBZDy7tXqUx52rZTzcr7tzXE0IkIkGfCJ8+XXx7dPKSSXCiAZD14MqqdKYfLt8/d7WsqMxACBGLBL3CvPt19YvqjIgHAGS9i6PN1p+ul5/Ju3/tdjVHiFBARXXy6tfVb9izIUa9Ce0DAMh6F8Zy3LXb1acu3fvpejljYQkhYe3dn33Gv28XX346jUImRtYDALLeVVXpTKcu3Tt18V5FrYkQ4quRP/eM/4Bn/H01cjQOACDrXZvVxmpvVHx/seRyYRXHEYlY8EL3djFR7ToHumPSJAA4NOs5jktJScnPz5dIJKmpqUFBQfz+AwcOfPXVV0Kh8NVXX500aRKa+ze5V2k8qS05k3ffQFsIIZ0C3GKiAvp28ZVL8Q4NAC2R9VlZWQzDpKenX7hwIS0tbcOGDfz+Dz744PDhwzKZbOTIkaNGjVKr1WjxX8VyXF5B5dHzxZcLqwghKrl4eN+gmB7tAn1UaBwAaMmsz83NjYmJIYRERUXl5eXZ93fp0qW2tpYfasCAw6+izdZTl+4dyy0uq6YJIeFBmqG92/cK8xYJBWgcAGj5rDcYDPY+u0gkYllWIBAQQsLCwsaNG6dQKGJjY1UqdEsbVFpdd+x88alL90yMTSQUvNCj3bDe7Tv44Z9BAOBMWa9SqYxG44MhiF+CPj8//8SJE9nZ2QqFYsGCBUeOHHnppZce+3QfH9cIteao80ph5e7sG+evlnIc8XKXJQwNf2lA8O9cyaAtt2fbLBJ1ttk6HZ310dHRx48fj4uL02q14eHh/E61Wi2XyyUSCUVRnp6eOp2uoaeXl+td4tw3YZ0cx135d3XmmX/nF9UQQjoFusX2CYoO9xEJBQzNlNOMk9TpKu3ZlotEnW25TkdnfWxs7OnTpxMTEwkhaWlpmZmZNE0nJCRMmDBh8uTJEomkQ4cO8fHx+GcUIYTlOO2Nin/+8O/Ce3pCSPeOXqOeCw5rr0HLAEDzoTj+jkQtqo281XMcl1dYtefErTtlBoqQ6AifUc+GBPurna1OdJ3QmKgT/Xp4SgUlut0nbl67U0MRMqCb38jnQgK9lWgWAHAMZH2zu1dp3Pt9QW5+OSGke0evcQM7YoINACDrW486k3X/qcJjucUsx3UMcEsY1CmigweaBQCQ9a0Ex3Fn8u5/e+KWzsj4auQJgztFh/vgajIAQNa3HndK9duOXr9ZXCsRCeJf7BjXL0gsEqJZAABZ30rQZuve7wuyfyrmONI7wmfikM7e7lhwGACQ9a3IxVsVW47kV+nMfp6KKbFhz4R6oU0AAFnfeujqmPSsG2evlAoF1CvPhYx6LhiDNgCArG89OI47e7n0m2M3DLQltJ3bH0d0ae+LJd4AwKWyvri4+ObNmzExMSUlJfbbj4CdgbZ8ffha7vVyiViQODRsWO/2AgFm2gCAS2X9oUOHPv30U5qmd+7cmZiYuHDhwjFjxqC97PLvVG88eKVabw4P0swc2dUHN30FACfW4E0wNm3a9M0336hUKi8vr4yMjI0bN6KxeFYbu/f7gg92/FxrYOJf7LhwUi8EPQC4ar9eIBDY7y7i6+vLL0kP5TX0xgOXb5XovN1l/zM6snOgO9oEAFw468PCwrZt22a1Wq9evbpjx44uXbqgsbQ3KzYdvEKbrf27+SUNj1DI8Mk2ALiGBnvry5YtKy0tlUqlS5YsUalUy5cvb8vNxHLcjiPX1u2+aLWxM17u+j+vdEPQA0Br6NdLpdKePXsmJydXVVVlZ2crlW13AV4TY9144Ir2ZoW3u2x2fPcmX3EeAKDFsv6dd95hWXbo0KGEkB9//PHixYvvvvtuG2ygKp3p/+2+WFRm6BnmM+PlLiq5GL80ANB6sj4vL+/gwYOEEE9Pzw8//PCVV15pg61TeE+3bs/FWgMzqFfgvEnR1VVG/MYAQKvKepZly8rKfH19CSGVlZVtcB5Obn7ZpoNXLFY2cWhYbJ/2IiFmIgFAq8v6N954Iz4+vnfv3hzHXbx4ccmSJW2nUTiOO3T29p6TBVKxcO74Hj07e+MXBQBaZ9a/8sor/fr102q1IpFo6dKlfAe/jQT9N8duZJ0v9lBL543vgfsFAkBrznqdTpeVlVVTU8Nx3NWrVwkhc+bMaRNBn3UjK7c40FuZnNhTo5LiVwQAWnPWz5s3T61Wh4WFtZ0753EctyPrxrHc4kAf5VuJvdyUEvx+AEArz/qKiorNmze3qbbYc7LgWG5xex/lgkm93BQIegBoPRqcW9K1a9dr1661nYY4cu7OobO3/TzkCxIR9ADQZvr1N27ciI+P9/LykkqlHMdRFHXs2LHW2gqnL93bmX1To5IkJ/bE0A0AtKGsX79+/ZMfheO4lJSU/Px8iUSSmprK39ikoqJi/vz5FEVxHHft2rUFCxZMnDjRCZtAe7Ni86FrSpkoeWJP3AocANpW1vv4+Jw8edJoNBJCbDZbcXHxvHnzGnpwVlYWwzDp6ekXLlxIS0vbsGEDIcTb23vr1q2EEK1Wu3bt2gkTJjjh679eVPPpvjyRiJqXEBXog9sHAkAby/o5c+bQNH3nzp0+ffrk5OT07NmzkaPk5ubGxMQQQqKiovLy8h767nvvvffRRx854XyeojLD/9t9kWW5v4zvgZXoAaAVa/Cz2cLCwi1btsTGxr722mvffvttWVlZI0cxGAxq9YNrjkQiEcuy9m9lZ2eHh4cHBwc72yuv1pv/sUtLm60zR3bt3tELvwoA0Bb79V5eXhRFhYaG5ufnjx07lmGYRo6iUqn40R5CCMuy9RfPOXDgwPTp0xsvwsfH0demmhjr+9tyawzMH0d1e2VQ2BM+y/F1Ph3UicZEnW2kzibI+rCwsPfee2/SpEkLFiwoKyuzWCyNHCU6Ovr48eNxcXFarTY8PLz+t/Ly8nr16tV4EeXleke+ZpbjPt2Xd7O49oUe7V6I9HvCn+7jo3ZwnU/9O4o60Zios43U2QRZn5KS8vPPP3fu3Pkvf/nLmTNn1qxZ08hRYmNjT58+nZiYSAhJS0vLzMykaTohIaGqqso+tuM8DpwqzM0vjwjSTHspou1cFQwAbRnFcdxDuy5fvhwZGZmTk/PQ/r59+zZTEY58C71UULl21wUvd9myP/T9TTceQZcE/XrUiTpbVb8+PT39vffeW7du3X+9J1DUli1bXP2drbLWtOngFaGQ+nP8M7jDFAC0HY/J+vfee48QMmLEiMmTJ7eml2q1sZ/uzzPQlmkvRYT4u+HcA0Db0eCcyx07drSyl7or+2ZBie7ZSL+BPQNw4gGgrffref7+/tOmTYuKipJKH6zh7tLr15+7WsqvSj/tpS74PBYAkPUPNH6hrGu5V2ncfPiaVCz8c/wzUokQZx0AkPXk0V48x3HFxcUu+goZi23DvjwzY/vT6Mh2XkqccgBA1v/Htm3bPvroI5qm+S/bt29/9OhRV3yFe78vuFtuHBwd2L+bH843ALRNDX42++WXX+7fv//ll18+evRoampqjx49XPHlXS+qOZpT5OchnzC4M042ACDrH+bl5RUUFBQREXH9+vVXX321sLDQ5V6bmbF9eegqocjMkd2kYgzTAwCy/hFyufzs2bMRERHHjx8vLy/X6XQu99r2nLxVVk2/1K9D5/ZYrxgAkPWPs3Tp0uzs7JiYmJqamhEjRkydOtW1XlhRmeFYbnE7L0V8TChOMwC0cQ1+Nnv79u233npLIBB8/PHHrvjCdh2/yREyaWiYWITRGwBAv74BBw4cGDp06LJly86fP+9yr+pSQeXlwqrIUM9ncBMSAIBGsn7dunWHDh2Kjo7etGlTXFzc2rVrXeUl2Vh2V/ZNiiITMfcGAIAQ0sgYDiFEpVL17t37/v379+7d02q1rvKSTl28d7fCGNOjXXtf3CscAKDRrP/yyy//+c9/MgwzevTojRs3+vv7u8TrYSy2facKJWJB/IsdcXYBAH4l68vKylauXNm1a1fXej3HfiquNTAjnw3WqKQ4uwAAv5L1ixcvdrkXU2eyHvrhtkIqiuvfAacWAMBO0JpezL9y7hhN1hEDOihluOcUAEBrzHpdHXMkp8hNIR7WOwjnFQCgvseM4ezbt++xDx07dqwzv5Ij5+6YGdu4FztihXoAgF/P+h9//JEQcufOndu3bw8cOFAoFJ46dapz587OnPVmxva9tkStEOP+ggAAT5T1aWlphJCkpKQDBw54enoSQmpra2fPnu3ML+OHK/eNJuuo50KwIgIAwKMaHK8vKyvTaDT8tlwuLy8vd9rXwHHcsfPFQgE1uFcgzigAwBP163mDBg364x//OHz4cJZlv/vuuxEjRjjta7hyu/puhbF/Nz8PNebUAwD8lqx/++23jxw5cu7cOYqiZsyYMXToUKd9DcfOFxNChvVpj9MJAPDbsp4Q4u3t3blz51dfffXixYtO+wLKqusu3KzoGODWKQA3JAEA+I1Z//XXX2dlZZWVlY0YMWLZsmXjx4+fOXNmQw/mOC4lJSU/P18ikaSmpgYFPZjhfvHixdWrV/NvGx9++KFEImnyF3Di5xKOkKG90akHAGhQg5/NZmRkfPHFF3K5XKPR7N69e8+ePY0cJSsri2GY9PT05ORkfhoPb9myZatWrdq+fXtMTExJSUmTV2+x2k5duqeSi/tE+OJcAgD85qwXCAT2brhUKhUKG5vLmJubGxMTQwiJiorKy8vjdxYWFmo0ms2bNyclJdXW1oaEhDR59efzyw20JaZHO7FIgHMJANCQBsdw+vXrt3r1apqms7Kydu7cOWDAgEaOYjAY1Gr1gyOKRCzLCgSC6upqrVa7fPnyoKCgP/3pT88880z//v0f+3QfH/XTVX/qkpYQEj8k3Mdb6YDGeuo6HQx1ojFRZxupswmyfuHChbt27YqIiNi3b9/AgQMTExMbOYpKpTIajfw2H/SEEI1G06FDh9DQUEJITExMXl5eQ1lfXq5/itKLywxX/131TKiniGOf7gi/9dw74KegThSJOlFnc7whNZj1AoFg1KhRAwcO5DiOEFJWVhYQ0ODyA9HR0cePH4+Li9NqteHh4fzOoKCgurq6oqKioKCg3Nzc8ePHN+3rPKG9SwgZhOunAACeOuv/93//d+PGjRqNhqIojuMoijp27FhDD46NjT19+jTf909LS8vMzKRpOiEhITU19c033ySE9OrVa+DAgU1Yt4mxnsm776GWRnXG3cMBAJ4263fv3p2VlcWvh/OrKIpasWKF/Ut+3IYQ0r9//2+//bY56v7xSqmJsb3Ur4NQgE9lAQB+RYNB2a5dO3d3J706ieO44z/fFVDUi1FY1RIA4Hf060NCQiZPnty/f3/7zMs5c+Y4SdGF9/R3Sg29wryxAA4AwO/Kej8/Pz8/P+cs+sTPdwkhg6PxqSwAwO/LeufpxT/EaLKcu1rqo5F1C/HE+QMAeMqsj4+Pz8jI6NKlC0VR/B5+Hs7Vq1edoeIzl+4zVnZQz0DBL+UBAMBvzvqMjAxCyLVr15ywXI7jTl4oEQmp53u0w8kDAHj6rOdVVlYePHjQaDRyHMeybHFx8QcffNDi5d6tMJZUGHuH+7gpJDh5AABPqME5l3PmzLl69eqBAwdoms7OzhY4xzT289fKCCF9umBVSwCApsj66urq1atXDxkyZPjw4Vu3br1x44YzlJtzrUwsEvTohGtlAQCaIuv5C6lCQ0OvXbumVqutVmuL13q3wnivsu6ZUE+5VIQzBwDw5BoMzQEDBvzlL39ZtGjRjBkzLl++LJW2/FVL/ABOXwzgAAA0VdbPnz//zp07gYGBH330UU5OjjNMtz9/rUwkFER19sZpAwD4vVm/b98++/ZPP/1ECNFoNGfOnBk7dmwLFlpSYbxbYewV5o0BHACAJsj6H3/88bEPbdmsP5+PGTgAAE2X9fabg1ut1vz8fKFQGBERQbX0RaraGxVCARXVCQM4AABNkfW8M2fOLFy40NfXl2VZnU63du3aHj16tFSVujrm9n19RAeNQoYBHACApsv6999///PPP+/SpQsh5NKlS8uXL9+7d29LVXmlsIojJDIUi50BADyNBufXSyQSPugJId27d2/ZKvMKqwghz4TiEioAgCbt1/fo0WPJkiUTJkwQCoX//Oc/AwMDc3JyCCF9+/Z1cIksx+UVVrkpJUF+KpwwAICmzPpbt24RQv7+97/b96xbt46iqC1btji4xOIyg87IPBvpj0WMAQCaOOs/++wzhUJh//Lu3buBgS1zH6gHAzgdMVgPAPCUGhyvj4+P12q1/PaOHTsmTpzYUiXmFVRS+GAWAKA5+vWpqalvv/32kCFDrly5IpPJdu3a1SL1mRjrjeLaDv5qLFgPAND0/fo+ffpMnTp1x44dN2/enD17dkBAQIvUd+12jY3lumMABwCgOfr1U6dOFQqFBw8evHv3bnJy8uDBgxcvXuz4+m7erSWEdO3ggVMFAND0/fqXXnrp66+/bt++ff/+/ffu3Ws2m1ukvnuVRkJIoA9mWwIANEO/PikpKTc39/r16+PGjbty5cry5csbOQrHcSkpKfn5+RKJJDU1NSgoiN//1Vdf7d6929PTkxDy7rvvhoSE/Nb6SiqMKrlYrRDjVAEANH3Wf/3111lZWWVlZXFxccuWLRs/fvzMmTMbenBWVhbDMOnp6RcuXEhLS9uwYQO///Llyx988EG3bt2erjiLlS2roTsHulOYWQ8A8Ds0OIaTkZHxxRdfyOVyDw+P3bt379mzp5Gj5ObmxsTEEEKioqLy8vLs+y9fvvzZZ59Nnjx548aNT1FcaVUdx5EAbyXOEwBAs/TrBQKBRPJgmqNUKhUKhY0cxWAwqNXqB0cUiViWFQgEhJCRI0dOmTJFpVLNnj375MmTAwcOfOzTfXzUj91/rVhHCAkP9mzoAQ7mJGWgThSJOlFnk2V9v379Vq9eTdN0VlbWzp07BwwY0MhRVCqV0Wjkt+1BTwiZPn26SqUihAwcOPDKlSsNZX15uf7xWV9YQQhRy4QNPcDB594ZykCdKBJ1os6neENqcAxn4cKFwcHBERER+/btGzhw4KJFixo5SnR09MmTJwkhWq02PDzc3tkfNWoUTdMcx509ezYyMvK3vpKSyjpCSIAXxnAAAJqnXy8QCBITExMTE5/kKLGxsadPn+YfnJaWlpmZSdN0QkLCm2++mZSUJJVKn6feaeUAABf1SURBVH322RdffPG3Fnev0iiTCD3UUpwnAIBmyfrfhKKoFStW2L8MDQ3lN0aPHj169OinO6aNZe9X1nXwU2MSDgDA7yRw2srKa0w2lgvwUuAkAQA0Y9YXFxefOHHCZrMVFRU5vrKSCiMhmHAJANCcWX/o0KFZs2atXLmypqYmMTFx//79LZL17fDBLABA82X9pk2bvvnmG5VK5eXllZGR8XQXQ/0e/Eo4Ad4YwwEAaLasFwgE/NR4Qoivr699yrwD+/V1YpHA212OkwQA8Ds1OA8nLCxs27ZtVqv16tWrO3bs6NKliyPLYjnuXpXR31MhEGASDgBAs/Xrly1bVlpaKpVK//a3v6lUqsbXuWxyVbUmxsLig1kAgObt1+/atWv69OnJycktUta9qjpCSDtPDNYDADRnv760tHTChAkzZ87cv38/TdMOLqusmiaE+HpgsB4AoDmzftGiRdnZ2bNmzbpw4cLYsWPfeustR5ZVXkMTQnyQ9QAAzZr1hBCO4ywWi8VioSjKvr6xI7PeV4OsBwBoAg2O17/33ntZWVldu3YdPXr0O++8I5U6dAGyshpaJhGq5Lj1IABAc2Z9SEhIRkYGf6tYB+M4rryG9vdQYNUzAIDmyvqdO3dOnDixtrZ2x44d9ffPmTPHMTXVGhnGwmKwHgCgqQge261u2ZowWA8A0Oz9ev6WI4GBgfHx8fad27dvd1hN/IRLH2Q9AEDzZf1XX31lMBjS09Pv3r3L77HZbAcPHpwyZYoj+/UYwwEAaCqPGcMJDg5+aI9EIlm1apXj+vUYwwEAaO5+/eDBgwcPHjxixIhOnTrZd5pMJofVVF5DCwWUpxtuMwsA0GxZz7t58+b8+fPr6uo4jmNZlqbps2fPOijrq2kvN5lQIMDpAQBo3qz/8MMPV65cuXnz5jfeeOPUqVPV1dWOKYg2W3V1liA/Nc4NAEBTabDv7ObmNmDAgKioKL1eP3fuXK1W66BOPQbrAQAclvUymaywsLBTp07nzp1jGEav1zsq600EEy4BAByT9X/961/Xrl07ePDgH3744fnnnx82bJgj+/XIegCAJtTgeH2/fv369etHCNmzZ09tba27u7tjCnow4RKT6wEAmjXrk5KSHrvo2JYtWxzRr6+uI4R4u8twbgAAmjHr586d+1uPwnFcSkpKfn6+RCJJTU0NCgqq/91ly5ZpNJo333zzibK+xuSmEMulIpwbAICm8pjxen70hnpEI0fJyspiGCY9PT05OTktLa3+t9LT069fv/6E1dhYtlJnwuoIAADN3q/nrVu3jt+wWq35+fl9+vTp27dvQw/Ozc2NiYkhhERFReXl5dn3//zzz5cuXUpMTCwoKHiSamr0jI3lvN2R9QAADsn6rVu32reLiooe6q0/xGAwqNUPrn4SiUQsywoEgvLy8vXr12/YsOHQoUONF+Hj8+C5ZXqGENLeT23f41ScsyrUiSJRZ5utswmyvr6goKDGO+YqlcpoNPLbfNATQr777ruamprXX3+9vLzcbDZ37Nhx7Nixj316efmDyfsFRVWEEJlIYN/jVOfeCatCnSgSdbblOpsg699++2379q1bt8LDwxs5SnR09PHjx+Pi4rRarf2RSUlJSUlJhJCMjIzCwsKGgr6+Kp2ZEOLlhkk4AAAO6dfzk+sJIRRFxcXFPfvss40cJTY29vTp0/xNTtLS0jIzM2maTkhI+K3VVOlMhBCscAkA4KCsj4+PNxgMOp2O/7KioiIgIKChB1MUtWLFCvuXoaGhDx3qibPeTAjxRL8eAMAxWb969epdu3ZpNBpCCMdxFEUdO3asuaup0pkkYoFShsn1AAAOyfpjx459//33SqXSkdVU6kxebrLG5/IDAMBv1eDaZxEREQzDOLIUM2MzmqyeagzWAwA4ql8/ZsyY4cOHh4eHC4VCfk9zr4dTpec/mMVgPQCAo7L+/fffX7JkSSOfxzZ91uODWQAAB2e9Wq1+khnxTagSEy4BAByc9b179547d+6LL74oFov5Pc0d/b9Mrke/HgDAUVlP07RKpfrpp5/se5o/63HRLACAY7O+8cXOmiXr9SZCiAfm4QAAOCzrhwwZ8tA89+a+lqpSZ1bJxVKxEGcFAMBBWW9f09hqtR49erS559pzHFetM/l7KXBKAACaXIPXUgX+Ijg4+LXXXsvKymrWOgy0hbGyGKwHAHBovz4nJ8fe475x44bZbG7WOh5Mrlcj6wEAHJj19nsQUhTl4eGxatWqZs56EyHE0x0fzAIAODDrt27dWllZ6eXlRdN0WVlZcHBw82a9Hv16AIDmImgk61977TVCSFVV1RtvvLFz585mrYO/aBbj9QAADs36nTt3bt++nRASGBi4d+/ebdu2NW+/HgskAAA4PustFotEIuG37cskNGfWmwUU5a6S4JQAADS5Bsfrhw0bNn369BEjRhBC/vWvfw0dOrRZ69AZGbVSLBQIcEoAAByX9W+99dZ3332Xk5MjEommTZs2bNiwZq2jzmxVK8Q4HwAADs16QkhcXFxcXJxj6jAxVh+NHOcDAKA5OMWYicXKWm2cQoqVcAAAWm/W02YrIUQmFeF8AAC03qxnrIQQObIeAKDV9+vlEmQ9AEBrznobIUSO8XoAgObRNF1pjuNSUlLy8/MlEklqampQUBC//8iRI5s2bRIIBKNGjZo2bVpDTzeZMYYDAOD0/fqsrCyGYdLT05OTk+03L2RZ9qOPPvr666/T09N37NhRU1PT0NPrkPUAAM7fr8/NzY2JiSGEREVF5eXlPXgbEQgOHz4sEAgqKys5jmtkoQUTY0PWAwA4e7/eYDCo1eoH7x4iEcuy9rg/evTomDFj+vXrp1AofqVfL8F4PQCAE/frVSqV0Wjkt1mWFdRb1iY2NjY2NnbRokX79u2Lj49//BuOUEAIaefn5uOjdubGcvLyUCcaE3W2tTodnfXR0dHHjx+Pi4vTarXh4eH2zv6sWbO++OILiUQil8spimro6ZU1NCHERDPl5XpnPvfOXB7qRGOizjZYp6OzPjY29vTp04mJiYSQtLS0zMxMmqYTEhJGjx49depUsVgcERExZsyYhp5OYwwHAMD5+/UURa1YscL+ZWhoKL+RkJCQkJDwq0+nMQ8HAKA5Oct1sxQhUvTrAQBaddbbZFKhoOEBfQAAcPmsNzFWDOAAALT6fr0VC58BALTmrOc4jjbb0K8HAGjNWW+22FiOk2GRSwCAVpz1dSYsXg8A0Pqz3kIwuR4AoE306zGGAwCAfj0AALh+vx7j9QAArb5fj3k4AACtv1+vwBgOAEArznqjyUoIkSHrAQBadb/egn49AEArz3p+8XoZFjQGAGjFWW+kMecSAKC1Z30dbkoFAND6s562CChKIhLgZAAAtOZ+vVwqpHBTKgCA1pz1JtyUCgCg9We9RYYFEgAAWnfW02arAgskAAC07qznOFw0CwDQ2rOe4KJZAIC2kPXo1wMANKumCVmO41JSUvLz8yUSSWpqalBQEL8/MzNzy5YtIpEoPDw8JSWloafLsUACAIDz9+uzsrIYhklPT09OTk5LS+N3ms3mdevWbdu2bceOHXq9/vjx4w1mPfr1AADOn/W5ubkxMTGEkKioqLy8PH6nRCJJT0+XSCSEEKvVKpVKkfUAAC6c9QaDQa1W89sikYhlWUIIRVGenp6EkK1bt9I0/dxzzzWc9RjDAQBoRk3ToVapVEajkd9mWVYgePAWwnHcBx98cPv27fXr1zfydD8ftY+P2vkbyyWKRJ1oTNTZdup0dNZHR0cfP348Li5Oq9WGh4fb9y9dulQmk23YsKHxpzMmS3m53vnPvfMXiTrRmKizTdXp6KyPjY09ffp0YmIiISQtLS0zM5Om6cjIyL179/bu3TspKYmiqGnTpg0bNuyxT8f8egAAF+jXUxS1YsUK+5ehoaH8xpUrV57k6RivBwBoVi1/LdW4wZ19NHKcCQCA1pz1fxgVicXrAQBaedYDAACyHgAAkPUAAICsBwAAZD0AALIeAACQ9QAAgKwHAABkPQAAIOsBAABZDwAAyHoAAEDWAwAg6wEAAFkPAADIegAAQNYDAACyHgAAkPUAAICsBwAAZD0AALIeAACQ9QAAgKwHAIDWlPUcxy1fvjwxMXHatGlFRUX1v0XT9KRJkwoLC9HWAACunfVZWVkMw6SnpycnJ6elpdn35+XlTZ069aH0BwAAl8z63NzcmJgYQkhUVFReXp59v8Vi2bBhQ8eOHdHQAAAtSNQkRzEYDGq1+sERRSKWZQUCASGkV69ehBCO49DQAAAun/UqlcpoNPLb9qB/cj4+apdoLNTZButEY6LO1qFpxnCio6NPnjxJCNFqteHh4XgLBQBohf362NjY06dPJyYmEkLS0tIyMzNpmk5ISOC/S1EUGhoAoAVRGEwHAGj1cC0VAACyHgAAkPUAAICsBwCAtp31jayi4wysVuvChQunTJkyYcKE7OzsO3fuTJ48eerUqStWrHC2UisrKwcNGlRYWOjMRW7cuDExMXHcuHF79uxx2jqtVmtycnJiYuLUqVOdtj0vXLiQlJRECHm0vF27do0bNy4xMfHEiRPOU+fVq1enTJkybdq01157raqqymnr5B08eJCfUuhUddqLrKqq+vOf/5yUlDR58mQ+Np+0SK7l/Otf/1q8eDHHcVqtdtasWZyT2bNnz/vvv89xXG1t7aBBg954442cnByO45YtW3b06FHnqdNiscyePfull14qKChw2iJ//PHHN954g+M4o9H48ccfO22dWVlZf/3rXzmOO3369Ny5c52wzk2bNo0aNWrixIkcxz1UXnl5+ahRoywWi16vHzVqFMMwTlLn1KlTr127xnFcenr6qlWrnLZOjuMuX748ffp0/kvnqbN+kYsXLz58+DDHcWfPnj1x4sSTF9mS/fqGVtFxEiNGjJg3bx4hxGazCYXCK1eu9OnThxDy4osv/vDDD85T5+rVqydNmuTr68txnNMWeerUqfDw8D//+c+zZs0aNGiQ09YZEhJis9k4jtPr9SKRyAnrDA4O/uSTT/jty5cv28s7c+bMxYsXe/fuLRKJVCpVSEhIfn6+k9T5j3/8IyIigv9nk0Qicdo6q6ur165du2TJEv5L56mzfpE//fTT/fv3//jHP2ZmZvbv3//Ji2zJrH90FR2nynq5XK5QKAwGw7x58+bPn2+/EEGpVOr1eicpcu/evV5eXs8//zxfnr0NnapI/q8oLy9v3bp1KSkpCxYscNo6lUplcXFxXFzcsmXLkpKSnPCkx8bGCoVC+z/K7eUZDAaj0Wj/g1IoFC1bcP06vb29+ZDasWPHH/7wh/p/+M5TJ8uy77zzzuLFi+Vy+aMB1bJ11m/Mu3fvajSazZs3+/v7b9y48cmLbMms/52r6DjAvXv3pk+fHh8fP3LkSHt5RqPRzc3NebL+9OnTSUlJ+fn5ixYtqq6udsIiCSEajSYmJkYkEoWGhkqlUoPB4Jx1fvXVVzExMUeOHDlw4MCiRYssFotz1vngr/e/fydVKpXTNuyhQ4dWrFixceNGDw8P56zz8uXLd+7cSUlJSU5OvnnzZlpamnPWqdFoBg8eTAgZMmRIXl6eWq1+wiJbMl6dfBWdioqKmTNnvvXWW/Hx8YSQrl275uTkEEK+//773r17O0mR27Zt27p169atW7t06fLBBx/ExMQ4YZGEkN69e//f//0fIaS0tJSm6QEDBpw7d84J63R3d1epVIQQtVpttVq7devmnHXyunXrVv90d+/ePTc3l2EYvV5fUFAQFhbmJHXu379/+/btW7duDQwMJIT06NHD2erkOK579+4HDx7csmXLRx991Llz57ffftsJ6+T/lPjYzMnJCQsLe/KTLmrBoh9aRcfZ/pA+++wznU63YcOGTz75hKKoJUuWrFy50mKxdOrUKS4ujjilRYsWLV261AmLHDRo0Pnz58ePH89xXEpKSmBg4DvvvOOEdU6fPv1vf/vblClTrFbrggULIiMjnbPOx55uiqL46Rkcx7355psSicQZimRZ9v333w8ICJg9ezZFUf369ZszZ46z1fnYNbu8vb2dsD0XLVr0zjvvfPPNN2q1es2aNWq1+gmLxHo4AACtH66lAgBA1gMAALIeAACQ9QAAgKwHAABkPQAAIOuhjUpKSuIvI/r9DAbDuHHj4uPjb9++7fwv/OOPP87NzcUvACDrAX6bq1evSiSSjIyM4OBg56/23LlzzrZyFLgEXEsFLRNYn332mUwmu3XrVkRExJo1a0pLS5OSkrKzswkh69evJ4TMmTPnhRdeGDx48Pnz5318fCZPnrx169bS0tJVq1b16dMnKSnJ19e3sLCQELJ48eJ+/frV1dW9++67N27cYFn29ddff/nllzMyMjIyMmpqagYPHjx//nz+R1dWVi5ZsqSkpEQkEs2fPz8yMjIxMbGiomLAgAEbNmzgH8MwzIoVK3Jzc8Vi8axZs15++WWtVvv+++8zDOPh4fHuu+8GBQUlJSV169btzJkzDMMsWbJk69att27dmj59+vTp09evX19YWFhUVFRbWzthwoSZM2dyHJeamnr27FmKokaPHv36668/2gIikWjfvn1btmzhOC4yMnLZsmUSieSFF16Ii4vLzc0ViURr167NyclZsWKFr6/v+vXrT506tW/fPqFQ2L17dye8XQE4HQ6gJZaz79WrV2lpKcdx48ePP378eHFx8ZAhQ/jvfvzxxx9//DHHcREREdnZ2RzHJSUlJScncxyXkZExZ84cfkn0pUuXchx37dq1gQMHMgzz97//fevWrfxyxKNGjSoqKtq7d+/w4cNZlq3/o+fNm7d582aO4+7cufPCCy9UVlb++OOP/JKWdp9//jm/sim/ODjDMIMHD87Ly+M47vDhw+PGjeMLSEtL46sdPny42Wy+e/du3759+T2jR4+maVqv18fGxl65cmX79u182TRNjx8//sSJE/YWYFmWb4EbN25MnjzZbDZzHLdmzZpPP/2Ub4Fjx45xHLdq1apVq1bxPzcnJ8dqtQ4YMMBqtbIsm5KSwrckQCNEeLeDFhEeHu7r60sI6dSpU01NTUMP4+9wEBgYyC89FhAQUFtby39r/PjxhJCIiAhPT89bt26dOXPGbDbv3r2bEGIymW7evEkIiYyMfGipk7Nnz65cuZIQEhQU1LNnzwsXLiiVyod+aE5OzsSJEwkh3t7eBw8evHHjhkajiYyMJITExcUtX76cX1nwxRdf5GuLioqSSCQBAQH2FWVHjhwpk8kIIUOHDv3hhx+0Wi2/gp5MJnvllVfOnj07ePDgh1rg7t27t2/f5u9HYbVa+R9HCHnhhRcIIWFhYefPn7f3z4RCYXR09Lhx44YOHTplyhT+OACNQNZDy7Av0sRnMUX9ZzjRYrGIxeIHv6Ai0UMb//nd/WUPx3FisZhl2Q8//LBr1678QI27u/vBgwelUumj/5C1b7Msa7PZHvNXUe9n3blzh/+XQf0j8CPm9iLta4s/egSbzfZQ5XyUP9oCNpttxIgR/I0yaJrmC6Moin9Y/fbhffLJJxcuXPj+++9nzpy5Zs0a/qYlAA3BZ7PgFNzc3HQ6XXV1NcMw/OrHv+rgwYOEkEuXLhmNxpCQkAEDBuzYsYMQUlZWNnr06Hv37j32WQMGDOD7/kVFRT///HPPnj0ffUyfPn0OHz7Mv2ckJSUFBgbW1tbyt047dOhQQEBAQ6uE2+P46NGjFoultrb2xIkTzz//fP/+/fft28eyLE3TBw8e7N+//6PP7devX1ZWVlVVFX8f5q+++uqhdyb7u4jVaq2qqhoxYkR4ePjcuXOff/75lr21E6BfD/CkVCrVjBkzxo0bFxAQEBUVVb/D+1gURRmNxvj4eKFQuGbNGqFQOHv27BUrVrzyyissyy5cuDAoKMg+6FHfkiVLli1btmfPHoFAkJqa6u3tXVBQ8NBjJk+evHLlytGjR1MUtXTpUpVK9Y9//OPdd9+laVqj0axdu7ah2uw7ZTLZ5MmTjUbjn/70p06dOgUHBxcWFo4ZM8ZqtY4ZM2bYsGH8svj1denSZfbs2dOnT+c4rmvXrv/zP//z2J8SExOTkpKyevVq/kbtcrk8ICCAHyACaATm4QA0Mfs8IjQFOA+M4QAAoF8PAADo1wMAALIeAACQ9QAAgKwHAABkPQAA/Lr/D6CR+gyOtHu5AAAAAElFTkSuQmCC" />

We see that these 150 components account for just over 90% of the variance. That would lead us to believe that using these 150 components, we would recover most of the essential characteristics of the data. To make this more concrete, we can compare the input images with the images reconstructed from these 150 components (Figure 5-93):

In [ ]:
# Compute the components and projected faces
pca = RandomizedPCA(150).fit(faces.data)
components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

In [ ]:
%matplotlib notebook
# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap='binary_r')
ax[0, 0].set_ylabel('full-dim
input')
ax[1, 0].set_ylabel('150-dim
reconstruction');

> **Figure 5-93: _150-dimensional PCA reconstruction of the LFW data_**<br><img width="500" src="data:image/png;base64, ..." />

The top row here shows the input images, while the bottom row shows the reconstruction of the images from just 150 of the ~3,000 initial features. This visualization makes clear why the PCA feature selection used in “In-Depth: Support Vector Machines” on page 405 was so successful: although it reduces the dimensionality of the data by nearly a factor of 20, the projected images contain enough information that we might, by eye, recognize the individuals in the image. What this means is that our classification algorithm needs to be trained on 150-dimensional data rather than 3,000-dimensional data, which depending on the particular algorithm we choose, can lead to a much more efficient classification.

# Principal Component Analysis Summary

In this section we have discussed the use of principal component analysis for dimensionality reduction, for visualization of high-dimensional data, for noise filtering, and for feature selection within high-dimensional data. Because of the versatility and interpretability of PCA, it has been shown to be effective in a wide variety of contexts and disciplines. Given any high-dimensional dataset, I tend to start with PCA in order to visualize the relationship between points (as we did with the digits), to understand the main variance in the data (as we did with the eigenfaces), and to understand the intrinsic dimensionality (by plotting the explained variance ratio). Certainly PCA is not useful for every high-dimensional dataset, but it offers a straightforward and efficient path to gaining insight into high-dimensional data.

PCA’s main weakness is that it tends to be highly affected by outliers in the data. For this reason, many robust variants of PCA have been developed, many of which act to iteratively discard data points that are poorly described by the initial components. Scikit-Learn contains a couple interesting variants on PCA, including RandomizedPCA and SparsePCA, both also in the sklearn.decomposition submodule. Randomi zedPCA, which we saw earlier, uses a nondeterministic method to quickly approximate the first few principal components in very high-dimensional data, while SparsePCA introduces a regularization term (see “In Depth: Linear Regression” on page 390) that serves to enforce sparsity of the components.

In the following sections, we will look at other unsupervised learning methods that build on some of the ideas of PCA.